<a href="https://colab.research.google.com/github/ronyates47/Gedcom-Utils/blob/main/Gold__1_%26_2_%26_3_20251108_1200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PIP

In [ ]:
!pip install pandas
!pip install python-gedcom
!pip install openpyxl
!pip install xlsxwriter
!pip install mlxtend
!pip caas_jupyter_tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 12.2 MB/s eta 0:00:00
ERROR: unknown command "caas_jupyter_tools"


# Ron Rules-QUICK CODE CARD (v2025.10.27-Refined)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - Punctuation in strings use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# - Deliver Python code (inline, executable, CUT-ready section)
# - XHTML 1.0 Transitional; old-school friendly; Times New Roman body.
# - Use CUT markers; five # spacer lines follow the STOP marker.

# Style Sheet

In [ ]:
# ====== CUT START [1/1] UPDATE /partials/dna_tree_styles.css — SITE-WIDE SANS ==================
# RON RULES — QUICK CODE CARD (v2025.11.06-SharedCSS-SiteWideSans)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - Purpose: Switch site-wide body typeface to a sans stack in /partials/dna_tree_styles.css
# - Creates a timestamped .bak on the server before overwrite.

import os, re, socket, time, traceback
from ftplib import FTP_TLS

# ---------- Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_HOST = os.environ.get('FTP_HOST','')
FTP_USER = os.environ.get('FTP_USER','')
FTP_PASS = os.environ.get('FTP_PASS','')
FTP_PORT = int(os.environ.get('FTP_PORT','21'))
FTP_DIR  = (os.environ.get('FTP_DIR','') or '').strip('/')

TARGET_DIR  = 'partials'
TARGET_NAME = 'dna_tree_styles.css'
LOCAL_ORIG  = 'dna_tree_styles.orig.css'
LOCAL_NEW   = 'dna_tree_styles.css'

if not all([FTP_HOST, FTP_USER, FTP_PASS]):
    raise SystemExit("[EXIT] Missing FTP creds in Colab userdata: FTP_HOST/FTP_USER/FTP_PASS")

# ---------- Helpers ----------
def _connect():
    socket.setdefaulttimeout(30)
    ftps = FTP_TLS(timeout=30)
    ftps.connect(FTP_HOST, FTP_PORT)
    ftps.login(FTP_USER, FTP_PASS)
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(True)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split('/') if p]:
            try: ftps.cwd(p)
            except Exception:
                try: ftps.mkd(p)
                except Exception: pass
                ftps.cwd(p)
    return ftps

def _cwd(ftps, path):
    for p in [q for q in path.split('/') if q]:
        try:
            ftps.cwd(p)
        except Exception:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)

def _download(ftps, remote_name, local_name):
    with open(local_name, 'wb') as f:
        ftps.retrbinary('RETR ' + remote_name, f.write)
    print("[OK] Downloaded:", remote_name, "->", os.path.abspath(local_name))

def _upload(ftps, local_name, remote_name):
    with open(local_name, 'rb') as f:
        ftps.storbinary('STOR ' + remote_name, f)
    print("[OK] Uploaded:", os.path.abspath(local_name), "->", remote_name)

def _server_backup(ftps, remote_name):
    ts = time.strftime("%Y%m%d%H%M")
    bak = remote_name + "." + ts + ".bak"
    try:
        with open("__tmp_bak.css", "wb") as tmp:
            ftps.retrbinary('RETR ' + remote_name, tmp.write)
        with open("__tmp_bak.css", "rb") as tmp:
            ftps.storbinary('STOR ' + bak, tmp)
        try: os.remove("__tmp_bak.css")
        except Exception: pass
        print("[OK] Server backup created:", bak)
    except Exception as e:
        print("[WARN] Could not create server backup:", e)

def _set_body_sans(css_text):
    # Switch the body stack to sans-serif; add a body rule if missing
    if re.search(r'body\s*\{[^}]*font-family\s*:', css_text, flags=re.I|re.S):
        return re.sub(
            r'(body\s*\{[^}]*?)font-family\s*:\s*[^;]+;',
            r"\1font-family: Arial, Helvetica, sans-serif;",
            css_text, flags=re.I|re.S
        )
    else:
        return "body { font-family: Arial, Helvetica, sans-serif; }\n\n" + css_text

# ---------- Main ----------
try:
    ftps = _connect()
    _cwd(ftps, TARGET_DIR)

    # 1) Pull current css
    _download(ftps, TARGET_NAME, LOCAL_ORIG)

    # 2) Transform body font-family to sans stack
    with open(LOCAL_ORIG, 'r', encoding='utf-8', errors='ignore') as f:
        css = f.read()
    new_css = _set_body_sans(css)

    # 3) Save ISO-8859-15 safe
    with open(LOCAL_NEW, 'w', encoding='iso-8859-15', errors='xmlcharrefreplace') as f:
        f.write(new_css)
    print("[OK] Prepared modified CSS locally:", os.path.abspath(LOCAL_NEW))

    # 4) Server backup + upload
    _server_backup(ftps, TARGET_NAME)
    _upload(ftps, LOCAL_NEW, TARGET_NAME)

    try: ftps.quit()
    except Exception: pass

    print("\nDONE. Site-wide body font set to Arial, Helvetica, sans-serif.")
    print("If a page still shows serif, it likely has later inline <style> forcing Times New Roman.")
    print("Add ?v=1 to the page URL or hard-refresh to bust cache.")
except SystemExit as e:
    print(str(e))
except Exception as e:
    print("[ERROR] CSS update failed:", e)
    traceback.print_exc()
# ====== CUT STOP  [1/1] UPDATE /partials/dna_tree_styles.css — SITE-WIDE SANS ==================


[OK] Downloaded: dna_tree_styles.css -> /content/dna_tree_styles.orig.css
[OK] Prepared modified CSS locally: /content/dna_tree_styles.css
[OK] Server backup created: dna_tree_styles.css.202511062314.bak
[OK] Uploaded: /content/dna_tree_styles.css -> dna_tree_styles.css

DONE. Site-wide body font set to Arial, Helvetica, sans-serif.
If a page still shows serif, it likely has later inline <style> forcing Times New Roman.
Add ?v=1 to the page URL or hard-refresh to bust cache.


In [ ]:
# ====== CUT START [1/1] CENTERLINE FIX — Update shared CSS + Upload =============================
# RON RULES — QUICK CODE CARD (v2025.11.06-CSS-Centerline-Fix)
# - Complete & runnable in Colab; ISO-8859-15 safe in source.
# - Purpose: ensure "Last updated …" (meta) is centered site-wide via shared stylesheet.
# - Actions: edit/insert CENTERLINE FIX block in dna_tree_styles.css and FTPS-upload to /partials/.

import os, re, socket, posixpath, traceback
from ftplib import FTP_TLS
from datetime import datetime

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR',  '')
    os.environ.setdefault('FTP_PORT', '21')

FTP_DIR   = (os.environ.get("FTP_DIR","") or "").strip()
LOCAL_CSS = "dna_tree_styles.css"
REMOTE_CSS = posixpath.join("partials", "dna_tree_styles.css")

# ---------- 1) Ensure local css exists (create if missing) ----------
if not os.path.exists(LOCAL_CSS):
    with open(LOCAL_CSS, "w", encoding="utf-8") as f:
        f.write("/* yates.one-name.net shared stylesheet */\n")
    print("[INFO] Created new local stylesheet:", os.path.abspath(LOCAL_CSS))
else:
    print("[OK] Using existing local stylesheet:", os.path.abspath(LOCAL_CSS))

# ---------- 2) Insert/replace CENTERLINE FIX block ----------
with open(LOCAL_CSS, "r", encoding="utf-8", errors="replace") as f:
    css = f.read()

start_tag = "/* === CENTERLINE FIX (do not remove) START === */"
end_tag   = "/* === CENTERLINE FIX (do not remove) END === */"
block_re  = re.compile(re.escape(start_tag) + r".*?" + re.escape(end_tag), flags=re.S)

center_block = f"""{start_tag}
:root {{
  /* nothing here yet; reserved for future tokens */
}}
/* Utility: hard center text, wins against generic .meta rules */
.centerline {{ text-align: center !important; }}

/* Meta display: keep centered across pages, even if other CSS sets .meta {{text-align:left}} */
.wrap .meta, .intro .meta, .updated, .updated.centerline {{
  text-align: center !important;
}}

/* Table meta stamping alignment on partials and main pages */
#last-updated, #auto-count, #showing-count {{ /* inline metrics */ }}
{end_tag}
"""

if block_re.search(css):
    css = block_re.sub(center_block, css)
    action = "[OK] Replaced existing CENTERLINE FIX block."
else:
    # Append with a divider so it overrides earlier rules
    css += "\n\n/* ------------------------------------------------------------------ */\n" + center_block + "\n"
    action = "[OK] Appended CENTERLINE FIX block."

with open(LOCAL_CSS, "w", encoding="utf-8") as f:
    f.write(css)

print(action)

# ---------- 3) Upload via FTPS ----------
def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=30)
    socket.setdefaulttimeout(30)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT','21')))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(True)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split("/") if p]:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps

def ensure_remote_dirs(ftps: FTP_TLS, path: str):
    parts = [p for p in path.split("/")[:-1] if p]
    for seg in parts:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)

if all(os.environ.get(k) for k in ("FTP_HOST","FTP_USER","FTP_PASS")):
    try:
        ftps = ftp_connect()
        ensure_remote_dirs(ftps, REMOTE_CSS)
        with open(LOCAL_CSS, "rb") as fh:
            ftps.storbinary(f"STOR {REMOTE_CSS}", fh)
        try: sz = ftps.size(REMOTE_CSS)
        except Exception: sz = None
        try: ftps.quit()
        except Exception: pass
        print(f"[PUT] {LOCAL_CSS} -> /{REMOTE_CSS}  (size: {sz if sz is not None else 'unknown'})")
        print("Cache-bust tip: add ?v=" + datetime.utcnow().strftime("%Y%m%d%H%M%S") + " once if needed.")
    except Exception as e:
        print("[FAIL] FTPS upload:", e)
        traceback.print_exc()
else:
    print("[SKIP] Missing FTP creds; upload not attempted.")

print("\nNext step:")
print("1) In Cell 3 template, ensure the meta block uses class 'centerline':")
print("   <div class=\"meta centerline\">Last updated: <span id=\"last-updated\"></span> ...</div>")
print("   (Cell 2 already uses 'updated centerline'.)")
print("2) Re-run Cell 3 to regenerate HTML, or refresh the page with ?v=1.")
# ====== CUT STOP  [1/1] CENTERLINE FIX — Update shared CSS + Upload =============================



[OK] Using existing local stylesheet: /content/dna_tree_styles.css
[OK] Appended CENTERLINE FIX block.
[FAIL] FTPS upload: 553 Can't open that file: No such file or directory

Next step:
1) In Cell 3 template, ensure the meta block uses class 'centerline':
   <div class="meta centerline">Last updated: <span id="last-updated"></span> ...</div>
   (Cell 2 already uses 'updated centerline'.)
2) Re-run Cell 3 to regenerate HTML, or refresh the page with ?v=1.


Traceback (most recent call last):
  File "/tmp/ipython-input-992470288.py", line 109, in <cell line: 0>
    ftps.storbinary(f"STOR {REMOTE_CSS}", fh)
  File "/usr/lib/python3.12/ftplib.py", line 495, in storbinary
    with self.transfercmd(cmd, rest) as conn:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 393, in transfercmd
    return self.ntransfercmd(cmd, rest)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 771, in ntransfercmd
    conn, size = super().ntransfercmd(cmd, rest)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 359, in ntransfercmd
    resp = self.sendcmd(cmd)
           ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 281, in sendcmd
    return self.getresp()
           ^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ftplib.py", line 254, in getresp
    raise error_perm(resp)
ftplib.error_perm: 553 Can't open that file: No such file or dire

In [ ]:
# ====== CUT START [1/1] CELL — iOS Font/Spacing CSS Fix + Cache-Busted Reupload =================
# RON RULES — QUICK CODE CARD (v2025.11.07-iOS-CSS-Fix)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - No snippets; no fabrication; explicit prints.
# - Actions:
#   (1) Download /partials/dna_tree_styles.css, append a minimal iOS fix block (safe, idempotent).
#   (2) Upload patched css back to /partials/dna_tree_styles.css.
#   (3) For selected HTML pages in /partials/, download, inject/refresh '?v=TIMESTAMP' on
#       '/partials/dna_tree_styles.css' link, and upload back (cache-bust so iPhone fetches new CSS).
# - This cell does NOT change any other content; only the CSS and the cache-buster query on the CSS link.

import os, re, io, posixpath, socket, traceback, datetime
from ftplib import FTP_TLS

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR',  '')
    os.environ.setdefault('FTP_PORT', '21')

FTP_DIR     = (os.environ.get("FTP_DIR","") or "").strip()
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))
FTP_PASSIVE = True

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        parts = [p for p in FTP_DIR.split("/") if p]
        for p in parts:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ensure_remote_dirs(ftps: FTP_TLS, remote_path: str):
    if "/" not in remote_path: return
    pwd0 = ftps.pwd()
    for seg in [p for p in remote_path.split("/")[:-1] if p and p != "."]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)
    ftps.cwd(pwd0)

def ftp_download(ftps: FTP_TLS, remote_name: str) -> bytes:
    buf = io.BytesIO()
    ftps.retrbinary(f"RETR {remote_name}", buf.write)
    return buf.getvalue()

def ftp_download_if_exists(ftps: FTP_TLS, remote_name: str) -> bytes:
    try:
        return ftp_download(ftps, remote_name)
    except Exception as e:
        raise FileNotFoundError(f"Missing remote file: {remote_name} ({e})")

def ftp_upload_overwrite(ftps: FTP_TLS, content_bytes: bytes, remote_name: str):
    ensure_remote_dirs(ftps, remote_name)
    bio = io.BytesIO(content_bytes)
    ftps.storbinary(f"STOR {remote_name}", bio)

def ftp_size(ftps: FTP_TLS, remote_name: str):
    try:
        sz = ftps.size(remote_name)
        return int(sz) if sz is not None else None
    except Exception:
        return None

# ---------- 1) Targets ----------
CSS_REMOTE = "partials/dna_tree_styles.css"

# HTML pages where we refresh the cache-buster on the CSS link.
HTML_TARGETS = [
    "partials/yates_ancestor_register.htm",
    "partials/ons_yates_dna_register.htm",
    "partials/justdna.htm",
    "partials/just-trees.htm",
    "partials/yates_ancestor_register_plus.htm",
    "partials/work_plus.htm",
    "partials/match_count.htm",
    "partials/lineage_count.htm",
    "partials/cousin_list_print.htm",
]

# ---------- 2) Patch builder (idempotent) ----------
def patch_css_for_ios(css_text: str) -> str:
    # Minimal iOS-friendly adjustments. We wrap in a clearly-marked block; re-running does not duplicate.
    # - table.sortable th/td line-height and word-break normalization
    # - -webkit-text-size-adjust safeguard
    block_header = "/* --- iOS font/spacing fix (2025-11-07) --- */"
    block_rules  = (
        "html, body { -webkit-text-size-adjust:100%; }\n"
        "table.sortable th, table.sortable td { line-height:1.5; word-break:normal; }\n"
    )
    if block_header in css_text:
        # Already present; optionally refresh the block content in place to ensure exact rules.
        rx = re.compile(r"/\*\s*--- iOS font/spacing fix .*?--- \*/\s*[^/]*?(?=$|/\*)", re.S)
        if rx.search(css_text):
            css_text = rx.sub(block_header + "\n" + block_rules, css_text, count=1)
        else:
            # Edge case: header string present but regex did not match; append once.
            css_text = css_text.rstrip() + "\n\n" + block_header + "\n" + block_rules
    else:
        css_text = css_text.rstrip() + "\n\n" + block_header + "\n" + block_rules
    return css_text

def refresh_css_cache_buster(html_text: str, ts: str) -> str:
    # Replace any existing query (?v=...) on /partials/dna_tree_styles.css with the new timestamp.
    # Handles single/double-quoted attributes.
    def _repl(m):
        prefix = m.group(1)  # href=" or href='
        path   = m.group(2)  # /partials/dna_tree_styles.css
        return f'{prefix}{path}?v={ts}"'
    # Normalize to double-quote for replacement safety.
    html_text = html_text.replace("'", '"')
    pat = re.compile(r'(href=")(/partials/dna_tree_styles\.css)(?:\?[^"]*)?"', re.I)
    if pat.search(html_text):
        html_text = pat.sub(_repl, html_text)
    else:
        # If a link tag exists without direct match, try to inject one in <head> (rare fallback).
        head_pat = re.compile(r"<head[^>]*>", re.I)
        if head_pat.search(html_text):
            link_tag = f'<link rel="stylesheet" type="text/css" href="/partials/dna_tree_styles.css?v={ts}" />'
            html_text = head_pat.sub(lambda m: m.group(0) + "\n" + link_tag, html_text, count=1)
    return html_text

# ---------- 3) Execute patch + uploads ----------
def run_fix():
    if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
        print("[SKIP] Missing FTP creds; uploads skipped.")
        return

    ts = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")
    print("[INFO] Cache-buster timestamp:", ts)

    try:
        ftps = ftp_connect()

        # Patch CSS
        print("[STEP] Download existing CSS:", "/" + _remote_path(CSS_REMOTE))
        css_bytes = ftp_download_if_exists(ftps, _remote_path(CSS_REMOTE))
        try:
            css_text = css_bytes.decode("utf-8")
        except Exception:
            css_text = css_bytes.decode("iso-8859-15", errors="ignore")

        patched = patch_css_for_ios(css_text)
        # Keep CSS ASCII to maintain ISO-8859-15 safety; rules are ASCII-only.
        patched_bytes = patched.encode("ascii", errors="ignore")
        ftp_upload_overwrite(ftps, patched_bytes, _remote_path(CSS_REMOTE))
        print("[OK] Uploaded patched CSS -> /" + _remote_path(CSS_REMOTE))

        # Touch each HTML to refresh the cache-buster on the CSS link.
        for rel in HTML_TARGETS:
            rp = _remote_path(rel)
            try:
                html_bytes = ftp_download(ftps, rp)
            except Exception as e:
                print(f"[MISS] {rp} ({e}); skipping.")
                continue
            # Decode as ISO-8859-15 first (your pages are authored that way), fallback to utf-8.
            try:
                html_text = html_bytes.decode("iso-8859-15")
            except Exception:
                html_text = html_bytes.decode("utf-8", errors="ignore")

            updated = refresh_css_cache_buster(html_text, ts)
            # Write back in ISO-8859-15 with entity fallback.
            out_bytes = updated.encode("iso-8859-15", errors="xmlcharrefreplace")
            ftp_upload_overwrite(ftps, out_bytes, rp)
            print("[PUT]  cache-busted:", "/" + rp)

        # Size check (best-effort)
        print("\n--- SIZE Verification (if supported) ---")
        for p in [ _remote_path(CSS_REMOTE) ] + [ _remote_path(h) for h in HTML_TARGETS ]:
            try:
                sz = ftp_size(ftps, p)
                print(f"/{p} : {sz if sz is not None else '(SIZE unsupported)'}")
            except Exception:
                print(f"/{p} : (check skipped)")

        try: ftps.quit()
        except Exception: pass

        print("\n--- Open a couple of pages (manual hard-refresh optional) ---")
        print("DNA Register:          https://yates.one-name.net/partials/yates_ancestor_register.htm")
        print("Register PLUS:         https://yates.one-name.net/partials/yates_ancestor_register_plus.htm")
        print("Trees:                 https://yates.one-name.net/partials/just-trees.htm")
        print("Shared stylesheet:     https://yates.one-name.net/partials/dna_tree_styles.css")
        print("\nIf an iPhone still shows old spacing, try a hard-refresh (Safari: tap-reload or clear cache).")
    except Exception as e:
        print("[FAIL] Session:", e)
        traceback.print_exc()

run_fix()
# ====== CUT STOP  [1/1] CELL ====================================================================


/tmp/ipython-input-3198153314.py:154: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S")


[INFO] Cache-buster timestamp: 20251107221012
[STEP] Download existing CSS: /partials/dna_tree_styles.css
[OK] Uploaded patched CSS -> /partials/dna_tree_styles.css
[PUT]  cache-busted: /partials/yates_ancestor_register.htm
[PUT]  cache-busted: /partials/ons_yates_dna_register.htm
[PUT]  cache-busted: /partials/justdna.htm
[PUT]  cache-busted: /partials/just-trees.htm
[PUT]  cache-busted: /partials/yates_ancestor_register_plus.htm
[PUT]  cache-busted: /partials/work_plus.htm
[PUT]  cache-busted: /partials/match_count.htm
[PUT]  cache-busted: /partials/lineage_count.htm
[PUT]  cache-busted: /partials/cousin_list_print.htm

--- SIZE Verification (if supported) ---
/partials/dna_tree_styles.css : 3814
/partials/yates_ancestor_register.htm : 1267364
/partials/ons_yates_dna_register.htm : 1267364
/partials/justdna.htm : 1267364
/partials/just-trees.htm : 794896
/partials/yates_ancestor_register_plus.htm : 1343103
/partials/work_plus.htm : 1267364
/partials/match_count.htm : 18955
/partials/

# Cell 1

In [56]:
# ====== CUT START [1/1] CELL 1 — GEDCOM -> final_combined_df_with_value_labels.csv + working HTML ======
# RON RULES — QUICK CODE CARD (v2025.11.05-Cell1-NoValueScoring)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - Punctuation in strings use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# - Python code (inline, executable, full COLAB Cell paste-ready section).
# - XHTML 1.0 Transitional; Times New Roman body; absolute links to /partials/.
# - Outputs uploaded to /partials/: CSV + HTML; autosomal_count.txt stays local (Cell 2 handles upload).

import os, re, glob, logging, functools, socket, posixpath, traceback
from datetime import datetime
from collections import defaultdict, Counter
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import pandas as pd
from tqdm import tqdm
from ftplib import FTP_TLS
from IPython.display import display, Javascript
from string import Template

# ====== LOGGING ======
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ====== SECRETS + CONFIG + FTP HELPERS ======
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_HOST = os.environ.get('FTP_HOST','')
FTP_USER = os.environ.get('FTP_USER','')
FTP_PASS = os.environ.get('FTP_PASS','')
FTP_PORT = int(os.environ.get('FTP_PORT','21'))
FTP_DIR  = os.environ.get('FTP_DIR','').strip().strip('/')

def _ftps_connect():
    socket.setdefaulttimeout(30)
    ftps = FTP_TLS(timeout=30)
    ftps.connect(FTP_HOST, FTP_PORT)
    ftps.login(FTP_USER, FTP_PASS)
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(True)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split('/') if p]:
            try:
                ftps.cwd(p)
            except Exception:
                try: ftps.mkd(p)
                except Exception: pass
                ftps.cwd(p)
    return ftps

def _ftps_ensure_dir(ftps, path):
    if not path: return
    for p in [p for p in path.split('/') if p]:
        try:
            ftps.cwd(p)
        except Exception:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)

def _ftps_upload(ftps, local_path, remote_name):
    with open(local_path, 'rb') as fh:
        ftps.storbinary("STOR " + remote_name, fh)
    print(f"[OK] Uploaded: {local_path} -> {ftps.pwd().rstrip('/')}/{remote_name}")

# Output names (local + remote inside /partials/)
CSV_OUT_LOCAL   = "final_combined_df_with_value_labels.csv"  # kept for compatibility with Cell 2
HTML_OUT_LOCAL  = "cell1_work_table.htm"
REMOTE_DIR      = "partials"
REMOTE_CSV_NAME = os.path.basename(CSV_OUT_LOCAL)
REMOTE_HTML_NAME= os.path.basename(HTML_OUT_LOCAL)

# Absolute links used inside HTML
ABS_CSV_URL  = f"/{REMOTE_DIR}/{REMOTE_CSV_NAME}"
ABS_HOME_URL = "/partials/ons_yates_dna_register.htm"  # main register lives in /partials/

# ====== CORE STRUCTURES (dataset, GEDCOM parse) ======
anchor_gen1 = None
visited_pairs = set()
generation_table = []

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None
    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value
    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0]
        last_name = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1
        anchor_gen1 = self.anchor_gen1
        return self.gen_person.strip('@')
    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')
    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX','')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        elif '**' in npfx_value:
            cm_value = npfx_value.split('**')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value); return cm_value
        except ValueError:
            return ''
    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX','')
        if '&' in npfx_value:
            sort_part = npfx_value.split('&')[1]
            sort_value = sort_part.split('**')[0].strip() if '**' in sort_part else sort_part.strip()
            return sort_value
        return ''
    def get_extractable_YDNA(self):
        npfx_value = self.extractable_detail.get('NPFX','')
        if '**' in npfx_value:
            return npfx_value.split('**')[1].strip()
        return ''
    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC','').strip('@')

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []
    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            lines = f.readlines()
        current_dataset = None
        npfx_count = 0
        ydna_count = 0
        total_count = 0
        for line in lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0]); tag = parts[1]; value = parts[2] if len(parts) > 2 else None
            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME','FAMC']:
                    current_dataset.add_extractable_detail(tag, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)
                    if '**' in value:
                        ydna_count += 1
        autosomal_count = npfx_count - ydna_count
        print(f"GEDCOM contained {total_count} total records")
        print(f"Records tagged and filtered by NPFX: {npfx_count}")
        print(f"Records with YDNA information: {ydna_count}")
        print(f"Autosomal matches: {autosomal_count}")
        for ds in self.gedcom_datasets:
            if ds.get_extractable_NPFX():
                self.filter_pool.append(ds)
        # Optional manual second-level filter
        try:
            df_filter = pd.read_excel('filtered_ids.xlsx')
            manual_ids = set(df_filter['ID'])
            self.filter_pool = [d for d in self.filter_pool if d.get_gen_person() in manual_ids]
            print(f"After manual filter, total records: {len(self.filter_pool)}")
            logger.info(f"After manual filter, total records: {len(self.filter_pool)}")
        except FileNotFoundError:
            logger.warning("filtered_ids.xlsx not found. Skipping second-level manual filter.")
        return autosomal_count

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def quick_extract_name(full_text):
    name_marker = "\n1 NAME "
    idx = full_text.find(name_marker)
    if idx == -1:
        if full_text.startswith("1 NAME "):
            idx = 0
        else:
            return "UnknownName"
    start = idx + len(name_marker)
    end = full_text.find('\n', start)
    if end == -1: end = len(full_text)
    name_line = full_text[start:end].strip()
    if '/' not in name_line:
        return name_line[:10].replace(" ", "")
    first_name, last_name = name_line.split('/', 1)
    last_name = last_name.replace("/", "").strip()
    return last_name[:10].replace(" ", "") + first_name[:10].replace(" ", "")

def find_parents(individual_id, generation, parents_map):
    global visited_pairs, generation_table
    if individual_id not in parents_map: return
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id: return
    pair = (father_id, mother_id)
    if pair not in visited_pairs:
        visited_pairs.add(pair)
        generation_table.append((generation, pair))
    if father_id: find_parents(father_id, generation+1, parents_map)
    if mother_id: find_parents(mother_id, generation+1, parents_map)

def find_distant_ancestors(individual_id, parents_map, path=None):
    if path is None: path = []
    path.append(individual_id)
    if individual_id not in parents_map:
        return [path]
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id:
        return [path]
    paths = []
    if father_id: paths.extend(find_distant_ancestors(father_id, parents_map, path[:]))
    if mother_id: paths.extend(find_distant_ancestors(mother_id, parents_map, path[:]))
    return paths if paths else [path]

def filter_ancestral_line(winning_path_ids, generation_table_local, names_map):
    matching_table = []
    for generation, pair in generation_table_local:
        id1, id2 = pair
        if id1 in winning_path_ids or id2 in winning_path_ids:
            matching_table.append((generation, pair))
    matching_table.sort(key=lambda x: x[0])
    lines = []
    for gen, pair in matching_table:
        name_pair = [names_map.get(pid, "UnknownName") for pid in pair]
        lines.append(f"{name_pair[0]}&{name_pair[1]}")
    lines.reverse()
    return "~~~".join(lines)

def process_record_wrapper(individual_id, gedcom_instance, parents_map, names_map):
    global generation_table, visited_pairs, anchor_gen1
    generation_table = []; visited_pairs = set()
    find_parents(individual_id, 1, parents_map)
    distant_paths = find_distant_ancestors(individual_id, parents_map)
    best_score, best_path = None, None
    for path in distant_paths:
        name_path = [names_map.get(pid, "UnknownName") for pid in path]
        score = sum((idx+1) for idx, nm in enumerate(name_path) if 'Yates' in nm)
        if best_score is None or score > best_score:
            best_score, best_path = score, path
    best_path = best_path or []
    best_path_cleaned = [pid for pid in best_path if pid != individual_id]
    line_str = filter_ancestral_line(set(best_path_cleaned), generation_table, names_map)
    cm_value = ''; sort_value=''; ydna_value=''
    for ds in gedcom_instance.filter_pool:
        if ds.get_gen_person() == individual_id:
            cm_value = ds.get_extractable_cm()
            sort_value = ds.get_extractable_sort()
            ydna_value = ds.get_extractable_YDNA()
            break
    short_name = names_map.get(individual_id, "UnknownName")
    return [individual_id, sort_value, short_name, cm_value, line_str, ydna_value]

# ====== MAIN: BUILD DATAFRAME (No Value Scoring) ======
def main():
    files = glob.glob("*.ged")
    if not files:
        print("No GEDCOM files found."); return False
    print("Automatically selecting the first GEDCOM file.")
    gedcom_file_path = files[0]

    ged = Gedcom(gedcom_file_path)
    autosomal_count = ged.parse_gedcom()

    with open("autosomal_count.txt", "w") as f:
        f.write(str(autosomal_count))

    with open(gedcom_file_path, 'r', encoding='utf-8') as f:
        raw_data = f.read()

    blocks = raw_data.split('\n0 ')
    all_records = {}
    for blk in blocks:
        blk = blk.strip()
        if not blk: continue
        flend = blk.find('\n'); flend = len(blk) if flend == -1 else flend
        first_line = blk[:flend]
        if '@' in first_line:
            start = first_line.find('@') + 1
            end = first_line.find('@', start)
            rec_id = first_line[start:end].strip()
            all_records[rec_id] = blk

    parents_map, names_map, families = {}, {}, {}
    for rec_id, txt in all_records.items():
        if 'FAM' in txt[:50]:
            father_idx = txt.find('1 HUSB @')
            husb_id = txt[father_idx+len('1 HUSB @'):txt.find('@', father_idx+len('1 HUSB @'))] if father_idx != -1 else None
            wife_idx = txt.find('1 WIFE @')
            wife_id = txt[wife_idx+len('1 WIFE @'):txt.find('@', wife_idx+len('1 WIFE @'))] if wife_idx != -1 else None
            kids = [ln.split('@')[1] for ln in txt.split('\n') if ln.strip().startswith('1 CHIL @')]
            families[rec_id] = (husb_id, wife_id, kids)
    for fam_id, (f_id, m_id, k_list) in families.items():
        for kid in k_list:
            parents_map[kid] = (f_id, m_id)
    for rec_id, txt in all_records.items():
        names_map[rec_id] = quick_extract_name("\n" + txt)

    individual_ids = [d.get_gen_person() for d in ged.filter_pool]
    print(f"Processing {len(individual_ids)} individuals with chunk-based parallel...")

    combined_rows = []
    chunk_size = 50
    max_workers = os.cpu_count() or 4
    with ProcessPoolExecutor(max_workers=max_workers) as executor, tqdm(total=len(individual_ids), desc="Building Yates Lines (Stage 1)") as pbar:
        for chunk in chunks(individual_ids, chunk_size):
            func = functools.partial(process_record_wrapper, gedcom_instance=ged, parents_map=parents_map, names_map=names_map)
            results = list(executor.map(func, chunk))
            combined_rows.extend(results)
            pbar.update(len(chunk))

    columns = ["ID#", "Match to", "Name", "cM", "Yates DNA Ancestral Line", "haplogroup"]
    df = pd.DataFrame(combined_rows, columns=columns)
    df.index += 1

    # Trim the historical prefix if present
    def remove_specific_prefix(row):
        prefix = "YatesJohn&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesEdmund&CornellMargaret~~~YatesRichard&AshendonJoan~~~YatesJohn&HydeAlice~~~YatesThomas&FauconerElizabeth~~~"
        if str(row["Yates DNA Ancestral Line"]).startswith(prefix):
            row["Yates DNA Ancestral Line"] = row["Yates DNA Ancestral Line"][len(prefix):]
        return row
    df = df.apply(remove_specific_prefix, axis=1)

    # No Value/Range/Label computation; sort only by lineage for stability
    df.sort_values(by=["Yates DNA Ancestral Line"], inplace=True)

    # Export CSV (UTF-8-SIG for downstream safety; filename kept for Cell 2 compatibility)
    df.to_csv(CSV_OUT_LOCAL, index=False, encoding="utf-8-sig")
    logger.info("Exported CSV -> %s", CSV_OUT_LOCAL)

    # Simple working HTML (no Value columns)
    final_cols = ["ID#", "cM", "haplogroup", "Match to", "Yates DNA Ancestral Line"]
    table_html = df.to_html(index=False, columns=final_cols, escape=False, border=1)

    page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Cell 1 Working Table</title>
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { font-family: 'Times New Roman', Georgia, serif; background:#ffffff; color:#222; margin:0; padding:20px; }
  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }
  .meta { text-align:center; font-size:12px; color:#555; margin:2px 0 12px 0; }
  .downloads { text-align:center; margin:4px 0 12px 0; font-size:13px; }
  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }
  table { width:100%; border-collapse:collapse; }
  th, td { border:1px solid #333; padding: 6px 8px; vertical-align:top; }
  th { background:#e3eaf8; text-align:left; }
  td:nth-child(5) { text-align:left; white-space:normal; }
</style>
<script type="text/javascript">
//<![CDATA[
(function(){ function z(n){return (n<10?'0':'')+n;}
document.addEventListener('DOMContentLoaded', function(){
  var el = document.getElementById('last-updated');
  if(el){ var d=new Date(document.lastModified||new Date());
    el.innerHTML = d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes()); }
}, false); })();
//]]>
</script>
</head>
<body>
  <h1>Cell 1 Working Table</h1>
  <div class="meta">
    <a href="$HOME" target="_blank" rel="noopener">Home</a>
    &nbsp;|&nbsp; Last updated: <span id="last-updated"></span>
    &nbsp;|&nbsp; Download: <a href="$CSV">$CSV</a>
  </div>
  <div class="downloads"><a href="$CSV">/partials/$CSV_NAME</a></div>
  $TABLE
</body>
</html>""")

    page = page_tpl.safe_substitute(
        HOME=ABS_HOME_URL,
        CSV=ABS_CSV_URL,
        CSV_NAME=os.path.basename(ABS_CSV_URL),
        TABLE=table_html
    )

    with open(HTML_OUT_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(page)
    logger.info("Exported HTML -> %s", HTML_OUT_LOCAL)

    return True

ok = main()

# ====== UPLOAD ALL ARTIFACTS TO /partials/ ======
if ok and all([FTP_HOST, FTP_USER, FTP_PASS]):
    print("[INFO] Uploading artifacts to /partials/ ...")
    try:
        ftps = _ftps_connect()
        _ftps_ensure_dir(ftps, "partials")
        try:
            _ftps_upload(ftps, CSV_OUT_LOCAL, os.path.basename(CSV_OUT_LOCAL))
        except Exception as e:
            print("[ERROR] CSV upload failed:", e)
        try:
            _ftps_upload(ftps, HTML_OUT_LOCAL, os.path.basename(HTML_OUT_LOCAL))
        except Exception as e:
            print("[ERROR] HTML upload failed:", e)
        try: ftps.quit()
        except Exception: pass
        print("[OK] Uploads complete to /partials/")
    except Exception as e:
        print("[ERROR] FTP session failed:", e)
        traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload (missing creds or build failed).")

# ====== DONE ======
try:
    display(Javascript('alert("\\u2705 Cell 1 build complete (no value scoring): CSV + HTML uploaded to /partials/");'))
except Exception:
    pass
print("\n--- Cell 1 Complete (no value scoring): artifacts at /partials/ ---")
# ====== CUT STOP [1/1] CELL 1 ===================================================================


Automatically selecting the first GEDCOM file.
GEDCOM contained 62312 total records
Records tagged and filtered by NPFX: 1572
Records with YDNA information: 0
Autosomal matches: 1572
After manual filter, total records: 7
Processing 7 individuals with chunk-based parallel...


Building Yates Lines (Stage 1): 100%|██████████| 7/7 [00:04<00:00,  1.74it/s]


[INFO] Uploading artifacts to /partials/ ...
[OK] Uploaded: final_combined_df_with_value_labels.csv -> /partials/final_combined_df_with_value_labels.csv
[OK] Uploaded: cell1_work_table.htm -> /partials/cell1_work_table.htm
[OK] Uploads complete to /partials/


<IPython.core.display.Javascript object>


--- Cell 1 Complete (no value scoring): artifacts at /partials/ ---


# Cell 2

In [59]:
# ====== CUT START [1/1] CELL 2 — Build + Publish DNA Register (Unified: Best Display + Correct First-Ancestor) ======
# RON RULES — QUICK CODE CARD (v2025.11.08-Cell2-UNIFIED-BEST)
# - Complete & runnable in Colab; ASCII-only source (ISO-8859-15 safe outputs).
# - XHTML 1.0 Transitional; explicit prints; no fabrication.
# - Typography is controlled ONLY by /partials/dna_tree_styles.css (no inline font-family anywhere).
# - Row-highlight FIX is limited to PARTIALS ONLY (not the main register page).
# - Publishes SAME HTML to:
#       /partials/yates_ancestor_register.htm     (canonical, Find target)
#       /partials/ons_yates_dna_register.htm     (legacy clone)
#       /partials/justdna.htm                     (JUSTDNA alias)
# - Also writes PARTIALS (match_count.htm, lineage_count.htm, cousin_list_print.htm) and CSV/XLSX exports.

import os, re, io, posixpath, socket, traceback, urllib.parse as _u
from ftplib import FTP_TLS
import pandas as pd
import html as _html
from string import Template

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR',  '')
    os.environ.setdefault('FTP_PORT', '21')

# ---------- 1) Config ----------
CSV_IN = "final_combined_df_with_value_labels.csv"

LOCAL_HTML         = "yates_ancestor_register.htm"
REMOTE_HTML_CANON  = posixpath.join("partials", "yates_ancestor_register.htm")
REMOTE_HTML_LEG    = posixpath.join("partials", "ons_yates_dna_register.htm")
REMOTE_HTML_SIMPLE = posixpath.join("partials", "justdna.htm")

DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"
TREES_ABS        = "https://yates.one-name.net/partials/just-trees.htm"

EXPORT_BASENAME = "yates_ancestor_register"
LOCAL_CSV       = f"{EXPORT_BASENAME}.csv"
LOCAL_XLSX      = f"{EXPORT_BASENAME}.xlsx"
REMOTE_CSV      = posixpath.join("partials", os.path.basename(LOCAL_CSV))
REMOTE_XLSX     = posixpath.join("partials", os.path.basename(LOCAL_XLSX))

WORK_PLUS_LOCAL  = os.path.join("partials", "work_plus.htm")
WORK_PLUS_REMOTE = posixpath.join("partials", "work_plus.htm")

LOCAL_COUNT_FILE  = "/content/autosomal_count.txt"
REMOTE_COUNT_NAME = "autosomal_count.txt"

FTP_DIR         = (os.environ.get("FTP_DIR","") or "").strip()
TNG_BASE        = "https://yates.one-name.net/tng"
TNG_TREE        = "tree1"
HOME_URL        = "https://yates.one-name.net/partials/yates_ancestor_register.htm"
REMOTE_NAME_ABS = HOME_URL
COUNT_PUBLIC_URL = (f"/{FTP_DIR}/{REMOTE_COUNT_NAME}" if FTP_DIR else f"/{REMOTE_COUNT_NAME}")

# Layout
TABLE_WIDTH_PX = 3150
COL_A_PX       = 1100
FIND_COL_PX    = 118
ARROW_ENTITY   = "&rarr;"
REMOVE_PERIOD_AT_END = True

# Resolver (server mapping for Match-to -> Unmasked)
SERVER_PARTIALS_DIR        = "partials"
SERVER_MAPPING_BASENAME    = "match_to_unmasked.csv"
SERVER_MAPPING_REMOTE      = posixpath.join(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)
SERVER_MAPPING_LOCAL_CACHE = "match_to_unmasked.server.csv"

# External stylesheet (controls typography site-wide)
STYLESHEET_HREF = "/partials/dna_tree_styles.css"
CSS_VERSION     = "v2025-11-06"  # bump to bust caches when CSS changes
HEAD_LINK       = f'<link rel="stylesheet" type="text/css" href="{STYLESHEET_HREF}?{CSS_VERSION}" />'
HEAD_LINK_URL   = STYLESHEET_HREF + "?" + CSS_VERSION  # used inside JS (print window)

# ---------- 2) FTP helpers ----------
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))
FTP_PASSIVE = True

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        parts = [p for p in FTP_DIR.split("/") if p]
        for p in parts:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ensure_remote_dirs(ftps: FTP_TLS, remote_path: str):
    if "/" not in remote_path: return
    pwd0 = ftps.pwd()
    for seg in [p for p in remote_path.split("/")[:-1] if p and p != "."]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)
    ftps.cwd(pwd0)

def ftp_download_if_exists(ftps: FTP_TLS, remote_name: str, local_name: str) -> bool:
    try:
        with open(local_name, "wb") as f:
            ftps.retrbinary(f"RETR {remote_name}", f.write)
        print(f"[PULL] {remote_name} -> {os.path.abspath(local_name)}")
        return True
    except Exception as e:
        try:
            if os.path.exists(local_name): os.remove(local_name)
        except Exception: pass
        print(f"[MISS] {remote_name} ({e})")
        return False

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str):
    ensure_remote_dirs(ftps, remote_name)
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote_name}", fh)
    print(f"[PUT]  {local_path} -> {remote_name}")

def ftp_size(ftps: FTP_TLS, remote_name: str):
    try:
        sz = ftps.size(remote_name)
        return int(sz) if sz is not None else None
    except Exception:
        return None

# ---------- 3) Resolver ----------
def _read_mapping_csv(path: str) -> pd.DataFrame:
    encs = ("iso-8859-15","utf-8-sig","utf-8","cp1252","latin1")
    last = None; df = None
    for enc in encs:
        try:
            df = pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False)
            break
        except Exception as e:
            last = e
    if df is None:
        raise RuntimeError(f"Unable to read mapping CSV {path}: {last}")
    if df.shape[1] < 2:
        raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")
    df = df.iloc[:, :2].copy()
    df.columns = ["code","unmasked"]
    df["code"] = df["code"].astype(str).str.strip().str.lower()
    df["unmasked"] = df["unmasked"].astype(str).str.strip()
    df = df[df["code"] != ""].drop_duplicates(subset=["code"], keep="first")
    if df.empty:
        raise RuntimeError("Mapping CSV empty after normalization.")
    return df

def load_resolver_from_server() -> dict:
    with ftp_connect() as ftps:
        try:
            ftps.cwd(SERVER_PARTIALS_DIR)
        except Exception:
            pass
        ok = ftp_download_if_exists(ftps, SERVER_MAPPING_BASENAME, SERVER_MAPPING_LOCAL_CACHE)
        try: ftps.quit()
        except Exception: pass
    if not ok:
        raise RuntimeError(
            "Resolver not found on server: /" + _remote_path(SERVER_MAPPING_REMOTE) +
            ". Upload match_to_unmasked.csv into /partials/ and re-run."
        )
    df_map = _read_mapping_csv(SERVER_MAPPING_LOCAL_CACHE)
    print(f"[OK] Resolver loaded: {len(df_map)} codes")
    return dict(zip(df_map["code"], df_map["unmasked"]))

MATCH_TO_UNMASKED = {}
def _setup_resolver():
    global MATCH_TO_UNMASKED
    if not MATCH_TO_UNMASKED:
        MATCH_TO_UNMASKED = load_resolver_from_server()

def resolve_match_to(code: str) -> str:
    if not isinstance(code, str): return ""
    return MATCH_TO_UNMASKED.get(code.strip().lower(), code)

# ---------- 4) Name & text utils (UNIFIED: learnings from both versions) ----------
SEP_RE = re.compile(r"\s*(?:\u2192|&rarr;|;|>|,|~{2,}|/{2,}|\|{2,})\s*")
def split_tokens(s):
    if pd.isna(s): return []
    if not isinstance(s, str): s = str(s)
    return [p.strip() for p in SEP_RE.split(s) if str(p).strip()]

def _clean_piece(text: str) -> str:
    t = re.sub(r'~+', ' ', str(text)); t = re.sub(r'\s+', ' ', t)
    return t.strip()

_PARTICLES = {"de","del","della","der","van","von","da","dos","das","di","la","le","du","of"}

def _smart_title(token: str) -> str:
    if not token: return token
    token = re.sub(r"(^|\b)([a-z])(['’])([a-z])", lambda m: m.group(1)+m.group(2).upper()+m.group(3)+m.group(4).upper(), token.lower())
    token = "-".join([w.capitalize() for w in token.split("-")])
    token = re.sub(r"\bmc([a-z])",  lambda m: "Mc"+m.group(1).upper(),  token)
    token = re.sub(r"\bmac([a-z])", lambda m: "Mac"+m.group(1).upper(), token)
    return token

def smart_titlecase(name: str) -> str:
    name = _clean_piece(name)
    if not name: return name
    if "," in name:
        last, first = [p.strip() for p in name.split(",", 1)]
        pieces = (first + " " + last).split()
    else:
        pieces = name.split()
    out = []
    for i, w in enumerate(pieces):
        out.append(w.lower() if (i>0 and w.lower() in _PARTICLES) else _smart_title(w))
    return " ".join(out)

# Split CamelCase tokens like "YatesJohn" -> ("Yates","John") then render "John Yates"
_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")
def surname_given_from_token(token):
    token = token.strip()
    if not token: return (token,)
    idx = None
    for i in range(1, len(token)):
        if token[i-1].islower() and token[i].isupper():
            idx = i; break
    if idx is None:
        for i in range(1, len(token)):
            if token[i].isupper():
                idx = i; break
    if idx is None: return (token,)
    surname = token[:idx]; given = token[idx:]
    given_spaced = re.sub(r'(?<!^)([A-Z])', r' \1', given)
    return (f"{given_spaced.strip()} {surname.strip()}",)

def normalize_person_name(s: str) -> str:
    if pd.isna(s): return ""
    s = _clean_piece(str(s))
    if "," in s:
        last, first = [p.strip() for p in s.split(",", 1)]
        s = f"{first} {last}"
    if " " not in s and s.isalpha():
        return smart_titlecase(surname_given_from_token(s)[0])
    return smart_titlecase(s)

def norm_matchee_name(raw: str) -> str:
    raw = str(raw or "").strip()
    if not raw: return ""
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1: return nm
        return f"{parts[0]} {parts[-1]}".strip()
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1: return nm
        return f"{ps[0]} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates: return surname
    given = smart_titlecase(given_candidates[0])
    return f"{given} {surname}".strip()

def truncate_first(name: str, n: int = 7) -> str:
    name = name.strip()
    if not name: return name
    parts = name.split()
    return parts[0][:n] if len(parts) == 1 else f"{parts[0][:n]} {parts[-1]}"

def derive_common_from_first_token(tokens):
    if not tokens: return ("","")
    first = _clean_piece(tokens[0])
    parts = re.split(r"\s*(?:&| and )\s*", first, maxsplit=1, flags=re.I)
    if len(parts) != 2: return ("","")
    def _norm(s):
        # Accept "YatesJohn" and "Yates, John" and plain "John Yates"
        return smart_titlecase(s) if " " in s or "," in s else smart_titlecase(surname_given_from_token(s)[0])
    return (_norm(parts[0]), _norm(parts[1]))

def degree_label_from_generations(g):
    if g <= 1: return ("parents" if g == 1 else "self")
    if g == 2: return "grandparents"
    greats = g - 2
    if greats == 1: return "great-grandparents"
    return f"{greats}x-great-grandparents"

def build_header(subject_name, cm_val, matchee_name_html, gens, husband, wife):
    try:
        cm_str = f"{int(round(float(cm_val)))}"
    except Exception:
        cm_str = (str(cm_val).strip() or "0")
    degree_label = degree_label_from_generations(gens)
    parts = [
        f"{subject_name} is a {cm_str} cM cousin match to {matchee_name_html}, whose",
        f"{degree_label} (back {gens} Gens)",
        "are",
        f"{husband} & {wife}."
    ]
    s = " ".join(parts)
    if REMOVE_PERIOD_AT_END: s = re.sub(r'\.\s*$', '', s)
    return s

# ---------- 5) Read CSV; detect columns ----------
def find_col(df, patterns, prefer_exact=None):
    cols = list(df.columns)
    lowmap = {c.lower(): c for c in cols}
    if prefer_exact:
        for name in prefer_exact:
            if name in df.columns: return name
            if name and name.lower() in lowmap: return lowmap[name.lower()]
    for pat in patterns:
        rx = re.compile(pat, re.I)
        for c in cols:
            if rx.search(c): return c
    return None

_encs = ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None
if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"[OK] Loaded CSV: {len(df)} rows, {len(df.columns)} cols")

id_col    = find_col(df, [r'^(id#|personid)$'], ["ID#","ID","PersonID","personID"])
match_col = find_col(df, [r'^match\s*to$'], ["Match to","Match"])
name_col  = find_col(df, [r'^name$'], ["Name"])
cm_col    = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM","cm"])
path_col  = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'], ["Yates DNA Ancestral Line","Ancestral Line","Lineage"])

if not id_col:    raise ValueError("CSV missing an ID#/PersonID column.")
if not match_col: raise ValueError("CSV missing 'Match to' column.")
if not name_col:  raise ValueError("CSV missing 'Name' column.")
if not cm_col:    raise ValueError("CSV missing 'cM' column.")
if not path_col:  raise ValueError("CSV missing lineage/path column.")

# ---------- 6) Transform -> display_df ----------
ID_PAT = re.compile(r"\bI\d+\b", re.I)
def extract_person_id(s: str) -> str:
    m = ID_PAT.search(str(s or ""))
    return m.group(0).upper() if m else ""

_setup_resolver()

headers, lineages, findcol = [], [], []
subjects, first_ancestors  = [], []

for _, row in df.iterrows():
    subject_raw  = row.get(match_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b = f"<strong>{subject_name}</strong>" if subject_name else subject_name

    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name

    if pid:
        matchee_name_html = (
            f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0&display=vertical&generations=15" '
            f'target="_blank" rel="noopener">{matchee_name}</a>'
        )
    else:
        matchee_name_html = matchee_name

    cm_val = row.get(cm_col, "0")
    tokens = split_tokens(row.get(path_col, ""))
    gens_total = len(tokens)
    tokens_disp = tokens[:7]

    # Prefer explicit husband/wife columns if present, else derive from first token (CamelCase aware)
    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw = str(row.get("common_husband","")).strip()
        wife_raw    = str(row.get("common_wife","")).strip()
        if not husband_raw and not wife_raw:
            husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    header_html = build_header(
        subject_name_b,
        cm_val,
        matchee_name_html,
        gens_total,
        husband_raw,
        wife_raw
    )

    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"
    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    q = _u.quote(subject_name)
    quick = (
        f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
        f'title="Open a filtered view for {subject_name}">Find</a>'
    )

    headers.append(header_html)
    lineages.append(lineage_text)
    findcol.append(quick)
    subjects.append(subject_name)
    first_ancestors.append(tokens[0] if tokens else "")

LINEAGE_HEADER_SAFE = "Lineage (Starting with oldest ancestor, the line is:)"
df["Match Summary"]     = headers
df[LINEAGE_HEADER_SAFE] = lineages
df["Find"]              = findcol
df["Subject"]           = subjects
df["First Ancestor"]    = [_clean_piece(x) for x in first_ancestors]
display_df = df[["Find","Match Summary", LINEAGE_HEADER_SAFE]]

# ---------- 6.1) Clean exports (ISO-8859-15 safe; robust) ----------
TAG_RE = re.compile(r"<[^>]+>")
def _html_to_text(s: str) -> str:
    t = TAG_RE.sub("", str(s or ""))
    t = _html.unescape(t)
    t = t.replace("\u2192", "->")
    return re.sub(r"\s+", " ", t).strip()

def _extract_find_url(cell_html: str) -> str:
    m = re.search(r'href="([^"]+)"', str(cell_html or ""))
    return _html.unescape(m.group(1)) if m else ""

export_df = pd.DataFrame({
    "Find URL":      [ _extract_find_url(v) for v in display_df["Find"].tolist() ],
    "Match Summary": [ _html_to_text(v)     for v in display_df["Match Summary"].tolist() ],
    "Lineage":       [ _html_to_text(v)     for v in display_df[LINEAGE_HEADER_SAFE].tolist() ],
})

# Write CSV with fallback error handler for older pandas/envs
try:
    export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
except TypeError:
    # Older pandas may not recognize encoding error args; do manual encode
    buf = io.StringIO()
    export_df.to_csv(buf, index=False)
    with open(LOCAL_CSV, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(buf.getvalue())

try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    with pd.ExcelWriter(LOCAL_XLSX) as _writer:
        export_df.to_excel(_writer, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))

# ---------- 7) HTML (Register main page; no inline font-family) ----------
TABLE_CSS = """
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { font-size:100%; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }
  .wrap { max-width:3150px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:48px; }
  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }
  h1 { margin:0 0 4px 0; font-size:26px; line-height:1.2; text-align:center; }
  .centerline { text-align:center; }
  .downloads { text-align:center; margin:4px 0 10px 0; font-size:13px; }
  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }
  .sortbar { margin:6px 0 10px 0; font-size:13px; background:#ffffff; padding:6px 8px; border-radius:6px; display:flex; flex-wrap:wrap; gap:5px; align-items:center; border:1px solid #ddd; justify-content:center; }
  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px; text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; }
  .btn:hover { background:#4668aa; }
  input.btn.search { background:#fff; color:#111; border-color:#bbb; }
  .find-cell { white-space:nowrap; }
  .selbox { margin-right:6px; vertical-align:middle; }
  .table-scroll { max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }
  table.sortable { border-collapse:collapse; width:3150px; table-layout:fixed; }
  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; }
  table.sortable th { background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; cursor:pointer; }
  #first-row td { border-top:2px solid #999; }
  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px; border:1px solid #3e5a97; background:#5b79b8; color:#fff; cursor:pointer; border-radius:6px; font-size:12px; display:none; z-index:9999; }
  .back-to-top:hover { background:#4668aa; }
  #dynamicContent { margin:10px 0 14px 0; }
  @media screen and (max-width: 820px) { .wrap { padding:12px; } h1 { font-size:22px; } }
</style>
"""

DYNAMIC_BLOCK = (
    '<div class="sortbar">'
    '<a class="btn" href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm" target="_blank" rel="noopener">Study Details</a>'
    '<a class="btn" href="https://yates.one-name.net/gengen/dna_theory_of_the_case.htm" target="_blank" rel="noopener">Theory in Action</a>'
    '<a class="btn" href="/gengen/images/cousin-calculator.jpg" target="_blank" rel="noopener">Cousin Connection</a>'
    '<a class="btn" href="/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" rel="noopener">Cousin by DNA</a>'
    '<a class="btn" href="/partials/match_count.htm" target="_blank" rel="noopener">Match Count</a>'
    '<a class="btn" href="/partials/lineage_count.htm" target="_blank" rel="noopener">Lineage Count</a>'
    f'<a class="btn" href="{DNA_REGISTER_ABS}" target="_blank" rel="noopener">DNA Register</a>'
    f'<a class="btn" href="{TREES_ABS}" target="_blank" rel="noopener">Trees</a>'
    '<span class="btn" id="show-selected" title="Show all rows for the checked name(s)">Show Selected</span>'
    '<span class="btn" id="show-all" title="Show All">Show All</span>'
    '<span class="btn" id="print-cousin-list" style="cursor:pointer;" title="Open a printable list of the *currently visible* rows">Cousin List (Printable)</span>'
    '<span class="btn" id="clear-selected">Reset</span>'
    '<input type="text" id="search-box" class="btn search" size="24" value="" placeholder="Search&amp;hellip;" />'
    '</div>'
)

JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")
UPDATED_BLOCK = (
    '<div class="updated centerline">'
    'Last updated: <span id="last-updated"></span>'
    ' &nbsp;|&nbsp; Autosomal matches: <span id="auto-count" class="js-count"></span>'
    ' &nbsp;|&nbsp; Showing: <span id="showing-count"></span>'
    '</div>'
)

DOWNLOADS_BLOCK = (
    '<p class="downloads centerline">Download: '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_CSV))}">CSV</a> | '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_XLSX))}">Excel</a></p>'
)

page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
$HEAD_LINK
$TABLE_CSS
<script type="text/javascript">var HEAD_LINK_URL = "$HEAD_LINK_URL";</script>
</head>
<body id="top">
<div class="wrap">
  <h1 class="centerline">ONS Yates Study Autosomal DNA Register</h1>
  $DOWNLOADS_BLOCK
  $UPDATED_BLOCK
  $DYNAMIC_BLOCK
  <div class="table-scroll">
    $HTML_TABLE
  </div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]); var asc=(dir==='asc');
    rows.sort(function(a,b){var A=textOf(a.cells[colIndex]),B=textOf(b.cells[colIndex]); if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;});
    var frag=document.createDocumentFragment(); for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]); tb.appendChild(frag);
    updateShowing();
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++){(function(idx){
      var th = ths[idx]; var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+[↑↓]/,''); }
        th.innerHTML += (dir==='asc' ? ' &uarr;' : ' &darr;');
        sortTable(tbl,idx,dir);
      },false);
    })(i);}
  }
  function stampLastUpdated(){
    var el = document.getElementById('last-updated'); if(!el) return;
    var d  = new Date(document.lastModified || new Date());
    var months = ['January','February','March','April','May','June','July','August','September','October','November','December'];
    el.innerHTML = d.getDate() + ' ' + months[d.getMonth()] + ' ' + d.getFullYear();
  }
  function formatWithCommas(n){
    try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
  }
  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0;
    for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; }
    return n;
  }
  function updateShowing(){
    var el=document.getElementById('showing-count'); if(!el) return;
    el.textContent = formatWithCommas(visibleRowCount());
  }
  function loadAutoCount(){
    var el=document.getElementById('auto-count'); if(!el) return;
    var url='$JS_COUNT_URL';
    try{
      var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){ if(xhr.readyState===4){
        if(xhr.status>=200&&xhr.status<300){
          var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
          el.textContent = formatWithCommas(num) || '(unavailable)';
        } else { el.textContent='(unavailable)'; }
      }};
      xhr.send(null);
    }catch(e){ el.textContent='(unavailable)'; }
  }
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function rowText(tr){ var t=''; for(var i=1;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q = String(q||''); var parts = q.split('|').map(function(s){return norm(s);}).filter(function(s){return !!s;});
      for(var i=0;i<cached.length;i++){
        var hit = !parts.length || parts.some(function(p){ return cached[i].txt.indexOf(p)>-1; });
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false);
    box.addEventListener('search', onInput, false);
    var q0=getParam('q');
    if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null, '', location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }
  function allRowCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[];
    for(var i=0;i<tb.rows.length;i++){
      var cb=tb.rows[i].querySelector('.selbox');
      if(cb) out.push(cb);
    }
    return out;
  }
  function bindGroupSync(){
    var tbl=document.getElementById('refactor-table'); if(!tbl) return;
    tbl.addEventListener('click', function(e){
      var t=e.target||e.srcElement;
      if(!(t && t.classList && t.classList.contains('selbox'))) return;
      var nm = t.getAttribute('data-name') || '';
      var checked = !!t.checked;
      var cbs = allRowCheckboxes();
      for(var i=0;i<cbs.length;i++){
        if((cbs[i].getAttribute('data-name')||'') === nm){ cbs[i].checked = checked; }
      }
      updateShowing();
    }, false);
  }
  function bindShowSelected(){
    var btn=document.getElementById('show-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs = allRowCheckboxes();
      var names = {};
      for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ names[cbs[i].getAttribute('data-name')||''] = true; } }
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var r=0;r<tb.rows.length;r++){
        var cb = tb.rows[r].querySelector('.selbox');
        var nm = cb ? (cb.getAttribute('data-name')||'') : '';
        tb.rows[r].style.display = names[nm] ? '' : 'none';
      }
      updateShowing();
    }, false);
  }
  function bindShowAll(){
    var btn=document.getElementById('show-all'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var i=0;i<tb.rows.length;i++){ tb.rows[i].style.display=''; }
      updateShowing();
    }, false);
  }
  function bindClear(){
    var btn=document.getElementById('clear-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs=allRowCheckboxes(); for(var i=0;i<cbs.length;i++) cbs[i].checked=false;
      var tbl=document.getElementById('refactor-table'); if(tbl && tbl.tBodies && tbl.tBodies[0]){
        var tb=tbl.tBodies[0]; for(var j=0;j<tb.rows.length;j++){ tb.rows[j].style.display=''; }
      }
      updateShowing();
    }, false);
  }
  function addCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
    var tb=tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i]; var cell=tr.cells[0]; var findBtn=cell ? cell.querySelector('.find-btn') : null;
      var name = findBtn ? (findBtn.getAttribute('title')||'').replace('Open a filtered view for ','') : ('Row '+(i+1));
      if(cell){
        cell.classList.add('find-cell');
        cell.innerHTML = '<input type="checkbox" class="selbox" title="Select this row" data-name="'+name.replace(/"/g,'&quot;')+'" /> ' + cell.innerHTML.replace(/^\\s*/, '');
      }
    }
  }
  function bindPrintCousinList(){
    var btn=document.getElementById('print-cousin-list'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl=document.getElementById('refactor-table'); if(!tbl) return;
      var headerHtml = ''; try{ headerHtml = tbl.tHead.innerHTML; }catch(e){}
      var tb=tbl.tBodies[0];
      var visibleRowsHtml = ''; var visibleCount = 0;
      if(tb){
        for (var i = 0; i < tb.rows.length; i++) {
          if (tb.rows[i].style.display !== 'none') {
            visibleRowsHtml += tb.rows[i].outerHTML;
            visibleCount++;
          }
        }
      }
      var css = '<style type="text/css">' +
        "body { font-size:12px; margin: 20px; }" +
        "h1 { font-size:20px; text-align:center; }" +
        "table { border-collapse:collapse; width:100%; table-layout:fixed; }" +
        "th, td { border:1px solid #999; padding: 5px 7px; vertical-align:top; text-align:left; word-wrap:break-word; }" +
        "th { background:#f0f0f0; }" +
        "a { color:#000; text-decoration:none; }" +
        "th:first-child, td:first-child { display:none; }" +
        "th:nth-child(2), td:nth-child(2) { width: 40% !important; }" +
        "th:nth-child(3), td:nth-child(3) { width: 60% !important; }" +
        '</style>';
      var link = '<link rel="stylesheet" type="text/css" href="'+(window.HEAD_LINK_URL || (typeof HEAD_LINK_URL!=='undefined'?HEAD_LINK_URL:''))+'" />';
      var tableHtml = '<table border="1">' + '<thead>' + headerHtml + '</thead><tbody>' + visibleRowsHtml + '</tbody></table>';
      var docHtml = '<html><head><title>Cousin List (Filtered)</title>' + link + css + '</head><body onload="window.print(); window.close();">' +
                    '<h1>Cousin List</h1>' +
                    '<p>Showing ' + visibleCount + ' filtered records.</p>' +
                    tableHtml +
                    '</body></html>';
      var win = window.open('', 'CousinPrint');
      win.document.open(); win.document.write(docHtml); win.document.close(); win.focus();
    }, false);
  }
  function initShowingStatic(){ try{ document.getElementById('showing-count').textContent = document.getElementById('refactor-table').tBodies[0].rows.length; }catch(e){} }
  document.addEventListener('DOMContentLoaded', function(){
    addCheckboxes();
    stampLastUpdated();
    loadAutoCount();
    bindHeaderSort();
    bindSearch();
    bindGroupSync();
    bindShowSelected();
    bindShowAll();
    bindClear();
    bindPrintCousinList();
    initShowingStatic();
  });
})();
 //]]>
</script>
</body>
</html>
""")

html_table = display_df.to_html(index=False, escape=False, classes="dataframe sortable")
html_table = html_table.replace('<table border="1" class="dataframe sortable">', '<table border="1" class="dataframe sortable" id="refactor-table">', 1)
html_table = html_table.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
html_table = html_table.replace("<th>Find</th>", '<th>Select:</th>', 1)
html_table = html_table.replace('<th>Match Summary</th>', '<th>Match Summary&amp;ndash;click to sort</th>', 1)
html_table = html_table.replace(f'<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>', '<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>', 1)

colgroup_html = (
    "<colgroup>\n"
    f"  <col style=\"width:{FIND_COL_PX}px;\" />\n"
    f"  <col style=\"width:{COL_A_PX}px;\" />\n"
    "  <col />\n"
    "</colgroup>\n"
)
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable" id="refactor-table">',
    '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html,
    1
)

final_html = page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK,
    TABLE_CSS=TABLE_CSS,
    UPDATED_BLOCK=UPDATED_BLOCK,
    DYNAMIC_BLOCK=DYNAMIC_BLOCK,
    HTML_TABLE=html_table,
    JS_COUNT_URL=JS_COUNT_URL,
    DOWNLOADS_BLOCK=DOWNLOADS_BLOCK,
    HEAD_LINK_URL=HEAD_LINK_URL
)

# ---------- 8) PARTIALS (Row-color highlight fix applies here only) ----------
def _norm_code_for_count(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = re.sub(r'\s+', ' ', t).strip()
    t = t.lower()
    return t

def _partial_css_wrapper_simple():
    return (
        HEAD_LINK +
        "<style type=\"text/css\">\n"
        "  html { scroll-behavior: smooth; }\n"
        f"  .wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px 20px 18px 20px; }}\n"
        "  a { text-decoration:none; } a:hover { text-decoration:underline; }\n"
        "  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }\n"
        "  .meta { text-align:center; font-size:12px; color:#555; margin:2px 0 10px 0; }\n"
        "  .toolbar { display:flex; gap:10px; align-items:center; margin:6px 0 10px 0; flex-wrap:wrap; justify-content:center; }\n"
        "  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px; text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; }\n"
        "  .btn:disabled { opacity:0.5; cursor:not-allowed; }\n"
        "  table { border-collapse:collapse; width:100%; table-layout:auto; }\n"
        "  th, td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; text-align:left; }\n"
        "  th { background:#e3eaf8; position:sticky; top:0; z-index:2; }\n"
        "  /* Row selection highlight FIX (partials only) */\n"
        "  tbody tr.sel td { background:#fff7d6 !important; }\n"
        "  tbody tr:hover td { background:#f9f6e8; }\n"
        "  .count a { font-weight:bold; }\n"
        "</style>\n"
    )

def _partial_head(title):
    return (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
        "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n<head>\n"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        f"<title>{_html.escape(title)}</title>\n"
    )

def _partial_toolbar():
    safe_home = HOME_URL.replace('"','&quot;')
    return (
        "<div class=\"toolbar\">"
        f"<a class=\"btn\" href=\"{safe_home}\" target=\"_blank\" rel=\"noopener\">DNA Register</a>"
        " <button id=\"mc-show-selected\" class=\"btn\" title=\"Open DNA Register filtered to selected\">Show Selected</button>"
        " <button id=\"mc-show-all\" class=\"btn\" title=\"Show all rows (this table)\">Show All</button>"
        " <button id=\"mc-reset\" class=\"btn\" title=\"Clear selection and show all\">Reset</button>"
        " <button id=\"view\" class=\"btn\" title=\"Open DNA Register with selected (alias)\">View Now</button>"
        "</div>\n"
    )

def _partial_js_common():
    _safe_home = HOME_URL.replace("'", "%27")
    safe_count = COUNT_PUBLIC_URL.replace("'", "%27")
    return (
        "<script type=\"text/javascript\">\n//<![CDATA[\n"
        "(function(){\n"
        "  var REG = '" + _safe_home + "';\n"
        "  function fmt(n){ try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return '0'; return x.toLocaleString('en-US'); }catch(e){ return String(n||'0'); } }\n"
        "  function selected(){ var out=[]; var tb=document.getElementById('ref-tb'); if(!tb) return out; var rows=tb.rows; for(var i=0;i<rows.length;i++){ if((' '+rows[i].className+' ').indexOf(' sel ')>-1) out.push(rows[i]); } return out; }\n"
        "  function update(){ var sel=selected(), sum=0; for(var i=0;i<sel.length;i++){ var v=parseInt((sel[i].getAttribute('data-count')||'0').replace(/[^0-9\\-]/g,''),10); if(!isNaN(v)) sum+=v; } var nEl=document.getElementById('sel-n'); var sEl=document.getElementById('sel-sum'); if(nEl) nEl.innerHTML=fmt(sel.length); if(sEl) sEl.innerHTML=fmt(sum); }\n"
        "  function qJoin(parts){ var out=[]; var seen={}; for(var i=0;i<parts.length;i++){ var p=String(parts[i]||''); if(p && !seen[p]){ seen[p]=1; out.push(encodeURIComponent(p)); } } return out.join('%7C'); }\n"
        "  function openRegisterForSelected(){ var sel=selected(); if(!sel.length) return; var qs=[]; for(var i=0;i<sel.length;i++){ qs.push(sel[i].getAttribute('data-q')||''); } var q = qJoin(qs); var url = REG + '?q=' + q; var w=null; try{ w=window.open(url,'RegisterFiltered'); if(!w) throw new Error('popup'); w.focus(); } catch(e){ window.location.href = url; } }\n"
        "  function toggleFrom(el){ var tr=el; while(tr && tr.nodeName && tr.nodeName.toLowerCase()!=='tr'){ tr=tr.parentNode; } if(!tr) return; var c=tr.className||''; tr.className = ((' '+c+' ').indexOf(' sel ')>-1) ? c.replace(/\\bsel\\b/,'').replace(/\\s{2,}/g,' ').trim() : (c?c+' ':'')+'sel'; update(); }\n"
        "  document.addEventListener('click', function(e){ var t=e.target||e.srcElement; if(!t) return; if(t.classList && t.classList.contains('count-pick')){ e.preventDefault(); toggleFrom(t); return; } if(t.id=='view' || t.id=='mc-show-selected'){ e.preventDefault(); openRegisterForSelected(); return; } if(t.id=='mc-reset'){ e.preventDefault(); var tb=document.getElementById('ref-tb'); if(tb){ var rows=tb.rows; for(var i=0;i<rows.length;i++){ rows[i].className = rows[i].className.replace(/\\bsel\\b/,'').replace(/\\s{2,}/g,' ').trim(); rows[i].style.display=''; } } update(); return; } if(t.id=='mc-show-all'){ e.preventDefault(); var tb2=document.getElementById('ref-tb'); if(!tb2) return; for(var k=0;k<tb2.rows.length;k++){ tb2.rows[k].style.display=''; } return; } }, false);\n"
        "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date()); function z(n){return (n<10?'0':'')+n;} el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}\n"
        "  function load(){var el=document.getElementById('auto-count'); if(!el) return; var URL='" + safe_count + "'; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true); xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent=(m?m[1]:'');} else {el.textContent='(unavailable)';}}}; xhr.send(null);}catch(e){el.textContent='(unavailable)';}}\n"
        "  document.addEventListener('DOMContentLoaded', function(){ stamp(); load(); update(); }, false);\n"
        "})();\n"
        "//]]>\n</script>\n"
    )

def _shell_open(title):
    return (
        _partial_head(title) +
        _partial_css_wrapper_simple() +
        "</head>\n<body>\n<div class=\"wrap\">\n" +
        f"<h1>{_html.escape(title)}</h1>\n" +
        "<div class=\"meta\">"
        "Last updated: <span id=\"last-updated\"></span>"
        " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\">&mdash;</span>"
        " &nbsp;|&nbsp; Selected: <span id=\"sel-n\">0</span> &nbsp; Sum: <span id=\"sel-sum\">0</span>"
        "</div>\n" +
        _partial_toolbar() +
        "<div class=\"table-scroll\">\n"
    )

def _shell_close():
    return (
        "</div>\n</div>\n" +
        _partial_js_common() +
        "</body>\n</html>"
    )

def _html_escape_text(s):
    return _html.escape(str(s), quote=True)

def build_match_count_partial(main_df: pd.DataFrame) -> str:
    codes_raw = main_df[match_col].astype(str).map(lambda x: x.strip())
    keys_norm = codes_raw.map(_norm_code_for_count)
    counts_series = keys_norm.value_counts(dropna=False)
    counts = counts_series.reset_index()
    if counts.shape[1] >= 2:
        counts.columns = ["norm_key", "Count"]
    else:
        counts["norm_key"] = counts.index.astype(str)
        counts["Count"] = counts_series.values
        counts = counts[["norm_key","Count"]]
    first_display = {}
    for code_disp, k in zip(codes_raw.tolist(), keys_norm.tolist()):
        if k not in first_display and str(k) != "":
            first_display[k] = code_disp
    counts["Code"] = counts["norm_key"].map(lambda k: first_display.get(k, k))
    counts["Unmasked"] = counts["norm_key"].map(lambda k: MATCH_TO_UNMASKED.get(k, ""))
    counts = counts.sort_values(by=["Code","Count"], ascending=[True, False], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_shell_open("Match Count"))
    html.append('<table id="ref-table" border="1" class="sortable" style="width:100%"><thead><tr>')
    html.append('<th style="width:35%">Code</th><th style="width:45%">Unmasked</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in counts.iterrows():
        code = r.get("Code","")
        unm  = r.get("Unmasked","")
        cnt  = int(str(r.get("Count","0")).strip() or "0")
        label = (unm or code).strip()
        tr = (
            f'<tr data-q="{_html_escape_text(label)}" data-count="{cnt}">'
            f'<td>{_html_escape_text(code)}</td>'
            f'<td>{_html_escape_text(unm)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td>'
            f'</tr>'
        )
        html.append(tr)
    html.append('</tbody></table>')
    html.append(_shell_close())
    return "".join(html)

def build_lineage_count_partial(main_df: pd.DataFrame) -> str:
    first_series = main_df.get("First Ancestor", pd.Series(dtype=str)).astype(str).map(lambda x: x.strip())
    vc = first_series[first_series != ""].value_counts(dropna=False)
    lin_df = vc.reset_index()
    if lin_df.shape[1] >= 2:
        lin_df.columns = ["First Ancestor","Count"]
    else:
        lin_df["First Ancestor"] = lin_df.index.astype(str)
        lin_df["Count"] = vc.values
        lin_df = lin_df[["First Ancestor","Count"]]
    lin_df = lin_df.sort_values(["Count","First Ancestor"], ascending=[False, True], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_shell_open("Lineage Count"))
    html.append('<table id="ref-table" border="1" class="sortable" style="width:100%"><thead><tr>')
    html.append('<th style="width:80%">First Ancestor</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in lin_df.iterrows():
        first = str(r.get("First Ancestor","")).strip()
        cnt   = int(str(r.get("Count","0")).strip() or "0")
        tr = (
            f'<tr data-q="{_html_escape_text(first)}" data-count="{cnt}">'
            f'<td>{_html.escape(first)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td>'
            f'</tr>'
        )
        html.append(tr)
    html.append('</tbody></table>')
    html.append(_shell_close())
    return "".join(html)

def build_and_write_partials(main_df: pd.DataFrame):
    _setup_resolver()
    os.makedirs("partials", exist_ok=True)

    mc_html = build_match_count_partial(main_df)
    mc_local = os.path.join("partials", "match_count.htm")
    with open(mc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(mc_html)
    print("[OK] Wrote partial:", mc_local)

    lc_html = build_lineage_count_partial(main_df)
    lc_local = os.path.join("partials", "lineage_count.htm")
    with open(lc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(lc_html)
    print("[OK] Wrote partial:", lc_local)

    cousin_df = main_df[["Match Summary"]].copy()
    cousin_df = cousin_df.sort_values(by="Match Summary", ascending=True, kind="mergesort").reset_index(drop=True)
    cousin_rows = ['<table border="1" id="refactor-table"><thead><tr><th>Match Summary</th></tr></thead><tbody>']
    for v in cousin_df["Match Summary"].tolist():
        cousin_rows.append(f"<tr><td>{v}</td></tr>")
    cousin_rows.append("</tbody></table>")
    cousin_html = (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\"><head>"
        f"{HEAD_LINK}"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />"
        "<title>Cousin List (Printable)</title>"
        "<style type=\"text/css\"> body{font-size:12px;margin:20px;} h1{text-align:center;font-size:20px;} table{border-collapse:collapse;width:100%;} th,td{border:1px solid #999;padding:5px 7px;vertical-align:top;text-align:left;} th{background:#f0f0f0;} a{text-decoration:none;} </style>"
        "</head><body onload=\"window.print();\">"
        "<h1>Cousin List (Printable)</h1>" + "".join(cousin_rows) +
        "</body></html>"
    )
    cl_local = os.path.join("partials", "cousin_list_print.htm")
    with open(cl_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(cousin_html)
    print("[OK] Wrote partial:", cl_local)

    return mc_local, lc_local, cl_local

# Build partials + main page
PARTIAL_MATCH_LOCAL, PARTIAL_LINEAGE_LOCAL, PARTIAL_COUSIN_LOCAL = build_and_write_partials(df)

def build_register_html_for_abs(remote_abs_path: str) -> str:
    q_links = []
    subs = df["Subject"].astype(str).tolist()
    for subject_name in subs:
        q = _u.quote(subject_name)
        q_links.append(
            f'<a class="find-btn" href="{remote_abs_path}?q={q}" target="_blank" rel="noopener" '
            f'title="Open a filtered view for {subject_name}">Find</a>'
        )
    df_plus = df.copy()
    df_plus["Find"] = q_links
    disp_plus = df_plus[["Find","Match Summary", LINEAGE_HEADER_SAFE]]
    tbl = disp_plus.to_html(index=False, escape=False, classes="dataframe sortable")
    tbl = tbl.replace('<table border="1" class="dataframe sortable">', '<table border="1" class="dataframe sortable" id="refactor-table">', 1)
    tbl = tbl.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
    tbl = tbl.replace("<th>Find</th>", '<th>Select:</th>', 1)
    tbl = tbl.replace("<th>Match Summary</th>", '<th>Match Summary&amp;ndash;click to sort</th>', 1)
    tbl = tbl.replace(f"<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>", "<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>", 1)
    colgroup_html_local = (
        "<colgroup>\n"
        f"  <col style=\"width:{FIND_COL_PX}px;\" />\n"
        f"  <col style=\"width:{COL_A_PX}px;\" />\n"
        "  <col />\n"
        "</colgroup>\n"
    )
    tbl = tbl.replace(
        '<table border="1" class="dataframe sortable" id="refactor-table">',
        '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html_local, 1
    )
    return page_tpl.safe_substitute(
        HEAD_LINK=HEAD_LINK,
        TABLE_CSS=TABLE_CSS,
        UPDATED_BLOCK=UPDATED_BLOCK,
        DYNAMIC_BLOCK=DYNAMIC_BLOCK,
        HTML_TABLE=tbl,
        JS_COUNT_URL=JS_COUNT_URL,
        DOWNLOADS_BLOCK=DOWNLOADS_BLOCK,
        HEAD_LINK_URL=HEAD_LINK_URL
    )

os.makedirs("partials", exist_ok=True)
final_html_plus = build_register_html_for_abs(HOME_URL)

with open(LOCAL_HTML, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html)
print("[OK] Saved canonical render:", os.path.abspath(LOCAL_HTML))

with open(WORK_PLUS_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html_plus)
print("[OK] Saved:", os.path.abspath(WORK_PLUS_LOCAL), "(partials clone)")

# ---------- 9) Uploads ----------
def save_and_upload_all():
    if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
        print("[SKIP] Missing FTP creds; uploads skipped.")
        return
    try:
        ftps = ftp_connect()

        try:
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_CANON))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_LEG))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_SIMPLE))
        except Exception as e:
            print("[WARN] Upload main HTML failed:", e)

        try:
            if os.path.exists(LOCAL_CSV):
                ftp_upload_overwrite(ftps, LOCAL_CSV, _remote_path(REMOTE_CSV))
            if os.path.exists(LOCAL_XLSX):
                ftp_upload_overwrite(ftps, LOCAL_XLSX, _remote_path(REMOTE_XLSX))
            print("[OK] Uploaded CSV/XLSX -> /partials/")
        except Exception as e:
            print("[WARN] Upload CSV/XLSX failed:", e)

        if os.path.exists(LOCAL_COUNT_FILE):
            try:
                ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, _remote_path(REMOTE_COUNT_NAME))
            except Exception as e:
                print("[WARN] Upload autosomal count failed:", e)

        try:
            ftp_upload_overwrite(ftps, os.path.join("partials","match_count.htm"),       _remote_path(posixpath.join("partials","match_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","lineage_count.htm"),     _remote_path(posixpath.join("partials","lineage_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","cousin_list_print.htm"), _remote_path(posixpath.join("partials","cousin_list_print.htm")))
        except Exception as e:
            print("[WARN] Upload partials failed:", e)

        try:
            ftp_upload_overwrite(ftps, WORK_PLUS_LOCAL, _remote_path(WORK_PLUS_REMOTE))
        except Exception as e:
            print("[WARN] Upload work_plus.htm failed:", e)

        print("\n--- SIZE Verification (if supported) ---")
        for p in [
            _remote_path(REMOTE_HTML_CANON),
            _remote_path(REMOTE_HTML_LEG),
            _remote_path(REMOTE_HTML_SIMPLE),
            _remote_path(REMOTE_CSV),
            _remote_path(REMOTE_XLSX),
            _remote_path(posixpath.join("partials","match_count.htm")),
            _remote_path(posixpath.join("partials","lineage_count.htm")),
            _remote_path(posixpath.join("partials","cousin_list_print.htm")),
            _remote_path(WORK_PLUS_REMOTE),
        ]:
            try:
                sz = ftp_size(ftps, p)
                print(f"{p} : {sz if sz is not None else '(SIZE unsupported)'}")
            except Exception:
                print(f"{p} : (check skipped)")

        try: ftps.quit()
        except Exception: pass

        print("\n--- Open URLs ---")
        print("Canonical (Find target):          https://yates.one-name.net/partials/yates_ancestor_register.htm")
        print("Legacy clone (same content):      https://yates.one-name.net/partials/ons_yates_dna_register.htm")
        print("JUSTDNA simple alias:             https://yates.one-name.net/partials/justdna.htm")
        print("Trees page:                       https://yates.one-name.net/partials/just-trees.htm")
        print("Match Count:                      https://yates.one-name.net/partials/match_count.htm")
        print("Lineage Count:                    https://yates.one-name.net/partials/lineage_count.htm")
        print("Cousin List (Printable):          https://yates.one-name.net/partials/cousin_list_print.htm")
        print("Work+ clone:                      https://yates.one-name.net/partials/work_plus.htm")

        print("\nIf anything looks cached, hard-refresh (Ctrl/Cmd+Shift+R) or append ?v=1 once.")
    except Exception as e:
        print("[FAIL] FTP session:", e)
        traceback.print_exc()

# ---------- 10) Upload ----------
save_and_upload_all()
# ====== CUT STOP [1/1] CELL 2 ===================================================================



[OK] Loaded CSV: 7 rows, 6 cols
[PULL] match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
[OK] Resolver loaded: 79 codes
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
[OK] Saved canonical render: /content/yates_ancestor_register.htm
[OK] Saved: /content/partials/work_plus.htm (partials clone)
[PUT]  yates_ancestor_register.htm -> partials/yates_ancestor_register.htm
[PUT]  yates_ancestor_register.htm -> partials/ons_yates_dna_register.htm
[PUT]  yates_ancestor_register.htm -> partials/justdna.htm
[PUT]  yates_ancestor_register.csv -> partials/yates_ancestor_register.csv
[PUT]  yates_ancestor_register.xlsx -> partials/yates_ancestor_register.xlsx
[OK] Uploaded CSV/XLSX -> /partials/
[PUT]  /content/autosomal_count.txt -> autosomal_count.txt
[PUT]  partials/match_count.htm -> partial

# Cell 3

In [58]:
# ====== CUT START [1/6] RULES + IMPORTS + SECRETS ==============================================
# RON RULES — QUICK CODE CARD (v2025.11.05-AncReg-Exports+Partials-JustTrees-NavDNA)
# 1) EXECUTION: Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# 2) PUNCTUATION IN STRINGS: Use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# 3) CONTENT: Deliver full runnable code (no snippets). No fabrication or inference.
# 4) Python code (inline, executable, full COLAB Cell paste-ready section); ISO-8859-15 (ASCII-only in source).
# 5) HTML: XHTML 1.0 Transitional style acceptable; avoid HTML5-only tags if not needed.
# 6) INTEGRITY: Work in CUT-ready sections only; exactly five # lines after each section.

# Core imports
import os, re, socket, posixpath, traceback
import pandas as pd
import html as _html

# Time imports (fix for ZoneInfo)
from datetime import datetime
try:
    from zoneinfo import ZoneInfo  # Python 3.9+
except Exception:
    ZoneInfo = None

# FTPS
from ftplib import FTP_TLS
# ====== CUT STOP  [1/6] RULES + IMPORTS + SECRETS ===============================================
#####
#####
#####
#####
#####



# ====== CUT START [2/6] SECRETS + LOAD DATA + COUNTS + PATHS ===================================
# --- Securely load secrets (Colab or env) ---
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_DIR  = os.environ.get('FTP_DIR', '').strip().strip('/')
COUNT_PUBLIC_URL = ("/%s/%s" % (FTP_DIR, "autosomal_count.txt")) if FTP_DIR else "/autosomal_count.txt"

# Inputs
INPUT_CSV   = "final_combined_df_with_value_labels.csv"

# OUTPUT html filename stays just-trees.htm
OUTPUT_NAME = "just-trees.htm"

# Button target (new nav button)
DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"

# Export names for CSV/XLSX (mirror visible table); served from /partials/
EXPORT_BASE = "yates_ancestor_register"
LOCAL_CSV   = EXPORT_BASE + ".csv"
LOCAL_XLSX  = EXPORT_BASE + ".xlsx"
REMOTE_CSV  = posixpath.join("partials", LOCAL_CSV)
REMOTE_XLSX = posixpath.join("partials", LOCAL_XLSX)

# Upload the HTML itself into /partials/ using the OUTPUT_NAME
REMOTE_HTML = posixpath.join("partials", OUTPUT_NAME)

# Load CSV (robust encodings)
df = None
_last_err = None
for enc in ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1"):
    try:
        df = pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=False, encoding=enc)
        break
    except Exception as e:
        _last_err = e
        df = None
if df is None:
    raise SystemExit("[ERROR] Unable to read CSV: %s (%r)" % (INPUT_CSV, _last_err))
print("[OK] Loaded CSV:", INPUT_CSV, "rows=%d, cols=%d" % (len(df), len(df.columns)))

# Normalize haplogroup column presence
if 'haplogroup' not in df.columns:
    df['haplogroup'] = ''
else:
    df['haplogroup'] = df['haplogroup'].fillna('')

# Read autosomal count locally if present (fallback display only)
autosomal_count = None
try:
    with open("autosomal_count.txt", "r") as f:
        autosomal_count = int(re.findall(r"(\d+)", f.read() or "")[0])
except Exception:
    pass

# Optional delta vs previous run
prev_count, additional_str = None, ""
if os.path.exists("autosomal_count_prev.txt"):
    try:
        with open("autosomal_count_prev.txt", "r") as f:
            prev_count = int(re.findall(r"(\d+)", f.read() or "")[0])
        if autosomal_count is not None and prev_count is not None:
            diff = autosomal_count - prev_count
            if diff != 0:
                additional_str = " (+%d since last run)" % diff
    except Exception:
        pass

# Human-readable fallback timestamp (JS will stamp dynamically at runtime)
try:
    _tz = ZoneInfo("America/New_York") if ZoneInfo else datetime.now().astimezone().tzinfo
except Exception:
    _tz = datetime.now().astimezone().tzinfo
now = datetime.now(_tz)
updated_fallback = now.strftime("%Y-%m-%d %H:%M")
# ====== CUT STOP  [2/6] SECRETS + LOAD DATA + COUNTS + PATHS ====================================
#####
#####
#####
#####
#####



# ====== CUT START [3/6] MAP COLUMN B (masked code) -> COLUMN C (unmasked name) =================
# Column letters in MAIN df:
#   A = ID#
#   B = match to (masked)
#   C = Unmasked Name (output)

A_IDX = 0
B_IDX = 1
C_IDX = 2

def _norm_code(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = t.replace("\u00a0", " ").strip()
    t = re.sub(r"\s{2,}", " ", t)
    return t.lower()

# Resolver lives on server under /partials/
REMOTE_PATH = "partials/match_to_unmasked.csv"
LOCAL_PATH  = "match_to_unmasked.csv"

# Pull resolver if not present
if not os.path.exists(LOCAL_PATH):
    print("Pulling resolver CSV from server...")
    with FTP_TLS(timeout=30) as ftps:
        ftps.connect(os.environ.get("FTP_HOST",""), int(os.environ.get("FTP_PORT","21")))
        ftps.login(os.environ.get("FTP_USER",""), os.environ.get("FTP_PASS",""))
        try: ftps.prot_p()
        except Exception: pass
        try: ftps.set_pasv(True)
        except Exception: pass
        if FTP_DIR:
            for p in FTP_DIR.split("/"):
                if not p: continue
                try: ftps.cwd(p)
                except Exception:
                    try: ftps.mkd(p)
                    except Exception: pass
                    ftps.cwd(p)
        try: ftps.cwd("partials")
        except Exception: pass
        with open(LOCAL_PATH, "wb") as f:
            ftps.retrbinary("RETR match_to_unmasked.csv", f.write)
    print("Resolver saved:", os.path.abspath(LOCAL_PATH))
else:
    print("Using cached resolver:", os.path.abspath(LOCAL_PATH))

def _load_resolver(path):
    last_err = None
    m = None
    for enc in ("utf-8-sig","iso-8859-15","utf-8","cp1252","latin1"):
        try:
            m = pd.read_csv(path, dtype=str, keep_default_na=False, encoding=enc)
            break
        except Exception as e:
            last_err = e
            m = None
    if m is None:
        raise RuntimeError("Unable to read resolver CSV: %s (%r)" % (path, last_err))
    cols = {c.lower(): c for c in m.columns}
    if "code" not in cols or "unmasked" not in cols:
        raise ValueError("Resolver CSV must have columns: code, unmasked")
    m = m[[cols["code"], cols["unmasked"]]].copy()
    m["__key__"] = m[cols["code"]].map(_norm_code)
    m["__val__"] = m[cols["unmasked"]].astype(str)
    m = m.drop_duplicates(subset="__key__", keep="first")
    return dict(zip(m["__key__"], m["__val__"]))

resolver_map = _load_resolver(LOCAL_PATH)

if df.shape[1] < 3:
    raise ValueError("Main df must have at least 3 columns: A(ID#), B(match to), C(unmasked).")

masked_raw = df.iloc[:, B_IDX].astype(str)
masked_key = masked_raw.map(_norm_code)
resolved   = masked_key.map(resolver_map)

df.iloc[:, C_IDX] = resolved.fillna("")

mapped = int(resolved.notna().sum())
total  = len(df)
print("[OK] Column B -> C mapping:", mapped, "/", total, "unmatched:", total - mapped)
# ====== CUT STOP  [3/6] MAP COLUMN B (masked code) -> COLUMN C (unmasked name) =================
#####
#####
#####
#####
#####



# ====== CUT START [4/6] XHTML TEMPLATE + TABLE + DOWNLOAD LINKS ================================
_BTN_BG   = "#5b79b8"
_BTN_BG_H = "#4668aa"
_TH_BG    = "#e3eaf8"
_LINK     = "#154b8b"

# Fallback number text for initial render; JS will overwrite with live values
auto_text = "Unknown" if autosomal_count is None else str(autosomal_count)

# Download links block points to /partials/{csv,xlsx}
DOWNLOADS_BLOCK = (
    "<p style=\"text-align:center; margin:4px 0 10px 0; font-size:13px;\">"
    "Download: "
    "<a href=\"/partials/%s\">CSV</a> | "
    "<a href=\"/partials/%s\">Excel</a>"
    "</p>" % (_html.escape(LOCAL_CSV), _html.escape(LOCAL_XLSX))
)

full_html_template = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Ancestor Register</title>
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { margin:0; padding:0; font-family: "Times New Roman", Georgia, serif; background:#ffffff; color:#222; font-size:14px; }
  a { color:%(LINK)s; text-decoration:none; } a:hover { text-decoration:underline; }

  .intro { padding:20px; text-align:center; }
  .intro h2 { margin:0 0 6px 0; font-size:24px; line-height:1.2; }
  .meta { font-size:12px; color:#555; margin:4px 0 8px 0; display:inline-block; }

  .toolbar { margin:6px auto 12px auto; display:flex; flex-wrap:wrap; gap:6px; justify-content:center; }
  .btn { display:inline-block; border:1px solid %(BTN_BG)s; background:%(BTN_BG)s; color:#fff;
         padding:4px 9px; border-radius:6px; font-size:13px; line-height:1.2; text-decoration:none;
         cursor:pointer; user-select:none; transition:background 0.2s, transform 0.1s; }
  .btn:hover { background:%(BTN_BG_H)s; transform:translateY(-1px); }
  .btn.light { background:#ffffff; color:#111; border-color:#bbb; }

  .output-table { max-height:75vh; overflow:auto; border:1px solid #ddd; margin:0 20px 24px 20px; }

  table.sortable { width:100%%; border-collapse:collapse; min-width:720px; table-layout:auto; }
  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; background:#ffffff; white-space:nowrap; }
  table.sortable th { position:sticky; top:0; z-index:2; text-align:left; cursor:pointer; background:%(TH_BG)s; box-shadow:0 1px 0 #ccc; }
  table.sortable tr#first-row td { border-top:2px solid #999 !important; }

  #searchBox { padding:4px 8px; font-size:13px; border:1px solid #bbb; border-radius:6px; outline:none; }

  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px;
                 border:1px solid %(BTN_BG)s; background:%(BTN_BG)s; color:#fff;
                 border-radius:6px; font-size:12px; display:none; z-index:9999; cursor:pointer; }
  .back-to-top:hover { background:%(BTN_BG_H)s; }

  @media screen and (max-width: 820px) {
    .intro { padding:14px; }
    .output-table { margin:0 12px 20px 12px; }
    .intro h2 { font-size:20px; }
    table.sortable { min-width:560px; }
  }
</style>
<script type="text/javascript">
//<![CDATA[
// Basic helpers
function _cellText(cell){
  var t = (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').replace(/^\\s+|\\s+$/g,'').toLowerCase();
  return t;
}
function _asNumber(s){
  var m = (s||'').replace(/[^0-9.\\-]/g,'');
  if(m.length===0) return NaN;
  var v = parseFloat(m);
  return isNaN(v) ? NaN : v;
}

// Sorting
function sortTableByColumn(tbl, colIndex, dirAsc){
  if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var tb = tbl.tBodies[0];
  var rows = Array.prototype.slice.call(tb.rows || []);
  rows.sort(function(a,b){
    var A = _cellText(a.cells[colIndex] || null);
       var B = _cellText(b.cells[colIndex] || null);
    var nA = _asNumber(A), nB = _asNumber(B);
    if(!isNaN(nA) && !isNaN(nB)){ return dirAsc ? (nA - nB) : (nB - nA); }
    if(A < B) return dirAsc ? -1 : 1;
    if(A > B) return dirAsc ?  1 : -1;
    return 0;
  });
  var frag = document.createDocumentFragment();
  for(var i=0;i<rows.length;i++){ frag.appendChild(rows[i]); }
  tb.appendChild(frag);
  updateShowingCount();
}
function bindHeaderSort(){
  var tbl = document.getElementById('refactor-table');
  if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
  var ths = tbl.tHead.rows[0].cells || [];
  for(var i=0;i<ths.length;i++){
    (function(idx){
      var th = ths[idx];
      var dirAsc = true;
      th.addEventListener('click', function(){
        for(var j=0;j<ths.length;j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+[\\u2191\\u2193]$/, ''); }
        sortTableByColumn(tbl, idx, dirAsc);
        th.innerHTML = th.innerHTML.replace(/\\s+[\\u2191\\u2193]$/, '') + (dirAsc ? ' \\u2191' : ' \\u2193');
        dirAsc = !dirAsc;
      }, false);
    })(i);
  }
}

// Filter + live showing count
function filterTable(){
  var q = (document.getElementById('searchBox').value || '').toLowerCase();
  var tbl = document.getElementById('refactor-table');
  if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var rows = tbl.tBodies[0].rows || [];
  for(var i=0;i<rows.length;i++){
    var cells = rows[i].cells, hit=false;
    for(var j=0;j<cells.length;j++){
      var txt = (cells[j].textContent || cells[j].innerText || '').toLowerCase();
      if(txt.indexOf(q) > -1){ hit=true; break; }
    }
    rows[i].style.display = hit ? '' : 'none';
  }
  updateShowingCount();
}
function updateShowingCount(){
  var el = document.getElementById('showing-count');
  var tbl = document.getElementById('refactor-table');
  if(!(el && tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var rows = tbl.tBodies[0].rows || [];
  var vis = 0;
  for(var i=0;i<rows.length;i++){ if(rows[i].style.display !== 'none') vis++; }
  el.textContent = vis;
}

// Dynamic stamps
function z(n){ return (n<10 ? '0' : '') + n; }
function stampLastUpdated(){
  var el = document.getElementById('last-updated'); if(!el) return;
  var d  = new Date(document.lastModified || new Date());
  var opts = {
    year: 'numeric',
    month: 'long',
    day: 'numeric',
    hour: '2-digit',
    minute: '2-digit',
    hour12: false
  };
  var formatted = d.toLocaleString('en-US', opts).replace(',', '');
  el.innerHTML = formatted;
}
  // Date stamp: "D Month YYYY"
  function stampLastUpdated(){
    var el = document.getElementById('last-updated'); if(!el) return;
    var d  = new Date(document.lastModified || new Date());
    var months = ['January','February','March','April','May','June','July','August','September','October','November','December'];
    el.innerHTML = d.getDate() + ' ' + months[d.getMonth()] + ' ' + d.getFullYear();
  }

function formatWithCommas(n){
  try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
}
function loadAutoCount(){
  var el=document.getElementById('auto-count'); if(!el) return;
  var url='{COUNT_URL}';
  try{
    var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
    xhr.onreadystatechange=function(){ if(xhr.readyState===4){
      if(xhr.status>=200&&xhr.status<300){
        var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
        el.textContent = formatWithCommas(num) || '(unavailable)';
      } else { el.textContent='(unavailable)'; }
    }};
    xhr.send(null);
  }catch(e){ el.textContent='(unavailable)'; }
}

// Back-to-top
function bindBackToTop(){
  var btn = document.getElementById('back-to-top');
  if(!btn) return;
  function toggle(){ btn.style.display = (window.scrollY > 200) ? 'block' : 'none'; }
  toggle(); window.addEventListener('scroll', toggle, {passive:true});
  btn.addEventListener('click', function(){ window.scrollTo(0,0); }, false);
}

document.addEventListener('DOMContentLoaded', function(){
  bindHeaderSort();
  bindBackToTop();
  stampLastUpdated();
  loadAutoCount();
  updateShowingCount();
}, false);
//]]>
</script>
</head>
<body>
  <div class="intro">
    <h2>Ancestor Register</h2>
    <div class="meta">
      Last updated: <span id="last-updated">%(UPDATED)s</span>
      &nbsp;|&nbsp; Autosomal matches: <span id="auto-count">%(AUTO)s</span>
      &nbsp;|&nbsp; Showing: <span id="showing-count">0</span>
    </div>
    %(DL)s
    <div class="toolbar">
      <a class="btn" href="%(DNA_ABS)s" target="_blank" rel="noopener">DNA Register</a>
      <input type="text" id="searchBox" class="btn light" placeholder="Search this page&hellip;" oninput="filterTable()" />
    </div>
  </div>

  <div class="output-table" id="table-container">
    <!-- TABLE_PLACEHOLDER -->
  </div>

  <div class="back-to-top" id="back-to-top">&#9650; Top</div>
</body>
</html>""" % {
    "BTN_BG": _BTN_BG, "BTN_BG_H": _BTN_BG_H, "TH_BG": _TH_BG, "LINK": _LINK,
    "UPDATED": _html.escape(updated_fallback),
    "AUTO": _html.escape("Unknown" if autosomal_count is None else str(autosomal_count)),
    "DL": DOWNLOADS_BLOCK,
    "DNA_ABS": DNA_REGISTER_ABS
}

# Build table HTML and mark first row
table_html = df.to_html(index=False, border=1, classes="sortable", table_id="refactor-table")
table_html = table_html.replace("<tbody>\n<tr>", "<tbody>\n<tr id=\"first-row\">", 1)

# Inject table and JS count URL
final_html = full_html_template.replace("<!-- TABLE_PLACEHOLDER -->", table_html)
final_html = final_html.replace("{COUNT_URL}", COUNT_PUBLIC_URL)

# Build export DataFrame mirroring the visible table order (use current df as-is)
export_df = df.copy()

# Save CSV (ISO-8859-15) and XLSX
export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    with pd.ExcelWriter(LOCAL_XLSX) as _writer:
        export_df.to_excel(_writer, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))
# ====== CUT STOP  [4/6] XHTML TEMPLATE + TABLE + DOWNLOAD LINKS ================================
#####
#####
#####
#####
#####



# ====== CUT START [5/6] SAVE LOCALLY + FTP UPLOAD (HTML + CSV/XLSX -> /partials) ==============
# Save locally (iso-8859-15 safe)
try:
    with open(OUTPUT_NAME, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(final_html)
    print("[OK] Saved locally:", OUTPUT_NAME)
except Exception as e:
    print("[ERROR] Saving local file failed:", e)
    traceback.print_exc()

# Upload if credentials exist
ftp_host = os.environ.get('FTP_HOST')
ftp_user = os.environ.get('FTP_USER')
ftp_pass = os.environ.get('FTP_PASS')
ftp_port = os.environ.get('FTP_PORT', '21')
ftp_dir  = os.environ.get('FTP_DIR', '')

def _ftps_ensure_dir(ftps, name):
    if not name: return
    for p in [p for p in name.split('/') if p]:
        try:
            ftps.cwd(p)
        except Exception:
            try:
                ftps.mkd(p)
            except Exception:
                pass
            ftps.cwd(p)

if all([ftp_host, ftp_user, ftp_pass]):
    print("[INFO] Attempting FTP upload...")
    try:
        socket.setdefaulttimeout(30)
        with FTP_TLS(timeout=30) as ftps:
            ftps.connect(ftp_host, int(ftp_port))
            ftps.login(ftp_user, ftp_pass)
            try: ftps.prot_p()
            except Exception: pass
            try: ftps.set_pasv(True)
            except Exception: pass

            # Navigate to base dir
            _ftps_ensure_dir(ftps, ftp_dir.strip('/'))

            # Ensure /partials exists then upload HTML + CSV/XLSX there
            try:
                _ftps_ensure_dir(ftps, "partials")

                # Upload HTML into /partials/ as just-trees.htm
                with open(OUTPUT_NAME, "rb") as fh:
                    ftps.storbinary("STOR " + os.path.basename(REMOTE_HTML), fh)
                print("[OK] Uploaded HTML to /partials/:", OUTPUT_NAME, "->", os.path.basename(REMOTE_HTML))

                # Upload CSV/XLSX into /partials/
                with open(LOCAL_CSV, "rb") as fh:
                    ftps.storbinary("STOR " + os.path.basename(REMOTE_CSV), fh)
                with open(LOCAL_XLSX, "rb") as fh:
                    ftps.storbinary("STOR " + os.path.basename(REMOTE_XLSX), fh)
                print("[OK] Uploaded exports to /partials/:", LOCAL_CSV, LOCAL_XLSX)

                print("Open URL: https://yates.one-name.net/partials/just-trees.htm")
            except Exception as e:
                print("[ERROR] Upload to /partials/ failed:", e)

            print("[OK] Uploads complete.")
    except Exception as e:
        print("[ERROR] FTP session failed:", e)
        traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload: missing FTP credentials.")
# ====== CUT STOP  [5/6] SAVE LOCALLY + FTP UPLOAD (HTML + CSV/XLSX -> /partials) ==============
#####
#####
#####
#####
#####



# ====== CUT START [6/6] PERSIST COUNT + DONE ====================================================
if autosomal_count is not None:
    try:
        with open("autosomal_count_prev.txt", "w") as f:
            f.write(str(autosomal_count))
        print("[OK] Persisted autosomal_count_prev.txt")
    except Exception as e:
        print("[WARN] Could not persist autosomal count:", e)

print("\n--- Ancestor Register Build + Exports Complete (HTML now at /partials/just-trees.htm; nav button -> justdna.htm) ---")
# ====== CUT STOP  [6/6] PERSIST COUNT + DONE ====================================================
#####
#####
#####
#####
#####


[OK] Loaded CSV: final_combined_df_with_value_labels.csv rows=7, cols=6
Using cached resolver: /content/match_to_unmasked.csv
[OK] Column B -> C mapping: 7 / 7 unmatched: 0
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Saved locally: just-trees.htm
[INFO] Attempting FTP upload...
[OK] Uploaded HTML to /partials/: just-trees.htm -> just-trees.htm
[OK] Uploaded exports to /partials/: yates_ancestor_register.csv yates_ancestor_register.xlsx
Open URL: https://yates.one-name.net/partials/just-trees.htm
[OK] Uploads complete.
[OK] Persisted autosomal_count_prev.txt

--- Ancestor Register Build + Exports Complete (HTML now at /partials/just-trees.htm; nav button -> justdna.htm) ---


# Test Cell 1

In [ ]:
# ====== CUT START [1/1] CELL 1 — GEDCOM -> final_combined_df_with_value_labels.csv + working HTML ======
# RON RULES — QUICK CODE CARD (v2025.11.05-Cell1-NoValueScoring)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - Punctuation in strings use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# - Python code (inline, executable, full COLAB Cell paste-ready section).
# - XHTML 1.0 Transitional; Times New Roman body; absolute links to /partials/.
# - Outputs uploaded to /partials/: CSV + HTML; autosomal_count.txt stays local (Cell 2 handles upload).

import os, re, glob, logging, functools, socket, posixpath, traceback
from datetime import datetime
from collections import defaultdict, Counter
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import pandas as pd
from tqdm import tqdm
from ftplib import FTP_TLS
from IPython.display import display, Javascript
from string import Template

# ====== LOGGING ======
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# ====== SECRETS + CONFIG + FTP HELPERS ======
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_HOST = os.environ.get('FTP_HOST','')
FTP_USER = os.environ.get('FTP_USER','')
FTP_PASS = os.environ.get('FTP_PASS','')
FTP_PORT = int(os.environ.get('FTP_PORT','21'))
FTP_DIR  = os.environ.get('FTP_DIR','').strip().strip('/')

def _ftps_connect():
    socket.setdefaulttimeout(30)
    ftps = FTP_TLS(timeout=30)
    ftps.connect(FTP_HOST, FTP_PORT)
    ftps.login(FTP_USER, FTP_PASS)
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(True)
    except Exception: pass
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split('/') if p]:
            try:
                ftps.cwd(p)
            except Exception:
                try: ftps.mkd(p)
                except Exception: pass
                ftps.cwd(p)
    return ftps

def _ftps_ensure_dir(ftps, path):
    if not path: return
    for p in [p for p in path.split('/') if p]:
        try:
            ftps.cwd(p)
        except Exception:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)

def _ftps_upload(ftps, local_path, remote_name):
    with open(local_path, 'rb') as fh:
        ftps.storbinary("STOR " + remote_name, fh)
    print(f"[OK] Uploaded: {local_path} -> {ftps.pwd().rstrip('/')}/{remote_name}")

# Output names (local + remote inside /partials/)
CSV_OUT_LOCAL   = "final_combined_df_with_value_labels.csv"  # kept for compatibility with Cell 2
HTML_OUT_LOCAL  = "cell1_work_table.htm"
REMOTE_DIR      = "partials"
REMOTE_CSV_NAME = os.path.basename(CSV_OUT_LOCAL)
REMOTE_HTML_NAME= os.path.basename(HTML_OUT_LOCAL)

# Absolute links used inside HTML
ABS_CSV_URL  = f"/{REMOTE_DIR}/{REMOTE_CSV_NAME}"
ABS_HOME_URL = "/partials/ons_yates_dna_register.htm"  # main register lives in /partials/

# ====== CORE STRUCTURES (dataset, GEDCOM parse) ======
anchor_gen1 = None
visited_pairs = set()
generation_table = []

class GedcomDataset:
    def __init__(self, gen_person):
        self.gen_person = gen_person
        self.extractable_detail = {}
        self.anchor_gen1 = None
    def add_extractable_detail(self, key, value):
        self.extractable_detail[key] = value
    def get_gen_person(self):
        name = self.extractable_detail.get('NAME', '')
        parts = name.split('/', 1)
        first_name = parts[0].split(' ')[0]
        last_name = parts[1].rstrip('/') if len(parts) > 1 else ""
        self.anchor_gen1 = last_name.replace(" ", "") + first_name.replace(" ", "")
        global anchor_gen1
        anchor_gen1 = self.anchor_gen1
        return self.gen_person.strip('@')
    def get_extractable_NPFX(self):
        return self.extractable_detail.get('NPFX', '')
    def get_extractable_cm(self):
        npfx_value = self.extractable_detail.get('NPFX','')
        if '&' in npfx_value:
            cm_value = npfx_value.split('&')[0].strip()
        elif '**' in npfx_value:
            cm_value = npfx_value.split('**')[0].strip()
        else:
            cm_value = npfx_value.strip()
        try:
            int(cm_value); return cm_value
        except ValueError:
            return ''
    def get_extractable_sort(self):
        npfx_value = self.extractable_detail.get('NPFX','')
        if '&' in npfx_value:
            sort_part = npfx_value.split('&')[1]
            sort_value = sort_part.split('**')[0].strip() if '**' in sort_part else sort_part.strip()
            return sort_value
        return ''
    def get_extractable_YDNA(self):
        npfx_value = self.extractable_detail.get('NPFX','')
        if '**' in npfx_value:
            return npfx_value.split('**')[1].strip()
        return ''
    def get_extractable_FAMC(self):
        return self.extractable_detail.get('FAMC','').strip('@')

class Gedcom:
    def __init__(self, file_name):
        self.file_name = file_name
        self.gedcom_datasets = []
        self.filter_pool = []
    def parse_gedcom(self):
        with open(self.file_name, 'r', encoding='utf-8-sig') as f:
            lines = f.readlines()
        current_dataset = None
        npfx_count = 0
        ydna_count = 0
        total_count = 0
        for line in lines:
            parts = line.strip().split(' ', 2)
            level = int(parts[0]); tag = parts[1]; value = parts[2] if len(parts) > 2 else None
            if level == 0 and tag.startswith('@') and tag.endswith('@') and value == 'INDI':
                total_count += 1
                current_dataset = GedcomDataset(tag)
                self.gedcom_datasets.append(current_dataset)
            elif current_dataset is not None:
                if level == 1 and tag in ['NAME','FAMC']:
                    current_dataset.add_extractable_detail(tag, value)
                elif level == 2 and tag == 'NPFX':
                    npfx_count += 1
                    current_dataset.add_extractable_detail(tag, value)
                    if '**' in value:
                        ydna_count += 1
        autosomal_count = npfx_count - ydna_count
        print(f"GEDCOM contained {total_count} total records")
        print(f"Records tagged and filtered by NPFX: {npfx_count}")
        print(f"Records with YDNA information: {ydna_count}")
        print(f"Autosomal matches: {autosomal_count}")
        for ds in self.gedcom_datasets:
            if ds.get_extractable_NPFX():
                self.filter_pool.append(ds)
        # Optional manual second-level filter
        try:
            df_filter = pd.read_excel('filtered_ids.xlsx')
            manual_ids = set(df_filter['ID'])
            self.filter_pool = [d for d in self.filter_pool if d.get_gen_person() in manual_ids]
            print(f"After manual filter, total records: {len(self.filter_pool)}")
            logger.info(f"After manual filter, total records: {len(self.filter_pool)}")
        except FileNotFoundError:
            logger.warning("filtered_ids.xlsx not found. Skipping second-level manual filter.")
        return autosomal_count

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

def quick_extract_name(full_text):
    name_marker = "\n1 NAME "
    idx = full_text.find(name_marker)
    if idx == -1:
        if full_text.startswith("1 NAME "):
            idx = 0
        else:
            return "UnknownName"
    start = idx + len(name_marker)
    end = full_text.find('\n', start)
    if end == -1: end = len(full_text)
    name_line = full_text[start:end].strip()
    if '/' not in name_line:
        # Return full line if no surname delimiter, removing spaces
        return name_line.replace(" ", "")
    first_name, last_name = name_line.split('/', 1)
    last_name = last_name.replace("/", "").strip()
    # Return full names, removing spaces (NO TRUNCATION)
    return last_name.replace(" ", "") + first_name.replace(" ", "")

def find_parents(individual_id, generation, parents_map):
    global visited_pairs, generation_table
    if individual_id not in parents_map: return
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id: return
    pair = (father_id, mother_id)
    if pair not in visited_pairs:
        visited_pairs.add(pair)
        generation_table.append((generation, pair))
    if father_id: find_parents(father_id, generation+1, parents_map)
    if mother_id: find_parents(mother_id, generation+1, parents_map)

def find_distant_ancestors(individual_id, parents_map, path=None):
    if path is None: path = []
    path.append(individual_id)
    if individual_id not in parents_map:
        return [path]
    father_id, mother_id = parents_map[individual_id]
    if not father_id and not mother_id:
        return [path]
    paths = []
    if father_id: paths.extend(find_distant_ancestors(father_id, parents_map, path[:]))
    if mother_id: paths.extend(find_distant_ancestors(mother_id, parents_map, path[:]))
    return paths if paths else [path]

def filter_ancestral_line(winning_path_ids, generation_table_local, names_map):
    matching_table = []
    for generation, pair in generation_table_local:
        id1, id2 = pair
        if id1 in winning_path_ids or id2 in winning_path_ids:
            matching_table.append((generation, pair))
    matching_table.sort(key=lambda x: x[0])
    lines = []
    for gen, pair in matching_table:
        name_pair = [names_map.get(pid, "UnknownName") for pid in pair]
        lines.append(f"{name_pair[0]}&{name_pair[1]}")
    lines.reverse()
    return "~~~".join(lines)

def process_record_wrapper(individual_id, gedcom_instance, parents_map, names_map):
    global generation_table, visited_pairs, anchor_gen1
    generation_table = []; visited_pairs = set()
    find_parents(individual_id, 1, parents_map)
    distant_paths = find_distant_ancestors(individual_id, parents_map)
    best_score, best_path = None, None
    for path in distant_paths:
        name_path = [names_map.get(pid, "UnknownName") for pid in path]
        score = sum((idx+1) for idx, nm in enumerate(name_path) if 'Yates' in nm)
        if best_score is None or score > best_score:
            best_score, best_path = score, path
    best_path = best_path or []
    best_path_cleaned = [pid for pid in best_path if pid != individual_id]
    line_str = filter_ancestral_line(set(best_path_cleaned), generation_table, names_map)
    cm_value = ''; sort_value=''; ydna_value=''
    for ds in gedcom_instance.filter_pool:
        if ds.get_gen_person() == individual_id:
            cm_value = ds.get_extractable_cm()
            sort_value = ds.get_extractable_sort()
            ydna_value = ds.get_extractable_YDNA()
            break
    short_name = names_map.get(individual_id, "UnknownName")
    # short_name populates the "Name" column
    # sort_value populates the "Match to" column
    return [individual_id, sort_value, short_name, cm_value, line_str, ydna_value]

# ====== MAIN: BUILD DATAFRAME (No Value Scoring) ======
def main():
    files = glob.glob("*.ged")
    if not files:
        print("No GEDCOM files found."); return False
    print("Automatically selecting the first GEDCOM file.")
    gedcom_file_path = files[0]

    ged = Gedcom(gedcom_file_path)
    autosomal_count = ged.parse_gedcom()

    with open("autosomal_count.txt", "w") as f:
        f.write(str(autosomal_count))

    with open(gedcom_file_path, 'r', encoding='utf-8') as f:
        raw_data = f.read()

    blocks = raw_data.split('\n0 ')
    all_records = {}
    for blk in blocks:
        blk = blk.strip()
        if not blk: continue
        flend = blk.find('\n'); flend = len(blk) if flend == -1 else flend
        first_line = blk[:flend]
        if '@' in first_line:
            start = first_line.find('@') + 1
            end = first_line.find('@', start)
            rec_id = first_line[start:end].strip()
            all_records[rec_id] = blk

    parents_map, names_map, families = {}, {}, {}
    for rec_id, txt in all_records.items():
        if 'FAM' in txt[:50]:
            father_idx = txt.find('1 HUSB @')
            husb_id = txt[father_idx+len('1 HUSB @'):txt.find('@', father_idx+len('1 HUSB @'))] if father_idx != -1 else None
            wife_idx = txt.find('1 WIFE @')
            wife_id = txt[wife_idx+len('1 WIFE @'):txt.find('@', wife_idx+len('1 WIFE @'))] if wife_idx != -1 else None
            kids = [ln.split('@')[1] for ln in txt.split('\n') if ln.strip().startswith('1 CHIL @')]
            families[rec_id] = (husb_id, wife_id, kids)
    for fam_id, (f_id, m_id, k_list) in families.items():
        for kid in k_list:
            parents_map[kid] = (f_id, m_id)
    for rec_id, txt in all_records.items():
        names_map[rec_id] = quick_extract_name("\n" + txt)

    individual_ids = [d.get_gen_person() for d in ged.filter_pool]
    print(f"Processing {len(individual_ids)} individuals with chunk-based parallel...")

    combined_rows = []
    chunk_size = 50
    max_workers = os.cpu_count() or 4
    with ProcessPoolExecutor(max_workers=max_workers) as executor, tqdm(total=len(individual_ids), desc="Building Yates Lines (Stage 1)") as pbar:
        for chunk in chunks(individual_ids, chunk_size):
            func = functools.partial(process_record_wrapper, gedcom_instance=ged, parents_map=parents_map, names_map=names_map)
            results = list(executor.map(func, chunk))
            combined_rows.extend(results)
            pbar.update(len(chunk))

    columns = ["ID#", "Match to", "Name", "cM", "Yates DNA Ancestral Line", "haplogroup"]
    df = pd.DataFrame(combined_rows, columns=columns)
    df.index += 1

    # Trim the historical prefix if present
    def remove_specific_prefix(row):
        prefix = "YatesJohn&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesWilliam&SearchingStill~~~YatesEdmund&CornellMargaret~~~YatesRichard&AshendonJoan~~~YatesJohn&HydeAlice~~~YatesThomas&FauconerElizabeth~~~"
        if str(row["Yates DNA Ancestral Line"]).startswith(prefix):
            row["Yates DNA Ancestral Line"] = row["Yates DNA Ancestral Line"][len(prefix):]
        return row
    df = df.apply(remove_specific_prefix, axis=1)

    # No Value/Range/Label computation; sort only by lineage for stability
    df.sort_values(by=["Yates DNA Ancestral Line"], inplace=True)

    # Export CSV (UTF-8-SIG for downstream safety; filename kept for Cell 2 compatibility)
    df.to_csv(CSV_OUT_LOCAL, index=False, encoding="utf-8-sig")
    logger.info("Exported CSV -> %s", CSV_OUT_LOCAL)

    # Simple working HTML (no Value columns)
    # MODIFIED: Changed "Match to" to "Name" to show the full name instead of the sort key
    final_cols = ["ID#", "cM", "haplogroup", "Name", "Yates DNA Ancestral Line"]
    table_html = df.to_html(index=False, columns=final_cols, escape=False, border=1)

    page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Cell 1 Working Table</title>
<style type="text/css">
  html { scroll-behavior: smooth; }
  /* MODIFIED: Removed explicit Times New Roman font-family to allow site-wide sans-serif CSS */
  body { background:#ffffff; color:#222; margin:0; padding:20px; }
  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }
  .meta { text-align:center; font-size:12px; color:#555; margin:2px 0 12px 0; }
  .downloads { text-align:center; margin:4px 0 12px 0; font-size:13px; }
  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }
  table { width:100%; border-collapse:collapse; }
  th, td { border:1px solid #333; padding: 6px 8px; vertical-align:top; }
  th { background:#e3eaf8; text-align:left; }
  td:nth-child(5) { text-align:left; white-space:normal; }
</style>
<script type="text/javascript">
//<![CDATA[
(function(){ function z(n){return (n<10?'0':'')+n;}
document.addEventListener('DOMContentLoaded', function(){
  var el = document.getElementById('last-updated');
  if(el){ var d=new Date(document.lastModified||new Date());
    el.innerHTML = d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes()); }
}, false); })();
//]]>
</script>
</head>
<body>
  <h1>Cell 1 Working Table</h1>
  <div class="meta">
    <a href="$HOME" target="_blank" rel="noopener">Home</a>
    &nbsp;|&nbsp; Last updated: <span id="last-updated"></span>
    &nbsp;|&nbsp; Download: <a href="$CSV">$CSV</a>
  </div>
  <div class="downloads"><a href="$CSV">/partials/$CSV_NAME</a></div>
  $TABLE
</body>
</html>""")

    page = page_tpl.safe_substitute(
        HOME=ABS_HOME_URL,
        CSV=ABS_CSV_URL,
        CSV_NAME=os.path.basename(ABS_CSV_URL),
        TABLE=table_html
    )

    with open(HTML_OUT_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(page)
    logger.info("Exported HTML -> %s", HTML_OUT_LOCAL)

    return True

ok = main()

# ====== UPLOAD ALL ARTIFACTS TO /partials/ ======
if ok and all([FTP_HOST, FTP_USER, FTP_PASS]):
    print("[INFO] Uploading artifacts to /partials/ ...")
    try:
        ftps = _ftps_connect()
        _ftps_ensure_dir(ftps, "partials")
        try:
            _ftps_upload(ftps, CSV_OUT_LOCAL, os.path.basename(CSV_OUT_LOCAL))
        except Exception as e:
            print("[ERROR] CSV upload failed:", e)
        try:
            _ftps_upload(ftps, HTML_OUT_LOCAL, os.path.basename(HTML_OUT_LOCAL))
        except Exception as e:
            print("[ERROR] HTML upload failed:", e)
        try: ftps.quit()
        except Exception: pass
        print("[OK] Uploads complete to /partials/")
    except Exception as e:
        print("[ERROR] FTP session failed:", e)
        traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload (missing creds or build failed).")

# ====== DONE ======
try:
    display(Javascript('alert("\\u2705 Cell 1 build complete (no value scoring): CSV + HTML uploaded to /partials/");'))
except Exception:
    pass
print("\n--- Cell 1 Complete (no value scoring): artifacts at /partials/ ---")
# ====== CUT STOP [1/1] CELL 1 ===================================================================

Automatically selecting the first GEDCOM file.
GEDCOM contained 62312 total records
Records tagged and filtered by NPFX: 1572
Records with YDNA information: 0
Autosomal matches: 1572
After manual filter, total records: 7
Processing 7 individuals with chunk-based parallel...


Building Yates Lines (Stage 1): 100%|██████████| 7/7 [00:03<00:00,  2.31it/s]


[INFO] Uploading artifacts to /partials/ ...
[OK] Uploaded: final_combined_df_with_value_labels.csv -> /partials/final_combined_df_with_value_labels.csv
[OK] Uploaded: cell1_work_table.htm -> /partials/cell1_work_table.htm
[OK] Uploads complete to /partials/


<IPython.core.display.Javascript object>


--- Cell 1 Complete (no value scoring): artifacts at /partials/ ---


In [ ]:
# ====== CUT START [1/1] CELL 2 — Build + Publish DNA Register (Responsive widths; external CSS handles typography) ======
# RON RULES — QUICK CODE CARD (v2025.11.08-Cell2-Responsive-FIXED-NAMES-ASCII)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - XHTML 1.0 Transitional; explicit prints; no fabrication.
# - Typography is controlled ONLY by /partials/dna_tree_styles.css (no inline font-family anywhere).
# - Publishes SAME HTML to:
#       /partials/yates_ancestor_register.htm     (canonical)
#       /partials/ons_yates_dna_register.htm     (legacy clone)
#       /partials/justdna.htm                     (JUSTDNA alias)

import os, re, posixpath, socket, traceback, urllib.parse as _u
from ftplib import FTP_TLS
import pandas as pd
import html as _html
from string import Template

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR'] = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR', '')
    os.environ.setdefault('FTP_PORT', '21')

# ---------- 1) Config ----------
CSV_IN = "final_combined_df_with_value_labels.csv"

LOCAL_HTML = "yates_ancestor_register.htm"
REMOTE_HTML_CANON = posixpath.join("partials", "yates_ancestor_register.htm")
REMOTE_HTML_LEG = posixpath.join("partials", "ons_yates_dna_register.htm")
REMOTE_HTML_SIMPLE = posixpath.join("partials", "justdna.htm")

DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"
TREES_ABS = "https://yates.one-name.net/partials/just-trees.htm"

EXPORT_BASENAME = "yates_ancestor_register"
LOCAL_CSV = f"{EXPORT_BASENAME}.csv"
LOCAL_XLSX = f"{EXPORT_BASENAME}.xlsx"
REMOTE_CSV = posixpath.join("partials", os.path.basename(LOCAL_CSV))
REMOTE_XLSX = posixpath.join("partials", os.path.basename(LOCAL_XLSX))

WORK_PLUS_LOCAL = os.path.join("partials", "work_plus.htm")
WORK_PLUS_REMOTE = posixpath.join("partials", "work_plus.htm")

LOCAL_COUNT_FILE = "/content/autosomal_count.txt"
REMOTE_COUNT_NAME = "autosomal_count.txt"

FTP_DIR = (os.environ.get("FTP_DIR","") or "").strip()
TNG_BASE = "https://yates.one-name.net/tng"
TNG_TREE = "tree1"
HOME_URL = "https://yates.one-name.net/partials/yates_ancestor_register.htm"
REMOTE_NAME_ABS = HOME_URL
COUNT_PUBLIC_URL = (f"/{FTP_DIR}/{REMOTE_COUNT_NAME}" if FTP_DIR else f"/{REMOTE_COUNT_NAME}")

# === Your requested widths (now used responsively) ===
TABLE_WIDTH_PX       = 2550
COL_A_PX             = 775
FIND_COL_PX          = 75
ARROW_ENTITY         = "&rarr;"
REMOVE_PERIOD_AT_END = True

SERVER_PARTIALS_DIR = "partials"
SERVER_MAPPING_BASENAME = "match_to_unmasked.csv"
SERVER_MAPPING_REMOTE = posixpath.join(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)
SERVER_MAPPING_LOCAL_CACHE = "match_to_unmasked.server.csv"

# External stylesheet (controls typography site-wide)
STYLESHEET_HREF = "/partials/dna_tree_styles.css"
CSS_VERSION = "v2025-11-06"
HEAD_LINK = f'<link rel="stylesheet" type="text/css" href="{STYLESHEET_HREF}?{CSS_VERSION}" />'

# ---------- 2) FTP helpers ----------
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))
FTP_PASSIVE = True

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        parts = [p for p in FTP_DIR.split("/") if p]
        for p in parts:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ensure_remote_dirs(ftps: FTP_TLS, remote_path: str):
    if "/" not in remote_path: return
    pwd0 = ftps.pwd()
    for seg in [p for p in remote_path.split("/")[:-1] if p and p != "."]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)
    ftps.cwd(pwd0)

def ftp_download_if_exists(ftps: FTP_TLS, remote_name: str, local_name: str) -> bool:
    try:
        with open(local_name, "wb") as f:
            ftps.retrbinary(f"RETR {remote_name}", f.write)
        print(f"[PULL] {remote_name} -> {os.path.abspath(local_name)}")
        return True
    except Exception as e:
        try:
            if os.path.exists(local_name): os.remove(local_name)
        except Exception: pass
        print(f"[MISS] {remote_name} ({e})")
        return False

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str):
    ensure_remote_dirs(ftps, remote_name)
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote_name}", fh)
    print(f"[PUT] {local_path} -> {remote_name}")

def ftp_size(ftps: FTP_TLS, remote_name: str):
    try:
        sz = ftps.size(remote_name)
        return int(sz) if sz is not None else None
    except Exception:
        return None

# ---------- 3) Resolver ----------
def _read_mapping_csv(path: str) -> pd.DataFrame:
    encs = ("iso-8859-15","utf-8-sig","utf-8","cp1252","latin1")
    last = None; df = None
    for enc in encs:
        try:
            df = pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False)
            break
        except Exception as e:
            last = e
    if df is None:
        raise RuntimeError(f"Unable to read mapping CSV {path}: {last}")
    if df.shape[1] < 2:
        raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")
    df = df.iloc[:, :2].copy()
    df.columns = ["code","unmasked"]
    df["code"] = df["code"].astype(str).str.strip().str.lower()
    df["unmasked"] = df["unmasked"].astype(str).str.strip()
    df = df[df["code"] != ""].drop_duplicates(subset=["code"], keep="first")
    if df.empty:
        raise RuntimeError("Mapping CSV empty after normalization.")
    return df

def load_resolver_from_server() -> dict:
    with ftp_connect() as ftps:
        try:
            ftps.cwd(SERVER_PARTIALS_DIR)
        except Exception:
            pass
        ok = ftp_download_if_exists(ftps, SERVER_MAPPING_BASENAME, SERVER_MAPPING_LOCAL_CACHE)
        try: ftps.quit()
        except Exception: pass
    if not ok:
        raise RuntimeError(
            "Resolver not found on server: /" + _remote_path(SERVER_MAPPING_REMOTE) +
            ". Upload match_to_unmasked.csv into /partials/ and re-run."
        )
    df_map = _read_mapping_csv(SERVER_MAPPING_LOCAL_CACHE)
    print(f"[OK] Resolver loaded: {len(df_map)} codes")
    return dict(zip(df_map["code"], df_map["unmasked"]))

MATCH_TO_UNMASKED = {}
def _setup_resolver():
    global MATCH_TO_UNMASKED
    if not MATCH_TO_UNMASKED:
        MATCH_TO_UNMASKED = load_resolver_from_server()

def resolve_match_to(code: str) -> str:
    if not isinstance(code, str): return ""
    return MATCH_TO_UNMASKED.get(code.strip().lower(), code)

# ---------- 4) Name & text utils ----------
SEP_RE = re.compile(r"\s*(?:\u2192|&rarr;|;|>|,|~{2,}|/{2,}|\|{2,})\s*")
def split_tokens(s):
    if pd.isna(s): return []
    if not isinstance(s, str): s = str(s)
    return [p.strip() for p in SEP_RE.split(s) if str(p).strip()]

def _clean_piece(text: str) -> str:
    t = re.sub(r'~+', ' ', str(text)); t = re.sub(r'\s+', ' ', t)
    return t.strip()

_PARTICLES = {"de","del","della","der","van","von","da","dos","das","di","la","le","du","of"}

def _smart_title(token: str) -> str:
    if not token: return token
    token = re.sub(r"(^|\b)([a-z])(['’])([a-z])", lambda m: m.group(1)+m.group(2).upper()+m.group(3)+m.group(4).upper(), token.lower())
    token = "-".join([w.capitalize() for w in token.split("-")])
    token = re.sub(r"\bmc([a-z])", lambda m: "Mc"+m.group(1).upper(), token)
    token = re.sub(r"\bmac([a-z])", lambda m: "Mac"+m.group(1).upper(), token)
    return token

def smart_titlecase(name: str) -> str:
    name = _clean_piece(name)
    if not name: return name
    if "," in name:
        last, first = [p.strip() for p in name.split(",", 1)]
        pieces = (first + " " + last).split()
    else:
        pieces = name.split()
    out = []
    for i, w in enumerate(pieces):
        out.append(w.lower() if (i>0 and w.lower() in _PARTICLES) else _smart_title(w))
    return " ".join(out)

def surname_given_from_token(token):
    token = token.strip()
    idx = None
    for i in range(1, len(token)):
        if token[i-1].islower() and token[i].isupper():
            idx = i; break
    if idx is None:
        for i in range(1, len(token)):
            if token[i].isupper():
                idx = i; break
    if idx is None: return (token,)
    surname = token[:idx]; given = token[idx:]
    given_spaced = re.sub(r'(?<!^)([A-Z])', r' \1', given)
    return (f"{given_spaced.strip()} {surname.strip()}",)

def normalize_person_name(s: str) -> str:
    if pd.isna(s): return ""
    s = _clean_piece(str(s))
    if "," in s:
        last, first = [p.strip() for p in s.split(",", 1)]
        s = f"{first} {last}"
    if " " not in s and s.isalpha():
        return smart_titlecase(surname_given_from_token(s)[0])
    return smart_titlecase(s)

_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")
def norm_matchee_name(raw: str) -> str:
    raw = str(raw or "").strip()
    if not raw: return ""
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1: return nm
        given = parts[0]; surname = parts[-1]
        return f"{given} {surname}".strip()
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1: return nm
        return f"{ps[0]} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates: return surname
    given = smart_titlecase(given_candidates[0])
    return f"{given} {surname}".strip()

def truncate_first(name: str, n: int = 7) -> str:
    name = name.strip()
    if not name: return name
    parts = name.split()
    return parts[0][:n] if len(parts) == 1 else f"{parts[0][:n]} {parts[-1]}"

def derive_common_from_first_token(tokens):
    if not tokens: return ("","")
    first = _clean_piece(tokens[0])
    parts = re.split(r"\s*(?:&| and )\s*", first, maxsplit=1, flags=re.I)
    if len(parts) != 2: return ("","")
    def _norm(s):
        return smart_titlecase(s) if " " in s else smart_titlecase(surname_given_from_token(s)[0])
    return (_norm(parts[0]), _norm(parts[1]))

def degree_label_from_generations(g):
    if g <= 1: return ("parents" if g == 1 else "self")
    if g == 2: return "grandparents"
    greats = g - 2
    if greats == 1: return "great-grandparents"
    return f"{greats}x-great-grandparents"

def build_header(subject_name, cm_val, matchee_name_html, gens, husband, wife):
    try:
        cm_str = f"{int(round(float(cm_val)))}"
    except Exception:
        cm_str = (str(cm_val).strip() or "0")
    degree_label = degree_label_from_generations(gens)
    parts = [
        f"{subject_name} is a {cm_str} cM cousin match to {matchee_name_html}, whose",
        f"{degree_label} (back {gens} Gens)",
        "are",
        f"{husband} & {wife}."
    ]
    s = " ".join(parts)
    if REMOVE_PERIOD_AT_END: s = re.sub(r'\.\s*$', '', s)
    return s

# ---------- 5) Read CSV; detect columns ----------
def find_col(df, patterns, prefer_exact=None):
    cols = list(df.columns)
    lowmap = {c.lower(): c for c in cols}
    if prefer_exact:
        for name in prefer_exact:
            if name in df.columns: return name
            if name and name.lower() in lowmap: return lowmap[name.lower()]
    for pat in patterns:
        rx = re.compile(pat, re.I)
        for c in cols:
            if rx.search(c): return c
    return None

_encs = ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None
if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"[OK] Loaded CSV: {len(df)} rows, {len(df.columns)} cols")

id_col = find_col(df, [r'^(id#|personid)$'], ["ID#","ID","PersonID","personID"])
match_col = find_col(df, [r'^match\s*to$'], ["Match to","Match"])
name_col = find_col(df, [r'^name$'], ["Name"])
cm_col = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM","cm"])
path_col = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'], ["Yates DNA Ancestral Line","Ancestral Line","Lineage"])

if not id_col: raise ValueError("CSV missing an ID#/PersonID column.")
if not match_col: raise ValueError("CSV missing 'Match to' column.")
if not name_col: raise ValueError("CSV missing 'Name' column.")
if not cm_col: raise ValueError("CSV missing 'cM' column.")
if not path_col: raise ValueError("CSV missing lineage/path column.")

# ---------- 6) Transform -> display_df ----------
ID_PAT = re.compile(r"\bI\d+\b", re.I)
def extract_person_id(s: str) -> str:
    m = ID_PAT.search(str(s or ""))
    return m.group(0).upper() if m else ""

# Resolver
def _setup_resolver_and_return():
    _setup_resolver()
    return MATCH_TO_UNMASKED
_ = _setup_resolver_and_return()

headers, lineages, findcol = [], [], []
subjects, first_ancestors = [], []

for _, row in df.iterrows():
    subject_raw = row.get(match_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b = f"<strong>{subject_name}</strong>" if subject_name else subject_name

    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name

    if pid:
        matchee_name_html = (
            f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0&display=vertical&generations=15" '
            f'target="_blank" rel="noopener">{matchee_name}</a>'
        )
    else:
        matchee_name_html = matchee_name

    cm_val = row.get(cm_col, "0")
    tokens = split_tokens(row.get(path_col, ""))
    gens_total = len(tokens)
    tokens_disp = tokens[:7]

    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw = str(row.get("common_husband","")).strip()
        wife_raw = str(row.get("common_wife","")).strip()
        if not husband_raw and not wife_raw:
            husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    header_html = build_header(
        subject_name_b,
        cm_val,
        matchee_name_html,
        gens_total,
        husband_raw,
        wife_raw
    )

    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"
    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    q = _u.quote(subject_name)
    quick = (
        f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
        f'title="Open a filtered view for {subject_name}">Find</a>'
    )

    headers.append(header_html)
    lineages.append(lineage_text)
    findcol.append(quick)
    subjects.append(subject_name)
    first_ancestors.append(tokens[0] if tokens else "")

LINEAGE_HEADER_SAFE = "Lineage (Starting with oldest ancestor, the line is:)"
df["Match Summary"] = headers
df[LINEAGE_HEADER_SAFE] = lineages
df["Find"] = findcol
df["Subject"] = subjects
df["First Ancestor"] = [_clean_piece(x) for x in first_ancestors]
display_df = df[["Find","Match Summary", LINEAGE_HEADER_SAFE]]

# ---------- 6.1) Clean exports ----------
TAG_RE = re.compile(r"<[^>]+>")
def _html_to_text(s: str) -> str:
    t = TAG_RE.sub("", str(s or ""))
    t = _html.unescape(t)
    t = t.replace("\u2192", "->")
    return re.sub(r"\s+", " ", t).strip()

def _extract_find_url(cell_html: str) -> str:
    m = re.search(r'href="([^"]+)"', str(cell_html or ""))
    return _html.unescape(m.group(1)) if m else ""

export_df = pd.DataFrame({
    "Find URL": [ _extract_find_url(v) for v in display_df["Find"].tolist() ],
    "Match Summary": [ _html_to_text(v) for v in display_df["Match Summary"].tolist() ],
    "Lineage": [ _html_to_text(v) for v in display_df[LINEAGE_HEADER_SAFE].tolist() ],
})
export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    with pd.ExcelWriter(LOCAL_XLSX) as _writer:
        export_df.to_excel(_writer, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))

# ---------- 7) HTML (Register main page) ----------
# Responsive CSS: table fills width; large screens honor px via colgroup; small screens auto/percent.
TABLE_CSS = f"""
<style type="text/css">
  html {{ scroll-behavior: smooth; }}
  body {{ font-size:100%; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }}
  .wrap {{ max-width:100%; margin:0 auto; background:#ffffff; padding:16px; padding-bottom:48px; }}
  a {{ color:#154b8b; text-decoration:none; }} a:hover {{ text-decoration:underline; }}
  h1 {{ margin:0 0 6px 0; font-size:26px; line-height:1.2; text-align:center; }}
  .centerline {{ text-align:center; }}
  .downloads {{ text-align:center; margin:4px 0 10px 0; font-size:13px; }}
  .updated {{ font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }}
  .sortbar {{ margin:6px 0 10px 0; font-size:13px; background:#ffffff; padding:6px 8px; border-radius:6px; display:flex; flex-wrap:wrap; gap:5px; align-items:center; border:1px solid #ddd; justify-content:center; }}
  .btn {{ display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px; text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; }}
  .btn:hover {{ background:#4668aa; }}
  input.btn.search {{ background:#fff; color:#111; border-color:#bbb; }}

  .table-scroll {{ max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }}
  table.sortable {{ border-collapse:collapse; width:100%; table-layout:fixed; }}
  table.sortable th, table.sortable td {{ border:1px solid #ddd; padding:6px 8px; vertical-align:top; word-wrap:break-word; overflow-wrap:break-word; }}
  table.sortable th {{ background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; cursor:pointer; }}
  #first-row td {{ border-top:2px solid #999; }}

  .find-cell {{ white-space:nowrap; }}
  .selbox {{ margin-right:6px; vertical-align:middle; }}

  .back-to-top {{ position:fixed; right:16px; bottom:16px; padding:6px 10px; border:1px solid #3e5a97; background:#5b79b8; color:#fff; cursor:pointer; border-radius:6px; font-size:12px; display:none; z-index:9999; }}
  .back-to-top:hover {{ background:#4668aa; }}

  /* Large screens: suggest px widths from your constants */
  @media screen and (min-width: 1200px) {{
    #refactor-table col:nth-child(1) {{ width:{FIND_COL_PX}px; }}
    #refactor-table col:nth-child(2) {{ width:{COL_A_PX}px; }}
    .wrap {{ max-width:{TABLE_WIDTH_PX}px; }}
  }}

  /* Medium screens: loosen widths to percentages */
  @media screen and (max-width: 1199px) {{
    #refactor-table {{ table-layout:auto; }}
    #refactor-table col:nth-child(1) {{ width:12%; }}
    #refactor-table col:nth-child(2) {{ width:44%; }}
    #refactor-table col:nth-child(3) {{ width:44%; }}
  }}

  /* Small screens: tighter padding and bigger first column tap targets */
  @media screen and (max-width: 700px) {{
    table.sortable th, table.sortable td {{ padding:5px 6px; }}
    #refactor-table col:nth-child(1) {{ width:16%; }}
    #refactor-table col:nth-child(2) {{ width:42%; }}
    #refactor-table col:nth-child(3) {{ width:42%; }}
    .btn {{ padding:4px 7px; }}
  }}
</style>
"""

DYNAMIC_BLOCK = (
    '<div class="sortbar">'
    '<a class="btn" href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm" target="_blank" rel="noopener">Study Details</a>'
    '<a class="btn" href="https://yates.one-name.net/gengen/dna_theory_of_the_case.htm" target="_blank" rel="noopener">Theory in Action</a>'
    '<a class="btn" href="/gengen/images/cousin-calculator.jpg" target="_blank" rel="noopener">Cousin Connection</a>'
    '<a class="btn" href="/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" rel="noopener">Cousin by DNA</a>'
    '<a class="btn" href="/partials/match_count.htm" target="_blank" rel="noopener">Match Count</a>'
    '<a class="btn" href="/partials/lineage_count.htm" target="_blank" rel="noopener">Lineage Count</a>'
    f'<a class="btn" href="{DNA_REGISTER_ABS}" target="_blank" rel="noopener">DNA Register</a>'
    f'<a class="btn" href="{TREES_ABS}" target="_blank" rel="noopener">Trees</a>'
    '<span class="btn" id="show-selected" title="Show all rows for the checked name(s)">Show Selected</span>'
    '<span class="btn" id="show-all" title="Show All">Show All</span>'
    '<span class="btn" id="print-cousin-list" style="cursor:pointer;" title="Open a printable list of the *currently visible* rows">Cousin List (Printable)</span>'
    '<span class="btn" id="clear-selected">Reset</span>'
    '<input type="text" id="search-box" class="btn search" size="24" value="" placeholder="Search&amp;hellip;" />'
    '</div>'
)

JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")
UPDATED_BLOCK = (
    '<div class="updated centerline">'
    'Last updated: <span id="last-updated"></span>'
    ' &nbsp;|&nbsp; Autosomal matches: <span id="auto-count" class="js-count"></span>'
    ' &nbsp;|&nbsp; Showing: <span id="showing-count"></span>'
    '</div>'
)

DOWNLOADS_BLOCK = (
    '<p class="downloads centerline">Download: '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_CSV))}">CSV</a> | '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_XLSX))}">Excel</a></p>'
)

page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
$HEAD_LINK
$TABLE_CSS
</head>
<body id="top">
<div class="wrap">
  <h1 class="centerline">ONS Yates Study Autosomal DNA Register</h1>
  $DOWNLOADS_BLOCK
  $UPDATED_BLOCK
  $DYNAMIC_BLOCK
  <div class="table-scroll">
    $HTML_TABLE
  </div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]); var asc=(dir==='asc');
    rows.sort(function(a,b){var A=textOf(a.cells[colIndex]),B=textOf(b.cells[colIndex]); if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;});
    var frag=document.createDocumentFragment(); for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]); tb.appendChild(frag);
    updateShowing();
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++)(function(idx){
      var th = ths[idx]; var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+\\(asc\\)|\\s+\\(desc\\)/,''); }
        th.innerHTML += (dir==='asc' ? ' (asc)' : ' (desc)');
        sortTable(tbl,idx,dir);
      },false);
    })(i);
  }
  function formatWithCommas(n){
    try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
  }
  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0; for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; } return n;
  }
  function updateShowing(){ var el=document.getElementById('showing-count'); if(!el) return; el.textContent = formatWithCommas(visibleRowCount()); }
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function rowText(tr){ var t=''; for(var i=1;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q = String(q||''); var parts = q.split('|').map(function(s){return norm(s);}).filter(function(s){return !!s;});
      for(var i=0;i<cached.length;i++){
        var hit = !parts.length || parts.some(function(p){ return cached[i].txt.indexOf(p)>-1; });
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false);
    box.addEventListener('search', onInput, false);
    var q0=getParam('q');
    if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null, '', location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }
  function allRowCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[];
    for(var i=0;i<tb.rows.length;i++){
      var cb=tb.rows[i].querySelector('.selbox');
      if(cb) out.push(cb);
    }
    return out;
  }
  function bindGroupSync(){
    var tbl=document.getElementById('refactor-table'); if(!tbl) return;
    tbl.addEventListener('click', function(e){
      var t=e.target||e.srcElement;
      if(!(t && t.classList && t.classList.contains('selbox'))) return;
      var nm = t.getAttribute('data-name') || '';
      var checked = !!t.checked;
      var cbs = allRowCheckboxes();
      for(var i=0;i<cbs.length;i++){
        if((cbs[i].getAttribute('data-name')||'') === nm){ cbs[i].checked = checked; }
      }
      updateShowing();
    }, false);
  }
  function bindShowSelected(){
    var btn=document.getElementById('show-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs = allRowCheckboxes();
      var names = {};
      for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ names[cbs[i].getAttribute('data-name')||''] = true; } }
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var r=0;r<tb.rows.length;r++){
        var cb = tb.rows[r].querySelector('.selbox');
        var nm = cb ? (cb.getAttribute('data-name')||'') : '';
        tb.rows[r].style.display = names[nm] ? '' : 'none';
      }
      updateShowing();
    }, false);
  }
  function bindShowAll(){
    var btn=document.getElementById('show-all'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var i=0;i<tb.rows.length;i++){ tb.rows[i].style.display=''; }
      updateShowing();
    }, false);
  }
  function bindClear(){
    var btn=document.getElementById('clear-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs=allRowCheckboxes(); for(var i=0;i<cbs.length;i++) cbs[i].checked=false;
      var tbl=document.getElementById('refactor-table'); if(tbl && tbl.tBodies && tbl.tBodies[0]){
        var tb=tbl.tBodies[0]; for(var j=0;j<tb.rows.length;j++){ tb.rows[j].style.display=''; }
      }
      updateShowing();
    }, false);
  }
  function addCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
    var tb=tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i]; var cell=tr.cells[0]; var findBtn=cell ? cell.querySelector('.find-btn') : null;
      var name = findBtn ? (findBtn.getAttribute('title')||'').replace('Open a filtered view for ','') : ('Row '+(i+1));
      if(cell){
        cell.classList.add('find-cell');
        cell.innerHTML = '<input type="checkbox" class="selbox" title="Select this row" data-name="'+name.replace(/"/g,'&quot;')+'" /> ' + cell.innerHTML.replace(/^\\s*/, '');
      }
    }
  }
  function initShowingStatic(){ try{ document.getElementById('showing-count').textContent = document.getElementById('refactor-table').tBodies[0].rows.length; }catch(e){}}
  document.addEventListener('DOMContentLoaded', function(){
    addCheckboxes();
    (function(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date()); function z(n){return (n<10?'0':'')+n;} el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());})();
    bindHeaderSort();
    bindSearch();
    bindGroupSync();
    bindShowSelected();
    bindShowAll();
    bindClear();
    initShowingStatic();
  });
})();
 //]]>
</script>
</body>
</html>
""")

html_table = display_df.to_html(index=False, escape=False, classes="dataframe sortable")

# Step 1: Replace default table tag with our ID and mark first row
html_table = html_table.replace('<table border="1" class="dataframe sortable">', '<table border="1" class="dataframe sortable" id="refactor-table">', 1)
html_table = html_table.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
html_table = html_table.replace("<th>Find</th>", '<th>Select:</th>', 1)

# Step 2: Rename headers (ASCII-safe)
html_table = html_table.replace('<th>Match Summary</th>', '<th>Match Summary&amp;ndash;click to sort</th>', 1)
html_table = html_table.replace(f'<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>', '<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>', 1)

# Step 3: Insert <colgroup> with your px widths (CSS overrides on small screens)
colgroup_html = (
    "<colgroup>\n"
    f"  <col style=\"width:{FIND_COL_PX}px;\" />\n"
    f"  <col style=\"width:{COL_A_PX}px;\" />\n"
    "  <col />\n"
    "</colgroup>\n"
)
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable" id="refactor-table">',
    '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html,
    1
)

# Build main page
final_html = page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK,
    TABLE_CSS=TABLE_CSS,
    UPDATED_BLOCK=UPDATED_BLOCK,
    DYNAMIC_BLOCK=DYNAMIC_BLOCK,
    HTML_TABLE=html_table,
    JS_COUNT_URL=JS_COUNT_URL,
    DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
).replace("$HEAD_LINK_URL_JS$", (STYLESHEET_HREF + "?" + CSS_VERSION))

# ---------- 8) Partials (keep simple; no font-family inline) ----------
def _norm_code_for_count(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = re.sub(r'\s+', ' ', t).strip()
    t = t.lower()
    return t

def _partial_css_wrapper_simple():
    return (
        "<style type=\"text/css\">\n"
        "  html { scroll-behavior: smooth; }\n"
        "  body { background:#ffffff; color:#222; margin:0; padding:0; }\n"
        "  .wrap { max-width:100%; margin:0 auto; padding:16px; }\n"
        "  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }\n"
        "  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }\n"
        "  .meta { text-align:center; font-size:12px; color:#555; margin:2px 0 10px 0; }\n"
        "  .toolbar { display:flex; gap:10px; align-items:center; margin:6px 0 10px 0; flex-wrap:wrap; justify-content:center; }\n"
        "  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px; text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; }\n"
        "  table { border-collapse:collapse; width:100%; table-layout:fixed; }\n"
        "  th, td { border:1px solid #ddd; padding:6px 8px; word-wrap:break-word; overflow-wrap:break-word; }\n"
        "  th { background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; }\n"
        "  tr.sel { background:#fff7d6; }\n"
        "  .count a { font-weight:bold; }\n"
        "  @media screen and (max-width: 700px) { th, td { padding:5px 6px; } }\n"
        "</style>\n"
    )

def _partial_head(title):
    return (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
        " \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n<head>\n"
        f"{HEAD_LINK}\n"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        f"<title>{_html.escape(title)}</title>\n"
    )

def _partial_toolbar():
    safe_home = HOME_URL.replace('"','&quot;')
    return (
        "<div class=\"toolbar\">"
        f"<a class=\"btn\" href=\"{safe_home}\" target=\"_blank\" rel=\"noopener\">DNA Register</a>"
        " <button id=\"mc-show-selected\" class=\"btn\" title=\"Open DNA Register filtered to selected\">Show Selected</button>"
        " <button id=\"mc-show-all\" class=\"btn\" title=\"Show all rows (this table)\">Show All</button>"
        " <button id=\"mc-reset\" class=\"btn\" title=\"Clear selection and show all\">Reset</button>"
        " <button id=\"view\" class=\"btn\" title=\"Open DNA Register with selected (alias)\">View Now</button>"
        "</div>\n"
    )

def _partial_js_common():
    _safe_home = HOME_URL.replace("'", "%27")
    return (
        "<script type=\"text/javascript\">\n//<![CDATA[\n"
        "(function(){\n"
        "  var REG = '" + _safe_home + "';\n"
        "  function fmt(n){ try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return '0'; return x.toLocaleString('en-US'); }catch(e){ return String(n||'0'); } }\n"
        "  function selected(){ var out=[]; var tb=document.getElementById('ref-tb'); if(!tb) return out; var rows=tb.rows; for(var i=0;i<rows.length;i++){ if((' '+rows[i].className+' ').indexOf(' sel ')>-1) out.push(rows[i]); } return out; }\n"
        "  function update(){ var sel=selected(), sum=0; for(var i=0;i<sel.length;i++){ var v=parseInt((sel[i].getAttribute('data-count')||'0').replace(/[^0-9\\-]/g,''),10); if(!isNaN(v)) sum+=v; } var nEl=document.getElementById('sel-n'); var sEl=document.getElementById('sel-sum'); if(nEl) nEl.innerHTML=fmt(sel.length); if(sEl) sEl.innerHTML=fmt(sum); }\n"
        "  function qJoin(parts){ var out=[]; var seen={}; for(var i=0;i<parts.length;i++){ var p=String(parts[i]||''); if(p && !seen[p]){ seen[p]=1; out.push(encodeURIComponent(p)); } } return out.join('%7C'); }\n"
        "  function openRegisterForSelected(){ var sel=selected(); if(!sel.length) return; var qs=[]; for(var i=0;i<sel.length;i++){ qs.push(sel[i].getAttribute('data-q')||''); } var q = qJoin(qs); var url = REG + '?q=' + q; var w=null; try{ w=window.open(url,'RegisterFiltered'); if(!w) throw new Error('popup'); w.focus(); } catch(e){ window.location.href = url; } }\n"
        "  function toggleFrom(el){ var tr=el; while(tr && tr.nodeName && tr.nodeName.toLowerCase()!=='tr'){ tr=tr.parentNode; } if(!tr) return; var c=tr.className||''; tr.className = ((' '+c+' ').indexOf(' sel ')>-1) ? c.replace(/\\bsel\\b/,'').replace(/\\s{2,}/g,' ').trim() : (c?c+' ':'')+'sel'; update(); }\n"
        "  document.addEventListener('click', function(e){ var t=e.target||e.srcElement; if(!t) return; if(t.classList && t.classList.contains('count-pick')){ e.preventDefault(); toggleFrom(t); return; } if(t.id=='view' || t.id=='mc-show-selected'){ e.preventDefault(); openRegisterForSelected(); return; } if(t.id=='mc-reset'){ e.preventDefault(); var tb=document.getElementById('ref-tb'); if(tb){ var rows=tb.rows; for(var i=0;i<rows.length;i++){ rows[i].className = rows[i].className.replace(/\\bsel\\b/,'').replace(/\\s{2,}/g,' ').trim(); rows[i].style.display=''; } } update(); return; } if(t.id=='mc-show-all'){ e.preventDefault(); var tb2=document.getElementById('ref-tb'); if(!tb2) return; for(var k=0;k<tb2.rows.length;k++){ tb2.rows[k].style.display=''; } return; } }, false);\n"
        "  document.addEventListener('DOMContentLoaded', update, false);\n"
        "})();\n"
        "//]]>\n</script>\n"
    )

def _shell_open(title):
    return (
        _partial_head(title) +
        _partial_css_wrapper_simple() +
        "</head>\n<body>\n<div class=\"wrap\">\n" +
        f"<h1>{_html.escape(title)}</h1>\n" +
        "<div class=\"meta\">"
        "Last updated: <span id=\"last-updated\"></span>"
        " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\">&mdash;</span>"
        " &nbsp;|&nbsp; Selected: <span id=\"sel-n\">0</span> &nbsp; Sum: <span id=\"sel-sum\">0</span>"
        "</div>\n" +
        _partial_toolbar() +
        "<div class=\"table-scroll\">\n"
    )

def _shell_close():
    safe_count = COUNT_PUBLIC_URL.replace("'", "%27")
    return (
        "</div>\n</div>\n"
        "<script type=\"text/javascript\">\n//<![CDATA[\n"
        "(function(){ function z(n){return (n<10?'0':'')+n;} function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date()); el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}\n"
        "function load(){var el=document.getElementById('auto-count'); if(!el) return; var URL='" + safe_count + "'; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true); xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent=(m?m[1]:'');} else {el.textContent='(unavailable)';}}}; xhr.send(null);}catch(e){el.textContent='(unavailable)';}}\n"
        "document.addEventListener('DOMContentLoaded', function(){ stamp(); load(); }, false); })();\n"
        "//]]>\n</script>\n"
        + _partial_js_common() +
        "</body>\n</html>"
    )

def build_match_count_partial(main_df: pd.DataFrame) -> str:
    codes_raw = main_df[match_col].astype(str).map(lambda x: x.strip())
    keys_norm = codes_raw.map(_norm_code_for_count)
    counts_series = keys_norm.value_counts(dropna=False)
    counts = counts_series.reset_index()
    if counts.shape[1] >= 2:
        counts.columns = ["norm_key", "Count"]
    else:
        counts["norm_key"] = counts.index.astype(str)
        counts["Count"] = counts_series.values
        counts = counts[["norm_key","Count"]]
    first_display = {}
    for code_disp, k in zip(codes_raw.tolist(), keys_norm.tolist()):
        if k not in first_display and str(k) != "":
            first_display[k] = code_disp
    counts["Code"] = counts["norm_key"].map(lambda k: first_display.get(k, k))
    counts["Unmasked"] = counts["norm_key"].map(lambda k: MATCH_TO_UNMASKED.get(k, ""))
    counts = counts.sort_values(by=["Code","Count"], ascending=[True, False], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_shell_open("Match Count"))
    html.append('<table id="ref-table" border="1" class="sortable" style="width:100%"><thead><tr>')
    html.append('<th style="width:35%">Code</th><th style="width:45%">Unmasked</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in counts.iterrows():
        code = r.get("Code","")
        unm  = r.get("Unmasked","")
        cnt  = int(str(r.get("Count","0")).strip() or "0")
        label = (unm or code).strip()
        tr = (
            f'<tr data-q="{_html.escape(label, quote=True)}" data-count="{cnt}">'
            f'<td>{_html.escape(code)}</td>'
            f'<td>{_html.escape(unm)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td>'
            f'</tr>'
        )
        html.append(tr)
    html.append('</tbody></table>')
    html.append(_shell_close())
    return "".join(html)

def build_lineage_count_partial(main_df: pd.DataFrame) -> str:
    first_series = main_df.get("First Ancestor", pd.Series(dtype=str)).astype(str).map(lambda x: x.strip())
    vc = first_series[first_series != ""].value_counts(dropna=False)
    lin_df = vc.reset_index()
    if lin_df.shape[1] >= 2:
        lin_df.columns = ["First Ancestor","Count"]
    else:
        lin_df["First Ancestor"] = lin_df.index.astype(str)
        lin_df["Count"] = vc.values
        lin_df = lin_df[["First Ancestor","Count"]]
    lin_df = lin_df.sort_values(["Count","First Ancestor"], ascending=[False, True], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_shell_open("Lineage Count"))
    html.append('<table id="ref-table" border="1" class="sortable" style="width:100%"><thead><tr>')
    html.append('<th style="width:80%">First Ancestor</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in lin_df.iterrows():
        first = str(r.get("First Ancestor","")).strip()
        cnt   = int(str(r.get("Count","0")).strip() or "0")
        tr = (
            f'<tr data-q="{_html.escape(first, quote=True)}" data-count="{cnt}">'
            f'<td>{_html.escape(first)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td>'
            f'</tr>'
        )
        html.append(tr)
    html.append('</tbody></table>')
    html.append(_shell_close())
    return "".join(html)

def build_and_write_partials(main_df: pd.DataFrame):
    _setup_resolver()
    os.makedirs("partials", exist_ok=True)

    mc_html = build_match_count_partial(main_df)
    mc_local = os.path.join("partials", "match_count.htm")
    with open(mc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(mc_html)
    print("[OK] Wrote partial:", mc_local)

    lc_html = build_lineage_count_partial(main_df)
    lc_local = os.path.join("partials", "lineage_count.htm")
    with open(lc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(lc_html)
    print("[OK] Wrote partial:", lc_local)

    # Printable cousin list
    cousin_df = main_df[["Match Summary"]].copy()
    cousin_df = cousin_df.sort_values(by="Match Summary", ascending=True, kind="mergesort").reset_index(drop=True)
    cousin_rows = ['<table border="1" id="refactor-table"><thead><tr><th>Match Summary</th></tr></thead><tbody>']
    for v in cousin_df["Match Summary"].tolist():
        cousin_rows.append(f"<tr><td>{v}</td></tr>")
    cousin_rows.append("</tbody></table>")
    cousin_html = (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\"><head>"
        f"{HEAD_LINK}"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />"
        "<title>Cousin List (Printable)</title>"
        "<style type=\"text/css\"> body{font-size:12px;margin:20px;} h1{text-align:center;font-size:20px;} table{border-collapse:collapse;width:100%;} th,td{border:1px solid #999;padding:5px 7px;vertical-align:top;text-align:left;} th{background:#f0f0f0;} a{color:#000;text-decoration:none;} </style>"
        "</head><body onload=\"window.print();\">"
        "<h1>Cousin List (Printable)</h1>" + "".join(cousin_rows) +
        "</body></html>"
    )
    cl_local = os.path.join("partials", "cousin_list_print.htm")
    with open(cl_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(cousin_html)
    print("[OK] Wrote partial:", cl_local)

    return mc_local, lc_local, cl_local

# Build partials + main page
PARTIAL_MATCH_LOCAL, PARTIAL_LINEAGE_LOCAL, PARTIAL_COUSIN_LOCAL = build_and_write_partials(df)

def build_register_html_for_abs(remote_abs_path: str) -> str:
    q_links = []
    subs = df["Subject"].astype(str).tolist()
    for subject_name in subs:
        q = _u.quote(subject_name)
        q_links.append(
            f'<a class="find-btn" href="{remote_abs_path}?q={q}" target="_blank" rel="noopener" '
            f'title="Open a filtered view for {subject_name}">Find</a>'
        )
    df_plus = df.copy()
    df_plus["Find"] = q_links
    disp_plus = df_plus[["Find","Match Summary", LINEAGE_HEADER_SAFE]]
    tbl = disp_plus.to_html(index=False, escape=False, classes="dataframe sortable")
    tbl = tbl.replace('<table border="1" class="dataframe sortable">','<table border="1" class="dataframe sortable" id="refactor-table">',1)
    tbl = tbl.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
    tbl = tbl.replace("<th>Find</th>", '<th>Select:</th>', 1)
    tbl = tbl.replace("<th>Match Summary</th>", '<th>Match Summary&amp;ndash;click to sort</th>', 1)
    tbl = tbl.replace(f"<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>", "<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>", 1)

    colgroup_html_local = (
        "<colgroup>\n"
        f"  <col style=\"width:{FIND_COL_PX}px;\" />\n"
        f"  <col style=\"width:{COL_A_PX}px;\" />\n"
        "  <col />\n"
        "</colgroup>\n"
    )
    tbl = tbl.replace(
        '<table border="1" class="dataframe sortable" id="refactor-table">',
        '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html_local, 1
    )
    return page_tpl.safe_substitute(
        HEAD_LINK=HEAD_LINK,
        TABLE_CSS=TABLE_CSS,
        UPDATED_BLOCK=UPDATED_BLOCK,
        DYNAMIC_BLOCK=DYNAMIC_BLOCK,
        HTML_TABLE=tbl,
        JS_COUNT_URL=JS_COUNT_URL,
        DOWNLOADS_BLOCK=DOWNLOADS_BLOCK
    ).replace("$HEAD_LINK_URL_JS$", (STYLESHEET_HREF + "?" + CSS_VERSION))

os.makedirs("partials", exist_ok=True)
final_html_plus = build_register_html_for_abs(HOME_URL)

with open(LOCAL_HTML, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html)
print("[OK] Saved canonical render:", os.path.abspath(LOCAL_HTML))

with open(WORK_PLUS_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html_plus)
print("[OK] Saved:", os.path.abspath(WORK_PLUS_LOCAL), "(partials clone)")

# ---------- 10) Uploads ----------
def save_and_upload_all():
    if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
        print("[SKIP] Missing FTP creds; uploads skipped.")
        return
    try:
        ftps = ftp_connect()
        try:
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_CANON))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_LEG))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_SIMPLE))
        except Exception as e:
            print("[WARN] Upload main HTML failed:", e)
        try:
            if os.path.exists(LOCAL_CSV):
                ftp_upload_overwrite(ftps, LOCAL_CSV, _remote_path(REMOTE_CSV))
            if os.path.exists(LOCAL_XLSX):
                ftp_upload_overwrite(ftps, LOCAL_XLSX, _remote_path(REMOTE_XLSX))
            print("[OK] Uploaded CSV/XLSX -> /partials/")
        except Exception as e:
            print("[WARN] Upload CSV/XLSX failed:", e)
        if os.path.exists(LOCAL_COUNT_FILE):
            try:
                ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, _remote_path(REMOTE_COUNT_NAME))
            except Exception as e:
                print("[WARN] Upload autosomal count failed:", e)
        try:
            ftp_upload_overwrite(ftps, os.path.join("partials","match_count.htm"),       _remote_path(posixpath.join("partials","match_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","lineage_count.htm"),     _remote_path(posixpath.join("partials","lineage_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","cousin_list_print.htm"), _remote_path(posixpath.join("partials","cousin_list_print.htm")))
        except Exception as e:
            print("[WARN] Upload partials failed:", e)
        try:
            ftp_upload_overwrite(ftps, WORK_PLUS_LOCAL, _remote_path(WORK_PLUS_REMOTE))
        except Exception as e:
            print("[WARN] Upload work_plus.htm failed:", e)

        print("\n--- SIZE Verification (if supported) ---")
        for p in [
            _remote_path(REMOTE_HTML_CANON),
            _remote_path(REMOTE_HTML_LEG),
            _remote_path(REMOTE_HTML_SIMPLE),
            _remote_path(REMOTE_CSV),
            _remote_path(REMOTE_XLSX),
            _remote_path(posixpath.join("partials","match_count.htm")),
            _remote_path(posixpath.join("partials","lineage_count.htm")),
            _remote_path(posixpath.join("partials","cousin_list_print.htm")),
            _remote_path(WORK_PLUS_REMOTE),
        ]:
            sz = ftp_size(ftps, p)
            print(f"{p} : {sz if sz is not None else '(SIZE unsupported)'}")
        try: ftps.quit()
        except Exception: pass

        print("\n--- Open URLs ---")
        print("Canonical (Find target):          https://yates.one-name.net/partials/yates_ancestor_register.htm")
        print("Legacy clone (same content):      https://yates.one-name.net/partials/ons_yates_dna_register.htm")
        print("JUSTDNA simple alias:             https://yates.one-name.net/partials/justdna.htm")
        print("Trees page:                       https://yates.one-name.net/partials/just-trees.htm")
        print("Match Count:                      https://yates.one-name.net/partials/match_count.htm")
        print("Lineage Count:                    https://yates.one-name.net/partials/lineage_count.htm")
        print("Cousin List (Printable):          https://yates.one-name.net/partials/cousin_list_print.htm")
        print("Work+ clone:                      https://yates.one-name.net/partials/work_plus.htm")

        print("\nIf a button still opens the wrong target, hard-refresh or append ?v=1 once to bust cache.")
    except Exception as e:
        print("[FAIL] FTP session:", e)
        traceback.print_exc()

# ---------- 11) Upload ----------
save_and_upload_all()
# ====== CUT STOP [1/1] CELL 2 ===================================================================


[OK] Loaded CSV: 7 rows, 6 cols
[PULL] match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
[OK] Resolver loaded: 79 codes
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
[OK] Saved canonical render: /content/yates_ancestor_register.htm
[OK] Saved: /content/partials/work_plus.htm (partials clone)
[PUT] yates_ancestor_register.htm -> partials/yates_ancestor_register.htm
[PUT] yates_ancestor_register.htm -> partials/ons_yates_dna_register.htm
[PUT] yates_ancestor_register.htm -> partials/justdna.htm
[PUT] yates_ancestor_register.csv -> partials/yates_ancestor_register.csv
[PUT] yates_ancestor_register.xlsx -> partials/yates_ancestor_register.xlsx
[OK] Uploaded CSV/XLSX -> /partials/
[PUT] /content/autosomal_count.txt -> autosomal_count.txt
[PUT] partials/match_count.htm -> partials/match

# TEST Cell 2


In [ ]:
## ====== CUT START [1/1] CELL 2 — Build + Publish DNA Register (Unified: Best Display + Correct First-Ancestor) ======
# RON RULES — QUICK CODE CARD (v2025.11.08-Cell2-UNIFIED-BEST)
# - Complete & runnable in Colab; ASCII-only source (ISO-8859-15 safe outputs).
# - XHTML 1.0 Transitional; explicit prints; no fabrication.
# - Typography is controlled ONLY by /partials/dna_tree_styles.css (no inline font-family anywhere).
# - Row-highlight FIX is limited to PARTIALS ONLY (not the main register page).
# - Publishes SAME HTML to:
#       /partials/yates_ancestor_register.htm     (canonical, Find target)
#       /partials/ons_yates_dna_register.htm     (legacy clone)
#       /partials/justdna.htm                     (JUSTDNA alias)
# - Also writes PARTIALS (match_count.htm, lineage_count.htm, cousin_list_print.htm) and CSV/XLSX exports.

import os, re, io, posixpath, socket, traceback, urllib.parse as _u
from ftplib import FTP_TLS
import pandas as pd
import html as _html
from string import Template

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR',  '')
    os.environ.setdefault('FTP_PORT', '21')

# ---------- 1) Config ----------
CSV_IN = "final_combined_df_with_value_labels.csv"

LOCAL_HTML         = "yates_ancestor_register.htm"
REMOTE_HTML_CANON  = posixpath.join("partials", "yates_ancestor_register.htm")
REMOTE_HTML_LEG    = posixpath.join("partials", "ons_yates_dna_register.htm")
REMOTE_HTML_SIMPLE = posixpath.join("partials", "justdna.htm")

DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"
TREES_ABS        = "https://yates.one-name.net/partials/just-trees.htm"

EXPORT_BASENAME = "yates_ancestor_register"
LOCAL_CSV       = f"{EXPORT_BASENAME}.csv"
LOCAL_XLSX      = f"{EXPORT_BASENAME}.xlsx"
REMOTE_CSV      = posixpath.join("partials", os.path.basename(LOCAL_CSV))
REMOTE_XLSX     = posixpath.join("partials", os.path.basename(LOCAL_XLSX))

WORK_PLUS_LOCAL  = os.path.join("partials", "work_plus.htm")
WORK_PLUS_REMOTE = posixpath.join("partials", "work_plus.htm")

LOCAL_COUNT_FILE  = "/content/autosomal_count.txt"
REMOTE_COUNT_NAME = "autosomal_count.txt"

FTP_DIR         = (os.environ.get("FTP_DIR","") or "").strip()
TNG_BASE        = "https://yates.one-name.net/tng"
TNG_TREE        = "tree1"
HOME_URL        = "https://yates.one-name.net/partials/yates_ancestor_register.htm"
REMOTE_NAME_ABS = HOME_URL
COUNT_PUBLIC_URL = (f"/{FTP_DIR}/{REMOTE_COUNT_NAME}" if FTP_DIR else f"/{REMOTE_COUNT_NAME}")

# Layout
TABLE_WIDTH_PX = 3150
COL_A_PX       = 1100
FIND_COL_PX    = 118
ARROW_ENTITY   = "&rarr;"
REMOVE_PERIOD_AT_END = True

# Resolver (server mapping for Match-to -> Unmasked)
SERVER_PARTIALS_DIR        = "partials"
SERVER_MAPPING_BASENAME    = "match_to_unmasked.csv"
SERVER_MAPPING_REMOTE      = posixpath.join(SERVER_PARTIALS_DIR, SERVER_MAPPING_BASENAME)
SERVER_MAPPING_LOCAL_CACHE = "match_to_unmasked.server.csv"

# External stylesheet (controls typography site-wide)
STYLESHEET_HREF = "/partials/dna_tree_styles.css"
CSS_VERSION     = "v2025-11-06"  # bump to bust caches when CSS changes
HEAD_LINK       = f'<link rel="stylesheet" type="text/css" href="{STYLESHEET_HREF}?{CSS_VERSION}" />'
HEAD_LINK_URL   = STYLESHEET_HREF + "?" + CSS_VERSION  # used inside JS (print window)

# ---------- 2) FTP helpers ----------
FTP_TIMEOUT = int(os.environ.get("FTP_TIMEOUT", "30"))
FTP_PASSIVE = True

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT', 21)))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        parts = [p for p in FTP_DIR.split("/") if p]
        for p in parts:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)
    return ftps

def _remote_path(name: str) -> str:
    return posixpath.join(FTP_DIR, name) if FTP_DIR else name

def ensure_remote_dirs(ftps: FTP_TLS, remote_path: str):
    if "/" not in remote_path: return
    pwd0 = ftps.pwd()
    for seg in [p for p in remote_path.split("/")[:-1] if p and p != "."]:
        try: ftps.cwd(seg)
        except Exception:
            try: ftps.mkd(seg)
            except Exception: pass
            ftps.cwd(seg)
    ftps.cwd(pwd0)

def ftp_download_if_exists(ftps: FTP_TLS, remote_name: str, local_name: str) -> bool:
    try:
        with open(local_name, "wb") as f:
            ftps.retrbinary(f"RETR {remote_name}", f.write)
        print(f"[PULL] {remote_name} -> {os.path.abspath(local_name)}")
        return True
    except Exception as e:
        try:
            if os.path.exists(local_name): os.remove(local_name)
        except Exception: pass
        print(f"[MISS] {remote_name} ({e})")
        return False

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str):
    ensure_remote_dirs(ftps, remote_name)
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote_name}", fh)
    print(f"[PUT]  {local_path} -> {remote_name}")

def ftp_size(ftps: FTP_TLS, remote_name: str):
    try:
        sz = ftps.size(remote_name)
        return int(sz) if sz is not None else None
    except Exception:
        return None

# ---------- 3) Resolver ----------
def _read_mapping_csv(path: str) -> pd.DataFrame:
    encs = ("iso-8859-15","utf-8-sig","utf-8","cp1252","latin1")
    last = None; df = None
    for enc in encs:
        try:
            df = pd.read_csv(path, encoding=enc, dtype=str, keep_default_na=False)
            break
        except Exception as e:
            last = e
    if df is None:
        raise RuntimeError(f"Unable to read mapping CSV {path}: {last}")
    if df.shape[1] < 2:
        raise RuntimeError("Mapping CSV must have at least two columns: code, unmasked")
    df = df.iloc[:, :2].copy()
    df.columns = ["code","unmasked"]
    df["code"] = df["code"].astype(str).str.strip().str.lower()
    df["unmasked"] = df["unmasked"].astype(str).str.strip()
    df = df[df["code"] != ""].drop_duplicates(subset=["code"], keep="first")
    if df.empty:
        raise RuntimeError("Mapping CSV empty after normalization.")
    return df

def load_resolver_from_server() -> dict:
    with ftp_connect() as ftps:
        try:
            ftps.cwd(SERVER_PARTIALS_DIR)
        except Exception:
            pass
        ok = ftp_download_if_exists(ftps, SERVER_MAPPING_BASENAME, SERVER_MAPPING_LOCAL_CACHE)
        try: ftps.quit()
        except Exception: pass
    if not ok:
        raise RuntimeError(
            "Resolver not found on server: /" + _remote_path(SERVER_MAPPING_REMOTE) +
            ". Upload match_to_unmasked.csv into /partials/ and re-run."
        )
    df_map = _read_mapping_csv(SERVER_MAPPING_LOCAL_CACHE)
    print(f"[OK] Resolver loaded: {len(df_map)} codes")
    return dict(zip(df_map["code"], df_map["unmasked"]))

MATCH_TO_UNMASKED = {}
def _setup_resolver():
    global MATCH_TO_UNMASKED
    if not MATCH_TO_UNMASKED:
        MATCH_TO_UNMASKED = load_resolver_from_server()

def resolve_match_to(code: str) -> str:
    if not isinstance(code, str): return ""
    return MATCH_TO_UNMASKED.get(code.strip().lower(), code)

# ---------- 4) Name & text utils (UNIFIED: learnings from both versions) ----------
SEP_RE = re.compile(r"\s*(?:\u2192|&rarr;|;|>|,|~{2,}|/{2,}|\|{2,})\s*")
def split_tokens(s):
    if pd.isna(s): return []
    if not isinstance(s, str): s = str(s)
    return [p.strip() for p in SEP_RE.split(s) if str(p).strip()]

def _clean_piece(text: str) -> str:
    t = re.sub(r'~+', ' ', str(text)); t = re.sub(r'\s+', ' ', t)
    return t.strip()

_PARTICLES = {"de","del","della","der","van","von","da","dos","das","di","la","le","du","of"}

def _smart_title(token: str) -> str:
    if not token: return token
    token = re.sub(r"(^|\b)([a-z])(['’])([a-z])", lambda m: m.group(1)+m.group(2).upper()+m.group(3)+m.group(4).upper(), token.lower())
    token = "-".join([w.capitalize() for w in token.split("-")])
    token = re.sub(r"\bmc([a-z])",  lambda m: "Mc"+m.group(1).upper(),  token)
    token = re.sub(r"\bmac([a-z])", lambda m: "Mac"+m.group(1).upper(), token)
    return token

def smart_titlecase(name: str) -> str:
    name = _clean_piece(name)
    if not name: return name
    if "," in name:
        last, first = [p.strip() for p in name.split(",", 1)]
        pieces = (first + " " + last).split()
    else:
        pieces = name.split()
    out = []
    for i, w in enumerate(pieces):
        out.append(w.lower() if (i>0 and w.lower() in _PARTICLES) else _smart_title(w))
    return " ".join(out)

# Split CamelCase tokens like "YatesJohn" -> ("Yates","John") then render "John Yates"
_CAMEL_WORDS = re.compile(r"[A-Z][a-z]*|[A-Z]+(?![a-z])|[a-z]+")
def surname_given_from_token(token):
    token = token.strip()
    if not token: return (token,)
    idx = None
    for i in range(1, len(token)):
        if token[i-1].islower() and token[i].isupper():
            idx = i; break
    if idx is None:
        for i in range(1, len(token)):
            if token[i].isupper():
                idx = i; break
    if idx is None: return (token,)
    surname = token[:idx]; given = token[idx:]
    given_spaced = re.sub(r'(?<!^)([A-Z])', r' \1', given)
    return (f"{given_spaced.strip()} {surname.strip()}",)

def normalize_person_name(s: str) -> str:
    if pd.isna(s): return ""
    s = _clean_piece(str(s))
    if "," in s:
        last, first = [p.strip() for p in s.split(",", 1)]
        s = f"{first} {last}"
    if " " not in s and s.isalpha():
        return smart_titlecase(surname_given_from_token(s)[0])
    return smart_titlecase(s)

def norm_matchee_name(raw: str) -> str:
    raw = str(raw or "").strip()
    if not raw: return ""
    if " " in raw or "," in raw:
        nm = smart_titlecase(raw)
        parts = nm.split()
        if len(parts) == 1: return nm
        return f"{parts[0]} {parts[-1]}".strip()
    words = _CAMEL_WORDS.findall(raw)
    while words and len(words[0]) == 1:
        words.pop(0)
    if not words:
        nm = smart_titlecase(surname_given_from_token(raw)[0])
        ps = nm.split()
        if len(ps) == 1: return nm
        return f"{ps[0]} {ps[-1]}".strip()
    surname = smart_titlecase(words[0])
    given_candidates = [w for w in words[1:] if w.lower() != surname.lower()]
    if not given_candidates: return surname
    given = smart_titlecase(given_candidates[0])
    return f"{given} {surname}".strip()

def truncate_first(name: str, n: int = 7) -> str:
    name = name.strip()
    if not name: return name
    parts = name.split()
    return parts[0][:n] if len(parts) == 1 else f"{parts[0][:n]} {parts[-1]}"

def derive_common_from_first_token(tokens):
    if not tokens: return ("","")
    first = _clean_piece(tokens[0])
    parts = re.split(r"\s*(?:&| and )\s*", first, maxsplit=1, flags=re.I)
    if len(parts) != 2: return ("","")
    def _norm(s):
        # Accept "YatesJohn" and "Yates, John" and plain "John Yates"
        return smart_titlecase(s) if " " in s or "," in s else smart_titlecase(surname_given_from_token(s)[0])
    return (_norm(parts[0]), _norm(parts[1]))

def degree_label_from_generations(g):
    if g <= 1: return ("parents" if g == 1 else "self")
    if g == 2: return "grandparents"
    greats = g - 2
    if greats == 1: return "great-grandparents"
    return f"{greats}x-great-grandparents"

def build_header(subject_name, cm_val, matchee_name_html, gens, husband, wife):
    try:
        cm_str = f"{int(round(float(cm_val)))}"
    except Exception:
        cm_str = (str(cm_val).strip() or "0")
    degree_label = degree_label_from_generations(gens)
    parts = [
        f"{subject_name} is a {cm_str} cM cousin match to {matchee_name_html}, whose",
        f"{degree_label} (back {gens} Gens)",
        "are",
        f"{husband} & {wife}."
    ]
    s = " ".join(parts)
    if REMOVE_PERIOD_AT_END: s = re.sub(r'\.\s*$', '', s)
    return s

# ---------- 5) Read CSV; detect columns ----------
def find_col(df, patterns, prefer_exact=None):
    cols = list(df.columns)
    lowmap = {c.lower(): c for c in cols}
    if prefer_exact:
        for name in prefer_exact:
            if name in df.columns: return name
            if name and name.lower() in lowmap: return lowmap[name.lower()]
    for pat in patterns:
        rx = re.compile(pat, re.I)
        for c in cols:
            if rx.search(c): return c
    return None

_encs = ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1")
_last_err = None
df = None
for _e in _encs:
    try:
        df = pd.read_csv(CSV_IN, encoding=_e, dtype=str, keep_default_na=False)
        break
    except Exception as _ex:
        _last_err = _ex
        df = None
if df is None:
    raise RuntimeError(f"Unable to read CSV: {CSV_IN} ({_last_err})")
print(f"[OK] Loaded CSV: {len(df)} rows, {len(df.columns)} cols")

id_col    = find_col(df, [r'^(id#|personid)$'], ["ID#","ID","PersonID","personID"])
match_col = find_col(df, [r'^match\s*to$'], ["Match to","Match"])
name_col  = find_col(df, [r'^name$'], ["Name"])
cm_col    = find_col(df, [r'^(c\s*:?m|cm)$', r'centi.?morgan'], ["cM","cm"])
path_col  = find_col(df, [r'(yates\s*dna\s*ancestral\s*line|ancestral\s*line|lineage)'], ["Yates DNA Ancestral Line","Ancestral Line","Lineage"])

if not id_col:    raise ValueError("CSV missing an ID#/PersonID column.")
if not match_col: raise ValueError("CSV missing 'Match to' column.")
if not name_col:  raise ValueError("CSV missing 'Name' column.")
if not cm_col:    raise ValueError("CSV missing 'cM' column.")
if not path_col:  raise ValueError("CSV missing lineage/path column.")

# ---------- 6) Transform -> display_df ----------
ID_PAT = re.compile(r"\bI\d+\b", re.I)
def extract_person_id(s: str) -> str:
    m = ID_PAT.search(str(s or ""))
    return m.group(0).upper() if m else ""

_setup_resolver()

headers, lineages, findcol = [], [], []
subjects, first_ancestors  = [], []

for _, row in df.iterrows():
    subject_raw  = row.get(match_col, "")
    subject_name = normalize_person_name(resolve_match_to(subject_raw))
    subject_name_b = f"<strong>{subject_name}</strong>" if subject_name else subject_name

    pid = extract_person_id(row.get(id_col, ""))
    matchee_name = norm_matchee_name(row.get(name_col, "")) or subject_name

    if pid:
        matchee_name_html = (
            f'<a href="{TNG_BASE}/verticalchart.php?personID={pid}&tree={TNG_TREE}&parentset=0&display=vertical&generations=15" '
            f'target="_blank" rel="noopener">{matchee_name}</a>'
        )
    else:
        matchee_name_html = matchee_name

    cm_val = row.get(cm_col, "0")
    tokens = split_tokens(row.get(path_col, ""))
    gens_total = len(tokens)
    tokens_disp = tokens[:7]

    # Prefer explicit husband/wife columns if present, else derive from first token (CamelCase aware)
    if "common_husband" in df.columns and "common_wife" in df.columns:
        husband_raw = str(row.get("common_husband","")).strip()
        wife_raw    = str(row.get("common_wife","")).strip()
        if not husband_raw and not wife_raw:
            husband_raw, wife_raw = derive_common_from_first_token(tokens)
    else:
        husband_raw, wife_raw = derive_common_from_first_token(tokens)

    header_html = build_header(
        subject_name_b,
        cm_val,
        matchee_name_html,
        gens_total,
        husband_raw,
        wife_raw
    )

    if tokens_disp:
        tokens_disp[0] = f"<strong>{tokens_disp[0]}</strong>"
    sep = f" {ARROW_ENTITY} "
    lineage_text = sep.join(tokens_disp) if tokens_disp else ""

    q = _u.quote(subject_name)
    quick = (
        f'<a class="find-btn" href="{REMOTE_NAME_ABS}?q={q}" target="_blank" rel="noopener" '
        f'title="Open a filtered view for {subject_name}">Find</a>'
    )

    headers.append(header_html)
    lineages.append(lineage_text)
    findcol.append(quick)
    subjects.append(subject_name)
    first_ancestors.append(tokens[0] if tokens else "")

LINEAGE_HEADER_SAFE = "Lineage (Starting with oldest ancestor, the line is:)"
df["Match Summary"]     = headers
df[LINEAGE_HEADER_SAFE] = lineages
df["Find"]              = findcol
df["Subject"]           = subjects
df["First Ancestor"]    = [_clean_piece(x) for x in first_ancestors]
display_df = df[["Find","Match Summary", LINEAGE_HEADER_SAFE]]

# ---------- 6.1) Clean exports (ISO-8859-15 safe; robust) ----------
TAG_RE = re.compile(r"<[^>]+>")
def _html_to_text(s: str) -> str:
    t = TAG_RE.sub("", str(s or ""))
    t = _html.unescape(t)
    t = t.replace("\u2192", "->")
    return re.sub(r"\s+", " ", t).strip()

def _extract_find_url(cell_html: str) -> str:
    m = re.search(r'href="([^"]+)"', str(cell_html or ""))
    return _html.unescape(m.group(1)) if m else ""

export_df = pd.DataFrame({
    "Find URL":      [ _extract_find_url(v) for v in display_df["Find"].tolist() ],
    "Match Summary": [ _html_to_text(v)     for v in display_df["Match Summary"].tolist() ],
    "Lineage":       [ _html_to_text(v)     for v in display_df[LINEAGE_HEADER_SAFE].tolist() ],
})

# Write CSV with fallback error handler for older pandas/envs
try:
    export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
except TypeError:
    # Older pandas may not recognize encoding error args; do manual encode
    buf = io.StringIO()
    export_df.to_csv(buf, index=False)
    with open(LOCAL_CSV, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(buf.getvalue())

try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    with pd.ExcelWriter(LOCAL_XLSX) as _writer:
        export_df.to_excel(_writer, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))

# ---------- 7) HTML (Register main page; no inline font-family) ----------
TABLE_CSS = """
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { font-size:100%; background:#ffffff; color:#222; margin:0; padding:0; line-height:1.5; }
  .wrap { max-width:3150px; margin:0 auto; background:#ffffff; padding:20px; padding-bottom:48px; }
  a { color:#154b8b; text-decoration:none; } a:hover { text-decoration:underline; }
  h1 { margin:0 0 4px 0; font-size:26px; line-height:1.2; text-align:center; }
  .centerline { text-align:center; }
  .downloads { text-align:center; margin:4px 0 10px 0; font-size:13px; }
  .updated { font-size:12px; color:#555; text-align:center; margin:2px 0 10px 0; }
  .sortbar { margin:6px 0 10px 0; font-size:13px; background:#ffffff; padding:6px 8px; border-radius:6px; display:flex; flex-wrap:wrap; gap:5px; align-items:center; border:1px solid #ddd; justify-content:center; }
  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px; text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; }
  .btn:hover { background:#4668aa; }
  input.btn.search { background:#fff; color:#111; border-color:#bbb; }
  .find-cell { white-space:nowrap; }
  .selbox { margin-right:6px; vertical-align:middle; }
  .table-scroll { max-height:70vh; overflow-y:auto; overflow-x:auto; border:1px solid #ddd; }
  table.sortable { border-collapse:collapse; width:3150px; table-layout:fixed; }
  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; }
  table.sortable th { background:#e3eaf8; text-align:left; position:sticky; top:0; z-index:2; box-shadow:0 1px 0 #ccc; cursor:pointer; }
  #first-row td { border-top:2px solid #999; }
  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px; border:1px solid #3e5a97; background:#5b79b8; color:#fff; cursor:pointer; border-radius:6px; font-size:12px; display:none; z-index:9999; }
  .back-to-top:hover { background:#4668aa; }
  #dynamicContent { margin:10px 0 14px 0; }
  @media screen and (max-width: 820px) { .wrap { padding:12px; } h1 { font-size:22px; } }
</style>
"""

DYNAMIC_BLOCK = (
    '<div class="sortbar">'
    '<a class="btn" href="https://yates.one-name.net/gengen/dna_cousin_surname_study.htm" target="_blank" rel="noopener">Study Details</a>'
    '<a class="btn" href="https://yates.one-name.net/gengen/dna_theory_of_the_case.htm" target="_blank" rel="noopener">Theory in Action</a>'
    '<a class="btn" href="/gengen/images/cousin-calculator.jpg" target="_blank" rel="noopener">Cousin Connection</a>'
    '<a class="btn" href="/gengen/images/Shared_cM_Project_v4.jpg" target="_blank" rel="noopener">Cousin by DNA</a>'
    '<a class="btn" href="/partials/match_count.htm" target="_blank" rel="noopener">Match Count</a>'
    '<a class="btn" href="/partials/lineage_count.htm" target="_blank" rel="noopener">Lineage Count</a>'
    f'<a class="btn" href="{DNA_REGISTER_ABS}" target="_blank" rel="noopener">DNA Register</a>'
    f'<a class="btn" href="{TREES_ABS}" target="_blank" rel="noopener">Trees</a>'
    '<span class="btn" id="show-selected" title="Show all rows for the checked name(s)">Show Selected</span>'
    '<span class="btn" id="show-all" title="Show All">Show All</span>'
    '<span class="btn" id="print-cousin-list" style="cursor:pointer;" title="Open a printable list of the *currently visible* rows">Cousin List (Printable)</span>'
    '<span class="btn" id="clear-selected">Reset</span>'
    '<input type="text" id="search-box" class="btn search" size="24" value="" placeholder="Search&amp;hellip;" />'
    '</div>'
)

JS_COUNT_URL = COUNT_PUBLIC_URL.replace("'", "%27")
UPDATED_BLOCK = (
    '<div class="updated centerline">'
    'Last updated: <span id="last-updated"></span>'
    ' &nbsp;|&nbsp; Autosomal matches: <span id="auto-count" class="js-count"></span>'
    ' &nbsp;|&nbsp; Showing: <span id="showing-count"></span>'
    '</div>'
)

DOWNLOADS_BLOCK = (
    '<p class="downloads centerline">Download: '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_CSV))}">CSV</a> | '
    f'<a href="/partials/{_html.escape(os.path.basename(LOCAL_XLSX))}">Excel</a></p>'
)

page_tpl = Template("""<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>ONS Yates Study Autosomal DNA Register</title>
$HEAD_LINK
$TABLE_CSS
<script type="text/javascript">var HEAD_LINK_URL = "$HEAD_LINK_URL";</script>
</head>
<body id="top">
<div class="wrap">
  <h1 class="centerline">ONS Yates Study Autosomal DNA Register</h1>
  $DOWNLOADS_BLOCK
  $UPDATED_BLOCK
  $DYNAMIC_BLOCK
  <div class="table-scroll">
    $HTML_TABLE
  </div>
</div>
<button id="back-to-top" class="back-to-top">&#9650; Top</button>
<script type="text/javascript">
//<![CDATA[
(function(){
  function textOf(cell){ return (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').trim().toLowerCase(); }
  function sortTable(tbl, colIndex, dir){
    var tb=tbl && tbl.tBodies ? tbl.tBodies[0] : null; if(!tb) return;
    var rows=[].slice.call(tb.rows||[]); var asc=(dir==='asc');
    rows.sort(function(a,b){var A=textOf(a.cells[colIndex]),B=textOf(b.cells[colIndex]); if(A<B) return asc?-1:1; if(A>B) return asc?1:-1; return 0;});
    var frag=document.createDocumentFragment(); for(var i=0;i<rows.length;i++) frag.appendChild(rows[i]); tb.appendChild(frag);
    updateShowing();
  }
  function bindHeaderSort(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
    var ths=tbl.tHead.rows[0].cells; if(!ths) return;
    for(var i=0;i<ths.length;i++){(function(idx){
      var th = ths[idx]; var dir='asc';
      th.addEventListener('click',function(){
        dir=(dir==='asc')?'desc':'asc';
        for (var j = 0; j < ths.length; j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+[↑↓]/,''); }
        th.innerHTML += (dir==='asc' ? ' &uarr;' : ' &darr;');
        sortTable(tbl,idx,dir);
      },false);
    })(i);}
  }
  function stampLastUpdated(){
    var el = document.getElementById('last-updated'); if(!el) return;
    var d  = new Date(document.lastModified || new Date());
    var months = ['January','February','March','April','May','June','July','August','September','October','November','December'];
    el.innerHTML = d.getDate() + ' ' + months[d.getMonth()] + ' ' + d.getFullYear();
  }
  function formatWithCommas(n){
    try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
  }
  function visibleRowCount(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return 0;
    var rows=tbl.tBodies[0].rows, n=0;
    for(var i=0;i<rows.length;i++){ if(rows[i].style.display!=='none') n++; }
    return n;
  }
  function updateShowing(){
    var el=document.getElementById('showing-count'); if(!el) return;
    el.textContent = formatWithCommas(visibleRowCount());
  }
  function loadAutoCount(){
    var el=document.getElementById('auto-count'); if(!el) return;
    var url='$JS_COUNT_URL';
    try{
      var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
      xhr.onreadystatechange=function(){ if(xhr.readyState===4){
        if(xhr.status>=200&&xhr.status<300){
          var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
          el.textContent = formatWithCommas(num) || '(unavailable)';
        } else { el.textContent='(unavailable)'; }
      }};
      xhr.send(null);
    }catch(e){ el.textContent='(unavailable)'; }
  }
  function getParam(name){ var m=location.search.match(new RegExp('[?&]'+name+'=([^&]+)')); return m?decodeURIComponent(m[1].replace(/\\+/g,' ')):''; }
  function norm(s){ return String(s||'').replace(/\\s+/g,' ').toLowerCase(); }
  function bindSearch(){
    var box=document.getElementById('search-box'); var tbl=document.getElementById('refactor-table'); if(!(box && tbl && tbl.tBodies && tbl.tBodies[0])) return; var tb=tbl.tBodies[0];
    function rowText(tr){ var t=''; for(var i=1;i<tr.cells.length;i++){ t+= ' ' + (tr.cells[i].textContent||tr.cells[i].innerText||''); } return norm(t); }
    var cached=[]; (function seed(){ var rows=tb.rows; cached=[]; for(var i=0;i<rows.length;i++){ cached.push({el:rows[i], txt:rowText(rows[i])}); } })();
    function apply(q){
      q = String(q||''); var parts = q.split('|').map(function(s){return norm(s);}).filter(function(s){return !!s;});
      for(var i=0;i<cached.length;i++){
        var hit = !parts.length || parts.some(function(p){ return cached[i].txt.indexOf(p)>-1; });
        cached[i].el.style.display = hit? '' : 'none';
      }
      updateShowing();
    }
    var to=null; function onInput(){ if(to) clearTimeout(to); to=setTimeout(function(){ apply(box.value); }, 60); }
    box.addEventListener('input', onInput, false);
    box.addEventListener('search', onInput, false);
    var q0=getParam('q');
    if(q0){ box.value=q0; apply(q0); try{ history.replaceState(null, '', location.pathname); }catch(e){} }
    else { box.value=''; apply(''); setTimeout(function(){ if(!getParam('q')){ box.value=''; apply(''); } }, 0); }
  }
  function allRowCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return [];
    var tb=tbl.tBodies[0], out=[];
    for(var i=0;i<tb.rows.length;i++){
      var cb=tb.rows[i].querySelector('.selbox');
      if(cb) out.push(cb);
    }
    return out;
  }
  function bindGroupSync(){
    var tbl=document.getElementById('refactor-table'); if(!tbl) return;
    tbl.addEventListener('click', function(e){
      var t=e.target||e.srcElement;
      if(!(t && t.classList && t.classList.contains('selbox'))) return;
      var nm = t.getAttribute('data-name') || '';
      var checked = !!t.checked;
      var cbs = allRowCheckboxes();
      for(var i=0;i<cbs.length;i++){
        if((cbs[i].getAttribute('data-name')||'') === nm){ cbs[i].checked = checked; }
      }
      updateShowing();
    }, false);
  }
  function bindShowSelected(){
    var btn=document.getElementById('show-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs = allRowCheckboxes();
      var names = {};
      for(var i=0;i<cbs.length;i++){ if(cbs[i].checked){ names[cbs[i].getAttribute('data-name')||''] = true; } }
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var r=0;r<tb.rows.length;r++){
        var cb = tb.rows[r].querySelector('.selbox');
        var nm = cb ? (cb.getAttribute('data-name')||'') : '';
        tb.rows[r].style.display = names[nm] ? '' : 'none';
      }
      updateShowing();
    }, false);
  }
  function bindShowAll(){
    var btn=document.getElementById('show-all'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
      var tb=tbl.tBodies[0];
      for(var i=0;i<tb.rows.length;i++){ tb.rows[i].style.display=''; }
      updateShowing();
    }, false);
  }
  function bindClear(){
    var btn=document.getElementById('clear-selected'); if(!btn) return;
    btn.addEventListener('click', function(){
      var cbs=allRowCheckboxes(); for(var i=0;i<cbs.length;i++) cbs[i].checked=false;
      var tbl=document.getElementById('refactor-table'); if(tbl && tbl.tBodies && tbl.tBodies[0]){
        var tb=tbl.tBodies[0]; for(var j=0;j<tb.rows.length;j++){ tb.rows[j].style.display=''; }
      }
      updateShowing();
    }, false);
  }
  function addCheckboxes(){
    var tbl=document.getElementById('refactor-table'); if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
    var tb=tbl.tBodies[0];
    for(var i=0;i<tb.rows.length;i++){
      var tr=tb.rows[i]; var cell=tr.cells[0]; var findBtn=cell ? cell.querySelector('.find-btn') : null;
      var name = findBtn ? (findBtn.getAttribute('title')||'').replace('Open a filtered view for ','') : ('Row '+(i+1));
      if(cell){
        cell.classList.add('find-cell');
        cell.innerHTML = '<input type="checkbox" class="selbox" title="Select this row" data-name="'+name.replace(/"/g,'&quot;')+'" /> ' + cell.innerHTML.replace(/^\\s*/, '');
      }
    }
  }
  function bindPrintCousinList(){
    var btn=document.getElementById('print-cousin-list'); if(!btn) return;
    btn.addEventListener('click', function(){
      var tbl=document.getElementById('refactor-table'); if(!tbl) return;
      var headerHtml = ''; try{ headerHtml = tbl.tHead.innerHTML; }catch(e){}
      var tb=tbl.tBodies[0];
      var visibleRowsHtml = ''; var visibleCount = 0;
      if(tb){
        for (var i = 0; i < tb.rows.length; i++) {
          if (tb.rows[i].style.display !== 'none') {
            visibleRowsHtml += tb.rows[i].outerHTML;
            visibleCount++;
          }
        }
      }
      var css = '<style type="text/css">' +
        "body { font-size:12px; margin: 20px; }" +
        "h1 { font-size:20px; text-align:center; }" +
        "table { border-collapse:collapse; width:100%; table-layout:fixed; }" +
        "th, td { border:1px solid #999; padding: 5px 7px; vertical-align:top; text-align:left; word-wrap:break-word; }" +
        "th { background:#f0f0f0; }" +
        "a { color:#000; text-decoration:none; }" +
        "th:first-child, td:first-child { display:none; }" +
        "th:nth-child(2), td:nth-child(2) { width: 40% !important; }" +
        "th:nth-child(3), td:nth-child(3) { width: 60% !important; }" +
        '</style>';
      var link = '<link rel="stylesheet" type="text/css" href="'+(window.HEAD_LINK_URL || (typeof HEAD_LINK_URL!=='undefined'?HEAD_LINK_URL:''))+'" />';
      var tableHtml = '<table border="1">' + '<thead>' + headerHtml + '</thead><tbody>' + visibleRowsHtml + '</tbody></table>';
      var docHtml = '<html><head><title>Cousin List (Filtered)</title>' + link + css + '</head><body onload="window.print(); window.close();">' +
                    '<h1>Cousin List</h1>' +
                    '<p>Showing ' + visibleCount + ' filtered records.</p>' +
                    tableHtml +
                    '</body></html>';
      var win = window.open('', 'CousinPrint');
      win.document.open(); win.document.write(docHtml); win.document.close(); win.focus();
    }, false);
  }
  function initShowingStatic(){ try{ document.getElementById('showing-count').textContent = document.getElementById('refactor-table').tBodies[0].rows.length; }catch(e){} }
  document.addEventListener('DOMContentLoaded', function(){
    addCheckboxes();
    stampLastUpdated();
    loadAutoCount();
    bindHeaderSort();
    bindSearch();
    bindGroupSync();
    bindShowSelected();
    bindShowAll();
    bindClear();
    bindPrintCousinList();
    initShowingStatic();
  });
})();
 //]]>
</script>
</body>
</html>
""")

html_table = display_df.to_html(index=False, escape=False, classes="dataframe sortable")
html_table = html_table.replace('<table border="1" class="dataframe sortable">', '<table border="1" class="dataframe sortable" id="refactor-table">', 1)
html_table = html_table.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
html_table = html_table.replace("<th>Find</th>", '<th>Select:</th>', 1)
html_table = html_table.replace('<th>Match Summary</th>', '<th>Match Summary&amp;ndash;click to sort</th>', 1)
html_table = html_table.replace(f'<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>', '<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>', 1)

colgroup_html = (
    "<colgroup>\n"
    f"  <col style=\"width:{FIND_COL_PX}px;\" />\n"
    f"  <col style=\"width:{COL_A_PX}px;\" />\n"
    "  <col />\n"
    "</colgroup>\n"
)
html_table = html_table.replace(
    '<table border="1" class="dataframe sortable" id="refactor-table">',
    '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html,
    1
)

final_html = page_tpl.safe_substitute(
    HEAD_LINK=HEAD_LINK,
    TABLE_CSS=TABLE_CSS,
    UPDATED_BLOCK=UPDATED_BLOCK,
    DYNAMIC_BLOCK=DYNAMIC_BLOCK,
    HTML_TABLE=html_table,
    JS_COUNT_URL=JS_COUNT_URL,
    DOWNLOADS_BLOCK=DOWNLOADS_BLOCK,
    HEAD_LINK_URL=HEAD_LINK_URL
)

# ---------- 8) PARTIALS (Row-color highlight fix applies here only) ----------
def _norm_code_for_count(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = re.sub(r'\s+', ' ', t).strip()
    t = t.lower()
    return t

def _partial_css_wrapper_simple():
    return (
        HEAD_LINK +
        "<style type=\"text/css\">\n"
        "  html { scroll-behavior: smooth; }\n"
        f"  .wrap {{ max-width:{TABLE_WIDTH_PX}px; margin:0 auto; background:#ffffff; padding:20px 20px 18px 20px; }}\n"
        "  a { text-decoration:none; } a:hover { text-decoration:underline; }\n"
        "  h1 { margin:0 0 8px 0; font-size:24px; line-height:1.2; text-align:center; }\n"
        "  .meta { text-align:center; font-size:12px; color:#555; margin:2px 0 10px 0; }\n"
        "  .toolbar { display:flex; gap:10px; align-items:center; margin:6px 0 10px 0; flex-wrap:wrap; justify-content:center; }\n"
        "  .btn { display:inline-block; border:1px solid #5b79b8; background:#5b79b8; color:#fff; padding:4px 9px; text-decoration:none; cursor:pointer; border-radius:5px; line-height:1.2; }\n"
        "  .btn:disabled { opacity:0.5; cursor:not-allowed; }\n"
        "  table { border-collapse:collapse; width:100%; table-layout:auto; }\n"
        "  th, td { border:1px solid #ddd; padding:6px 8px; vertical-align:top; text-align:left; }\n"
        "  th { background:#e3eaf8; position:sticky; top:0; z-index:2; }\n"
        "  /* Row selection highlight FIX (partials only) */\n"
        "  tbody tr.sel td { background:#fff7d6 !important; }\n"
        "  tbody tr:hover td { background:#f9f6e8; }\n"
        "  .count a { font-weight:bold; }\n"
        "</style>\n"
    )

def _partial_head(title):
    return (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\"\n"
        "  \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">\n"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\">\n<head>\n"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />\n"
        f"<title>{_html.escape(title)}</title>\n"
    )

def _partial_toolbar():
    safe_home = HOME_URL.replace('"','&quot;')
    return (
        "<div class=\"toolbar\">"
        f"<a class=\"btn\" href=\"{safe_home}\" target=\"_blank\" rel=\"noopener\">DNA Register</a>"
        " <button id=\"mc-show-selected\" class=\"btn\" title=\"Open DNA Register filtered to selected\">Show Selected</button>"
        " <button id=\"mc-show-all\" class=\"btn\" title=\"Show all rows (this table)\">Show All</button>"
        " <button id=\"mc-reset\" class=\"btn\" title=\"Clear selection and show all\">Reset</button>"
        " <button id=\"view\" class=\"btn\" title=\"Open DNA Register with selected (alias)\">View Now</button>"
        "</div>\n"
    )

def _partial_js_common():
    _safe_home = HOME_URL.replace("'", "%27")
    safe_count = COUNT_PUBLIC_URL.replace("'", "%27")
    return (
        "<script type=\"text/javascript\">\n//<![CDATA[\n"
        "(function(){\n"
        "  var REG = '" + _safe_home + "';\n"
        "  function fmt(n){ try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return '0'; return x.toLocaleString('en-US'); }catch(e){ return String(n||'0'); } }\n"
        "  function selected(){ var out=[]; var tb=document.getElementById('ref-tb'); if(!tb) return out; var rows=tb.rows; for(var i=0;i<rows.length;i++){ if((' '+rows[i].className+' ').indexOf(' sel ')>-1) out.push(rows[i]); } return out; }\n"
        "  function update(){ var sel=selected(), sum=0; for(var i=0;i<sel.length;i++){ var v=parseInt((sel[i].getAttribute('data-count')||'0').replace(/[^0-9\\-]/g,''),10); if(!isNaN(v)) sum+=v; } var nEl=document.getElementById('sel-n'); var sEl=document.getElementById('sel-sum'); if(nEl) nEl.innerHTML=fmt(sel.length); if(sEl) sEl.innerHTML=fmt(sum); }\n"
        "  function qJoin(parts){ var out=[]; var seen={}; for(var i=0;i<parts.length;i++){ var p=String(parts[i]||''); if(p && !seen[p]){ seen[p]=1; out.push(encodeURIComponent(p)); } } return out.join('%7C'); }\n"
        "  function openRegisterForSelected(){ var sel=selected(); if(!sel.length) return; var qs=[]; for(var i=0;i<sel.length;i++){ qs.push(sel[i].getAttribute('data-q')||''); } var q = qJoin(qs); var url = REG + '?q=' + q; var w=null; try{ w=window.open(url,'RegisterFiltered'); if(!w) throw new Error('popup'); w.focus(); } catch(e){ window.location.href = url; } }\n"
        "  function toggleFrom(el){ var tr=el; while(tr && tr.nodeName && tr.nodeName.toLowerCase()!=='tr'){ tr=tr.parentNode; } if(!tr) return; var c=tr.className||''; tr.className = ((' '+c+' ').indexOf(' sel ')>-1) ? c.replace(/\\bsel\\b/,'').replace(/\\s{2,}/g,' ').trim() : (c?c+' ':'')+'sel'; update(); }\n"
        "  document.addEventListener('click', function(e){ var t=e.target||e.srcElement; if(!t) return; if(t.classList && t.classList.contains('count-pick')){ e.preventDefault(); toggleFrom(t); return; } if(t.id=='view' || t.id=='mc-show-selected'){ e.preventDefault(); openRegisterForSelected(); return; } if(t.id=='mc-reset'){ e.preventDefault(); var tb=document.getElementById('ref-tb'); if(tb){ var rows=tb.rows; for(var i=0;i<rows.length;i++){ rows[i].className = rows[i].className.replace(/\\bsel\\b/,'').replace(/\\s{2,}/g,' ').trim(); rows[i].style.display=''; } } update(); return; } if(t.id=='mc-show-all'){ e.preventDefault(); var tb2=document.getElementById('ref-tb'); if(!tb2) return; for(var k=0;k<tb2.rows.length;k++){ tb2.rows[k].style.display=''; } return; } }, false);\n"
        "  function stamp(){var el=document.getElementById('last-updated'); if(!el) return; var d=new Date(document.lastModified||new Date()); function z(n){return (n<10?'0':'')+n;} el.innerHTML=d.getFullYear()+'-'+z(d.getMonth()+1)+'-'+z(d.getDate())+' '+z(d.getHours())+':'+z(d.getMinutes());}\n"
        "  function load(){var el=document.getElementById('auto-count'); if(!el) return; var URL='" + safe_count + "'; try{var xhr=new XMLHttpRequest(); xhr.open('GET', URL+(URL.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true); xhr.onreadystatechange=function(){if(xhr.readyState===4){if(xhr.status>=200&&xhr.status<300){var m=(xhr.responseText||'').match(/(\\d+)/); el.textContent=(m?m[1]:'');} else {el.textContent='(unavailable)';}}}; xhr.send(null);}catch(e){el.textContent='(unavailable)';}}\n"
        "  document.addEventListener('DOMContentLoaded', function(){ stamp(); load(); update(); }, false);\n"
        "})();\n"
        "//]]>\n</script>\n"
    )

def _shell_open(title):
    return (
        _partial_head(title) +
        _partial_css_wrapper_simple() +
        "</head>\n<body>\n<div class=\"wrap\">\n" +
        f"<h1>{_html.escape(title)}</h1>\n" +
        "<div class=\"meta\">"
        "Last updated: <span id=\"last-updated\"></span>"
        " &nbsp;|&nbsp; Autosomal matches: <span id=\"auto-count\">&mdash;</span>"
        " &nbsp;|&nbsp; Selected: <span id=\"sel-n\">0</span> &nbsp; Sum: <span id=\"sel-sum\">0</span>"
        "</div>\n" +
        _partial_toolbar() +
        "<div class=\"table-scroll\">\n"
    )

def _shell_close():
    return (
        "</div>\n</div>\n" +
        _partial_js_common() +
        "</body>\n</html>"
    )

def _html_escape_text(s):
    return _html.escape(str(s), quote=True)

def build_match_count_partial(main_df: pd.DataFrame) -> str:
    codes_raw = main_df[match_col].astype(str).map(lambda x: x.strip())
    keys_norm = codes_raw.map(_norm_code_for_count)
    counts_series = keys_norm.value_counts(dropna=False)
    counts = counts_series.reset_index()
    if counts.shape[1] >= 2:
        counts.columns = ["norm_key", "Count"]
    else:
        counts["norm_key"] = counts.index.astype(str)
        counts["Count"] = counts_series.values
        counts = counts[["norm_key","Count"]]
    first_display = {}
    for code_disp, k in zip(codes_raw.tolist(), keys_norm.tolist()):
        if k not in first_display and str(k) != "":
            first_display[k] = code_disp
    counts["Code"] = counts["norm_key"].map(lambda k: first_display.get(k, k))
    counts["Unmasked"] = counts["norm_key"].map(lambda k: MATCH_TO_UNMASKED.get(k, ""))
    counts = counts.sort_values(by=["Code","Count"], ascending=[True, False], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_shell_open("Match Count"))
    html.append('<table id="ref-table" border="1" class="sortable" style="width:100%"><thead><tr>')
    html.append('<th style="width:35%">Code</th><th style="width:45%">Unmasked</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in counts.iterrows():
        code = r.get("Code","")
        unm  = r.get("Unmasked","")
        cnt  = int(str(r.get("Count","0")).strip() or "0")
        label = (unm or code).strip()
        tr = (
            f'<tr data-q="{_html_escape_text(label)}" data-count="{cnt}">'
            f'<td>{_html_escape_text(code)}</td>'
            f'<td>{_html_escape_text(unm)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td>'
            f'</tr>'
        )
        html.append(tr)
    html.append('</tbody></table>')
    html.append(_shell_close())
    return "".join(html)

def build_lineage_count_partial(main_df: pd.DataFrame) -> str:
    first_series = main_df.get("First Ancestor", pd.Series(dtype=str)).astype(str).map(lambda x: x.strip())
    vc = first_series[first_series != ""].value_counts(dropna=False)
    lin_df = vc.reset_index()
    if lin_df.shape[1] >= 2:
        lin_df.columns = ["First Ancestor","Count"]
    else:
        lin_df["First Ancestor"] = lin_df.index.astype(str)
        lin_df["Count"] = vc.values
        lin_df = lin_df[["First Ancestor","Count"]]
    lin_df = lin_df.sort_values(["Count","First Ancestor"], ascending=[False, True], kind="mergesort").reset_index(drop=True)

    html = []
    html.append(_shell_open("Lineage Count"))
    html.append('<table id="ref-table" border="1" class="sortable" style="width:100%"><thead><tr>')
    html.append('<th style="width:80%">First Ancestor</th><th style="width:20%">Count</th>')
    html.append('</tr></thead><tbody id="ref-tb">')
    for _, r in lin_df.iterrows():
        first = str(r.get("First Ancestor","")).strip()
        cnt   = int(str(r.get("Count","0")).strip() or "0")
        tr = (
            f'<tr data-q="{_html_escape_text(first)}" data-count="{cnt}">'
            f'<td>{_html.escape(first)}</td>'
            f'<td class="count"><a href="#" class="count-pick" title="Toggle select">{cnt}</a></td>'
            f'</tr>'
        )
        html.append(tr)
    html.append('</tbody></table>')
    html.append(_shell_close())
    return "".join(html)

def build_and_write_partials(main_df: pd.DataFrame):
    _setup_resolver()
    os.makedirs("partials", exist_ok=True)

    mc_html = build_match_count_partial(main_df)
    mc_local = os.path.join("partials", "match_count.htm")
    with open(mc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(mc_html)
    print("[OK] Wrote partial:", mc_local)

    lc_html = build_lineage_count_partial(main_df)
    lc_local = os.path.join("partials", "lineage_count.htm")
    with open(lc_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(lc_html)
    print("[OK] Wrote partial:", lc_local)

    cousin_df = main_df[["Match Summary"]].copy()
    cousin_df = cousin_df.sort_values(by="Match Summary", ascending=True, kind="mergesort").reset_index(drop=True)
    cousin_rows = ['<table border="1" id="refactor-table"><thead><tr><th>Match Summary</th></tr></thead><tbody>']
    for v in cousin_df["Match Summary"].tolist():
        cousin_rows.append(f"<tr><td>{v}</td></tr>")
    cousin_rows.append("</tbody></table>")
    cousin_html = (
        "<!DOCTYPE html PUBLIC \"-//W3C//DTD XHTML 1.0 Transitional//EN\" \"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\">"
        "<html xmlns=\"http://www.w3.org/1999/xhtml\" lang=\"en\"><head>"
        f"{HEAD_LINK}"
        "<meta http-equiv=\"Content-Type\" content=\"text/html; charset=iso-8859-15\" />"
        "<title>Cousin List (Printable)</title>"
        "<style type=\"text/css\"> body{font-size:12px;margin:20px;} h1{text-align:center;font-size:20px;} table{border-collapse:collapse;width:100%;} th,td{border:1px solid #999;padding:5px 7px;vertical-align:top;text-align:left;} th{background:#f0f0f0;} a{text-decoration:none;} </style>"
        "</head><body onload=\"window.print();\">"
        "<h1>Cousin List (Printable)</h1>" + "".join(cousin_rows) +
        "</body></html>"
    )
    cl_local = os.path.join("partials", "cousin_list_print.htm")
    with open(cl_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(cousin_html)
    print("[OK] Wrote partial:", cl_local)

    return mc_local, lc_local, cl_local

# Build partials + main page
PARTIAL_MATCH_LOCAL, PARTIAL_LINEAGE_LOCAL, PARTIAL_COUSIN_LOCAL = build_and_write_partials(df)

def build_register_html_for_abs(remote_abs_path: str) -> str:
    q_links = []
    subs = df["Subject"].astype(str).tolist()
    for subject_name in subs:
        q = _u.quote(subject_name)
        q_links.append(
            f'<a class="find-btn" href="{remote_abs_path}?q={q}" target="_blank" rel="noopener" '
            f'title="Open a filtered view for {subject_name}">Find</a>'
        )
    df_plus = df.copy()
    df_plus["Find"] = q_links
    disp_plus = df_plus[["Find","Match Summary", LINEAGE_HEADER_SAFE]]
    tbl = disp_plus.to_html(index=False, escape=False, classes="dataframe sortable")
    tbl = tbl.replace('<table border="1" class="dataframe sortable">', '<table border="1" class="dataframe sortable" id="refactor-table">', 1)
    tbl = tbl.replace('<tbody>\n<tr>', '<tbody>\n<tr id="first-row">', 1)
    tbl = tbl.replace("<th>Find</th>", '<th>Select:</th>', 1)
    tbl = tbl.replace("<th>Match Summary</th>", '<th>Match Summary&amp;ndash;click to sort</th>', 1)
    tbl = tbl.replace(f"<th>{_html.escape(LINEAGE_HEADER_SAFE)}</th>", "<th>Lineage (Starting with oldest ancestor&amp;ndash;click to sort)</th>", 1)
    colgroup_html_local = (
        "<colgroup>\n"
        f"  <col style=\"width:{FIND_COL_PX}px;\" />\n"
        f"  <col style=\"width:{COL_A_PX}px;\" />\n"
        "  <col />\n"
        "</colgroup>\n"
    )
    tbl = tbl.replace(
        '<table border="1" class="dataframe sortable" id="refactor-table">',
        '<table border="1" class="dataframe sortable" id="refactor-table">\n' + colgroup_html_local, 1
    )
    return page_tpl.safe_substitute(
        HEAD_LINK=HEAD_LINK,
        TABLE_CSS=TABLE_CSS,
        UPDATED_BLOCK=UPDATED_BLOCK,
        DYNAMIC_BLOCK=DYNAMIC_BLOCK,
        HTML_TABLE=tbl,
        JS_COUNT_URL=JS_COUNT_URL,
        DOWNLOADS_BLOCK=DOWNLOADS_BLOCK,
        HEAD_LINK_URL=HEAD_LINK_URL
    )

os.makedirs("partials", exist_ok=True)
final_html_plus = build_register_html_for_abs(HOME_URL)

with open(LOCAL_HTML, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html)
print("[OK] Saved canonical render:", os.path.abspath(LOCAL_HTML))

with open(WORK_PLUS_LOCAL, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(final_html_plus)
print("[OK] Saved:", os.path.abspath(WORK_PLUS_LOCAL), "(partials clone)")

# ---------- 9) Uploads ----------
def save_and_upload_all():
    if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
        print("[SKIP] Missing FTP creds; uploads skipped.")
        return
    try:
        ftps = ftp_connect()

        try:
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_CANON))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_LEG))
            ftp_upload_overwrite(ftps, LOCAL_HTML, _remote_path(REMOTE_HTML_SIMPLE))
        except Exception as e:
            print("[WARN] Upload main HTML failed:", e)

        try:
            if os.path.exists(LOCAL_CSV):
                ftp_upload_overwrite(ftps, LOCAL_CSV, _remote_path(REMOTE_CSV))
            if os.path.exists(LOCAL_XLSX):
                ftp_upload_overwrite(ftps, LOCAL_XLSX, _remote_path(REMOTE_XLSX))
            print("[OK] Uploaded CSV/XLSX -> /partials/")
        except Exception as e:
            print("[WARN] Upload CSV/XLSX failed:", e)

        if os.path.exists(LOCAL_COUNT_FILE):
            try:
                ftp_upload_overwrite(ftps, LOCAL_COUNT_FILE, _remote_path(REMOTE_COUNT_NAME))
            except Exception as e:
                print("[WARN] Upload autosomal count failed:", e)

        try:
            ftp_upload_overwrite(ftps, os.path.join("partials","match_count.htm"),       _remote_path(posixpath.join("partials","match_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","lineage_count.htm"),     _remote_path(posixpath.join("partials","lineage_count.htm")))
            ftp_upload_overwrite(ftps, os.path.join("partials","cousin_list_print.htm"), _remote_path(posixpath.join("partials","cousin_list_print.htm")))
        except Exception as e:
            print("[WARN] Upload partials failed:", e)

        try:
            ftp_upload_overwrite(ftps, WORK_PLUS_LOCAL, _remote_path(WORK_PLUS_REMOTE))
        except Exception as e:
            print("[WARN] Upload work_plus.htm failed:", e)

        print("\n--- SIZE Verification (if supported) ---")
        for p in [
            _remote_path(REMOTE_HTML_CANON),
            _remote_path(REMOTE_HTML_LEG),
            _remote_path(REMOTE_HTML_SIMPLE),
            _remote_path(REMOTE_CSV),
            _remote_path(REMOTE_XLSX),
            _remote_path(posixpath.join("partials","match_count.htm")),
            _remote_path(posixpath.join("partials","lineage_count.htm")),
            _remote_path(posixpath.join("partials","cousin_list_print.htm")),
            _remote_path(WORK_PLUS_REMOTE),
        ]:
            try:
                sz = ftp_size(ftps, p)
                print(f"{p} : {sz if sz is not None else '(SIZE unsupported)'}")
            except Exception:
                print(f"{p} : (check skipped)")

        try: ftps.quit()
        except Exception: pass

        print("\n--- Open URLs ---")
        print("Canonical (Find target):          https://yates.one-name.net/partials/yates_ancestor_register.htm")
        print("Legacy clone (same content):      https://yates.one-name.net/partials/ons_yates_dna_register.htm")
        print("JUSTDNA simple alias:             https://yates.one-name.net/partials/justdna.htm")
        print("Trees page:                       https://yates.one-name.net/partials/just-trees.htm")
        print("Match Count:                      https://yates.one-name.net/partials/match_count.htm")
        print("Lineage Count:                    https://yates.one-name.net/partials/lineage_count.htm")
        print("Cousin List (Printable):          https://yates.one-name.net/partials/cousin_list_print.htm")
        print("Work+ clone:                      https://yates.one-name.net/partials/work_plus.htm")

        print("\nIf anything looks cached, hard-refresh (Ctrl/Cmd+Shift+R) or append ?v=1 once.")
    except Exception as e:
        print("[FAIL] FTP session:", e)
        traceback.print_exc()

# ---------- 10) Upload ----------
save_and_upload_all()
# ====== CUT STOP [1/1] CELL 2 ===================================================================

[OK] Loaded CSV: 7 rows, 6 cols
[PULL] match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
[OK] Resolver loaded: 79 codes
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
[OK] Saved canonical render: /content/yates_ancestor_register.htm
[OK] Saved: /content/partials/work_plus.htm (partials clone)
[PUT] yates_ancestor_register.htm -> partials/yates_ancestor_register.htm
[PUT] yates_ancestor_register.htm -> partials/ons_yates_dna_register.htm
[PUT] yates_ancestor_register.htm -> partials/justdna.htm
[PUT] yates_ancestor_register.csv -> partials/yates_ancestor_register.csv
[PUT] yates_ancestor_register.xlsx -> partials/yates_ancestor_register.xlsx
[OK] Uploaded CSV/XLSX -> /partials/
[PUT] /content/autosomal_count.txt -> autosomal_count.txt
[PUT] partials/match_count.htm -> partials/match

# TEST Cell 3

In [ ]:
# ====== CUT START [1/6] RULES + IMPORTS + SECRETS ==============================================
# RON RULES — QUICK CODE CARD (v2025.11.05-AncReg-Exports+Partials-JustTrees-NavDNA)
# 1) EXECUTION: Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# 2) PUNCTUATION IN STRINGS: Use HTML entities (&rsquo; &ldquo; &rdquo; &mdash; &rarr;).
# 3) CONTENT: Deliver full runnable code (no snippets). No fabrication or inference.
# 4) Python code (inline, executable, full COLAB Cell paste-ready section); ISO-8859-15 (ASCII-only in source).
# 5) HTML: XHTML 1.0 Transitional style acceptable; avoid HTML5-only tags if not needed.
# 6) INTEGRITY: Work in CUT-ready sections only; exactly five # lines after each section.

# Core imports
import os, re, socket, posixpath, traceback
import pandas as pd
import html as _html

# Time imports (fix for ZoneInfo)
from datetime import datetime
try:
    from zoneinfo import ZoneInfo  # Python 3.9+
except Exception:
    ZoneInfo = None

# FTPS
from ftplib import FTP_TLS
# ====== CUT STOP  [1/6] RULES + IMPORTS + SECRETS ===============================================
#####
#####
#####
#####
#####



# ====== CUT START [2/6] SECRETS + LOAD DATA + COUNTS + PATHS ===================================
# --- Securely load secrets (Colab or env) ---
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_DIR  = os.environ.get('FTP_DIR', '').strip().strip('/')
COUNT_PUBLIC_URL = ("/%s/%s" % (FTP_DIR, "autosomal_count.txt")) if FTP_DIR else "/autosomal_count.txt"

# Inputs
INPUT_CSV   = "final_combined_df_with_value_labels.csv"

# OUTPUT html filename stays just-trees.htm
OUTPUT_NAME = "just-trees.htm"

# Button target (new nav button)
DNA_REGISTER_ABS = "https://yates.one-name.net/partials/justdna.htm"

# Export names for CSV/XLSX (mirror visible table); served from /partials/
EXPORT_BASE = "yates_ancestor_register"
LOCAL_CSV   = EXPORT_BASE + ".csv"
LOCAL_XLSX  = EXPORT_BASE + ".xlsx"
REMOTE_CSV  = posixpath.join("partials", LOCAL_CSV)
REMOTE_XLSX = posixpath.join("partials", LOCAL_XLSX)

# Upload the HTML itself into /partials/ using the OUTPUT_NAME
REMOTE_HTML = posixpath.join("partials", OUTPUT_NAME)

# Load CSV (robust encodings)
df = None
_last_err = None
for enc in ("utf-8-sig","utf-8","cp1252","iso-8859-15","latin1"):
    try:
        df = pd.read_csv(INPUT_CSV, dtype=str, keep_default_na=False, encoding=enc)
        break
    except Exception as e:
        _last_err = e
        df = None
if df is None:
    raise SystemExit("[ERROR] Unable to read CSV: %s (%r)" % (INPUT_CSV, _last_err))
print("[OK] Loaded CSV:", INPUT_CSV, "rows=%d, cols=%d" % (len(df), len(df.columns)))

# Normalize haplogroup column presence
if 'haplogroup' not in df.columns:
    df['haplogroup'] = ''
else:
    df['haplogroup'] = df['haplogroup'].fillna('')

# Read autosomal count locally if present (fallback display only)
autosomal_count = None
try:
    with open("autosomal_count.txt", "r") as f:
        autosomal_count = int(re.findall(r"(\d+)", f.read() or "")[0])
except Exception:
    pass

# Optional delta vs previous run
prev_count, additional_str = None, ""
if os.path.exists("autosomal_count_prev.txt"):
    try:
        with open("autosomal_count_prev.txt", "r") as f:
            prev_count = int(re.findall(r"(\d+)", f.read() or "")[0])
        if autosomal_count is not None and prev_count is not None:
            diff = autosomal_count - prev_count
            if diff != 0:
                additional_str = " (+%d since last run)" % diff
    except Exception:
        pass

# Human-readable fallback timestamp (JS will stamp dynamically at runtime)
try:
    _tz = ZoneInfo("America/New_York") if ZoneInfo else datetime.now().astimezone().tzinfo
except Exception:
    _tz = datetime.now().astimezone().tzinfo
now = datetime.now(_tz)
updated_fallback = now.strftime("%Y-%m-%d %H:%M")
# ====== CUT STOP  [2/6] SECRETS + LOAD DATA + COUNTS + PATHS ====================================
#####
#####
#####
#####
#####



# ====== CUT START [3/6] MAP COLUMN B (masked code) -> COLUMN C (unmasked name) =================
# Column letters in MAIN df:
#   A = ID#
#   B = match to (masked)
#   C = Unmasked Name (output)

A_IDX = 0
B_IDX = 1
C_IDX = 2

def _norm_code(s):
    t = str(s or "").strip()
    if (t.startswith('"') and t.endswith('"')) or (t.startswith("'") and t.endswith("'")):
        t = t[1:-1]
    t = t.replace("\u00a0", " ").strip()
    t = re.sub(r"\s{2,}", " ", t)
    return t.lower()

# Resolver lives on server under /partials/
REMOTE_PATH = "partials/match_to_unmasked.csv"
LOCAL_PATH  = "match_to_unmasked.csv"

# Pull resolver if not present
if not os.path.exists(LOCAL_PATH):
    print("Pulling resolver CSV from server...")
    with FTP_TLS(timeout=30) as ftps:
        ftps.connect(os.environ.get("FTP_HOST",""), int(os.environ.get("FTP_PORT","21")))
        ftps.login(os.environ.get("FTP_USER",""), os.environ.get("FTP_PASS",""))
        try: ftps.prot_p()
        except Exception: pass
        try: ftps.set_pasv(True)
        except Exception: pass
        if FTP_DIR:
            for p in FTP_DIR.split("/"):
                if not p: continue
                try: ftps.cwd(p)
                except Exception:
                    try: ftps.mkd(p)
                    except Exception: pass
                    ftps.cwd(p)
        try: ftps.cwd("partials")
        except Exception: pass
        with open(LOCAL_PATH, "wb") as f:
            ftps.retrbinary("RETR match_to_unmasked.csv", f.write)
    print("Resolver saved:", os.path.abspath(LOCAL_PATH))
else:
    print("Using cached resolver:", os.path.abspath(LOCAL_PATH))

def _load_resolver(path):
    last_err = None
    m = None
    for enc in ("utf-8-sig","iso-8859-15","utf-8","cp1252","latin1"):
        try:
            m = pd.read_csv(path, dtype=str, keep_default_na=False, encoding=enc)
            break
        except Exception as e:
            last_err = e
            m = None
    if m is None:
        raise RuntimeError("Unable to read resolver CSV: %s (%r)" % (path, last_err))
    cols = {c.lower(): c for c in m.columns}
    if "code" not in cols or "unmasked" not in cols:
        raise ValueError("Resolver CSV must have columns: code, unmasked")
    m = m[[cols["code"], cols["unmasked"]]].copy()
    m["__key__"] = m[cols["code"]].map(_norm_code)
    m["__val__"] = m[cols["unmasked"]].astype(str)
    m = m.drop_duplicates(subset="__key__", keep="first")
    return dict(zip(m["__key__"], m["__val__"]))

resolver_map = _load_resolver(LOCAL_PATH)

if df.shape[1] < 3:
    raise ValueError("Main df must have at least 3 columns: A(ID#), B(match to), C(unmasked).")

masked_raw = df.iloc[:, B_IDX].astype(str)
masked_key = masked_raw.map(_norm_code)
resolved   = masked_key.map(resolver_map)

df.iloc[:, C_IDX] = resolved.fillna("")

mapped = int(resolved.notna().sum())
total  = len(df)
print("[OK] Column B -> C mapping:", mapped, "/", total, "unmatched:", total - mapped)
# ====== CUT STOP  [3/6] MAP COLUMN B (masked code) -> COLUMN C (unmasked name) =================
#####
#####
#####
#####
#####



# ====== CUT START [4/6] XHTML TEMPLATE + TABLE + DOWNLOAD LINKS ================================
_BTN_BG   = "#5b79b8"
_BTN_BG_H = "#4668aa"
_TH_BG    = "#e3eaf8"
_LINK     = "#154b8b"

# Fallback number text for initial render; JS will overwrite with live values
auto_text = "Unknown" if autosomal_count is None else str(autosomal_count)

# Download links block points to /partials/{csv,xlsx}
DOWNLOADS_BLOCK = (
    "<p style=\"text-align:center; margin:4px 0 10px 0; font-size:13px;\">"
    "Download: "
    "<a href=\"/partials/%s\">CSV</a> | "
    "<a href=\"/partials/%s\">Excel</a>"
    "</p>" % (_html.escape(LOCAL_CSV), _html.escape(LOCAL_XLSX))
)

full_html_template = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
 "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<title>Ancestor Register</title>
<style type="text/css">
  html { scroll-behavior: smooth; }
  body { margin:0; padding:0; font-family: "Times New Roman", Georgia, serif; background:#ffffff; color:#222; font-size:14px; }
  a { color:%(LINK)s; text-decoration:none; } a:hover { text-decoration:underline; }

  .intro { padding:20px; text-align:center; }
  .intro h2 { margin:0 0 6px 0; font-size:24px; line-height:1.2; }
  .meta { font-size:12px; color:#555; margin:4px 0 8px 0; display:inline-block; }

  .toolbar { margin:6px auto 12px auto; display:flex; flex-wrap:wrap; gap:6px; justify-content:center; }
  .btn { display:inline-block; border:1px solid %(BTN_BG)s; background:%(BTN_BG)s; color:#fff;
         padding:4px 9px; border-radius:6px; font-size:13px; line-height:1.2; text-decoration:none;
         cursor:pointer; user-select:none; transition:background 0.2s, transform 0.1s; }
  .btn:hover { background:%(BTN_BG_H)s; transform:translateY(-1px); }
  .btn.light { background:#ffffff; color:#111; border-color:#bbb; }

  .output-table { max-height:75vh; overflow:auto; border:1px solid #ddd; margin:0 20px 24px 20px; }

  table.sortable { width:100%%; border-collapse:collapse; min-width:720px; table-layout:auto; }
  table.sortable th, table.sortable td { border:1px solid #ddd; padding:6px 8px; background:#ffffff; white-space:nowrap; }
  table.sortable th { position:sticky; top:0; z-index:2; text-align:left; cursor:pointer; background:%(TH_BG)s; box-shadow:0 1px 0 #ccc; }
  table.sortable tr#first-row td { border-top:2px solid #999 !important; }

  #searchBox { padding:4px 8px; font-size:13px; border:1px solid #bbb; border-radius:6px; outline:none; }

  .back-to-top { position:fixed; right:16px; bottom:16px; padding:6px 10px;
                 border:1px solid %(BTN_BG)s; background:%(BTN_BG)s; color:#fff;
                 border-radius:6px; font-size:12px; display:none; z-index:9999; cursor:pointer; }
  .back-to-top:hover { background:%(BTN_BG_H)s; }

  @media screen and (max-width: 820px) {
    .intro { padding:14px; }
    .output-table { margin:0 12px 20px 12px; }
    .intro h2 { font-size:20px; }
    table.sortable { min-width:560px; }
  }
</style>
<script type="text/javascript">
//<![CDATA[
// Basic helpers
function _cellText(cell){
  var t = (cell && (cell.textContent || cell.innerText) || '').replace(/\\s+/g,' ').replace(/^\\s+|\\s+$/g,'').toLowerCase();
  return t;
}
function _asNumber(s){
  var m = (s||'').replace(/[^0-9.\\-]/g,'');
  if(m.length===0) return NaN;
  var v = parseFloat(m);
  return isNaN(v) ? NaN : v;
}

// Sorting
function sortTableByColumn(tbl, colIndex, dirAsc){
  if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var tb = tbl.tBodies[0];
  var rows = Array.prototype.slice.call(tb.rows || []);
  rows.sort(function(a,b){
    var A = _cellText(a.cells[colIndex] || null);
       var B = _cellText(b.cells[colIndex] || null);
    var nA = _asNumber(A), nB = _asNumber(B);
    if(!isNaN(nA) && !isNaN(nB)){ return dirAsc ? (nA - nB) : (nB - nA); }
    if(A < B) return dirAsc ? -1 : 1;
    if(A > B) return dirAsc ?  1 : -1;
    return 0;
  });
  var frag = document.createDocumentFragment();
  for(var i=0;i<rows.length;i++){ frag.appendChild(rows[i]); }
  tb.appendChild(frag);
  updateShowingCount();
}
function bindHeaderSort(){
  var tbl = document.getElementById('refactor-table');
  if(!(tbl && tbl.tHead && tbl.tHead.rows.length)) return;
  var ths = tbl.tHead.rows[0].cells || [];
  for(var i=0;i<ths.length;i++){
    (function(idx){
      var th = ths[idx];
      var dirAsc = true;
      th.addEventListener('click', function(){
        for(var j=0;j<ths.length;j++){ ths[j].innerHTML = ths[j].innerHTML.replace(/\\s+[\\u2191\\u2193]$/, ''); }
        sortTableByColumn(tbl, idx, dirAsc);
        th.innerHTML = th.innerHTML.replace(/\\s+[\\u2191\\u2193]$/, '') + (dirAsc ? ' \\u2191' : ' \\u2193');
        dirAsc = !dirAsc;
      }, false);
    })(i);
  }
}

// Filter + live showing count
function filterTable(){
  var q = (document.getElementById('searchBox').value || '').toLowerCase();
  var tbl = document.getElementById('refactor-table');
  if(!(tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var rows = tbl.tBodies[0].rows || [];
  for(var i=0;i<rows.length;i++){
    var cells = rows[i].cells, hit=false;
    for(var j=0;j<cells.length;j++){
      var txt = (cells[j].textContent || cells[j].innerText || '').toLowerCase();
      if(txt.indexOf(q) > -1){ hit=true; break; }
    }
    rows[i].style.display = hit ? '' : 'none';
  }
  updateShowingCount();
}
function updateShowingCount(){
  var el = document.getElementById('showing-count');
  var tbl = document.getElementById('refactor-table');
  if(!(el && tbl && tbl.tBodies && tbl.tBodies[0])) return;
  var rows = tbl.tBodies[0].rows || [];
  var vis = 0;
  for(var i=0;i<rows.length;i++){ if(rows[i].style.display !== 'none') vis++; }
  el.textContent = vis;
}

// Dynamic stamps
function z(n){ return (n<10 ? '0' : '') + n; }
function stampLastUpdated(){
  var el = document.getElementById('last-updated'); if(!el) return;
  var d  = new Date(document.lastModified || new Date());
  var opts = {
    year: 'numeric',
    month: 'long',
    day: 'numeric',
    hour: '2-digit',
    minute: '2-digit',
    hour12: false
  };
  var formatted = d.toLocaleString('en-US', opts).replace(',', '');
  el.innerHTML = formatted;
}


function formatWithCommas(n){
  try{ var x=parseInt(String(n||'').replace(/[^0-9\\-]/g,''),10); if(isNaN(x)) return ''; return x.toLocaleString('en-US'); }catch(e){ return String(n||''); }
}
function loadAutoCount(){
  var el=document.getElementById('auto-count'); if(!el) return;
  var url='{COUNT_URL}';
  try{
    var xhr=new XMLHttpRequest(); xhr.open('GET', url+(url.indexOf('?')>-1?'':'?v='+(new Date()).getTime()), true);
    xhr.onreadystatechange=function(){ if(xhr.readyState===4){
      if(xhr.status>=200&&xhr.status<300){
        var m=(xhr.responseText||'').match(/(\\d+)/); var num=m?m[1]:'';
        el.textContent = formatWithCommas(num) || '(unavailable)';
      } else { el.textContent='(unavailable)'; }
    }};
    xhr.send(null);
  }catch(e){ el.textContent='(unavailable)'; }
}

// Back-to-top
function bindBackToTop(){
  var btn = document.getElementById('back-to-top');
  if(!btn) return;
  function toggle(){ btn.style.display = (window.scrollY > 200) ? 'block' : 'none'; }
  toggle(); window.addEventListener('scroll', toggle, {passive:true});
  btn.addEventListener('click', function(){ window.scrollTo(0,0); }, false);
}

document.addEventListener('DOMContentLoaded', function(){
  bindHeaderSort();
  bindBackToTop();
  stampLastUpdated();
  loadAutoCount();
  updateShowingCount();
}, false);
//]]>
</script>
</head>
<body>
  <div class="intro">
    <h2>Ancestor Register</h2>
    <div class="meta">
      Last updated: <span id="last-updated">%(UPDATED)s</span>
      &nbsp;|&nbsp; Autosomal matches: <span id="auto-count">%(AUTO)s</span>
      &nbsp;|&nbsp; Showing: <span id="showing-count">0</span>
    </div>
    %(DL)s
    <div class="toolbar">
      <a class="btn" href="%(DNA_ABS)s" target="_blank" rel="noopener">DNA Register</a>
      <input type="text" id="searchBox" class="btn light" placeholder="Search this page&hellip;" oninput="filterTable()" />
    </div>
  </div>

  <div class="output-table" id="table-container">
    <!-- TABLE_PLACEHOLDER -->
  </div>

  <div class="back-to-top" id="back-to-top">&#9650; Top</div>
</body>
</html>""" % {
    "BTN_BG": _BTN_BG, "BTN_BG_H": _BTN_BG_H, "TH_BG": _TH_BG, "LINK": _LINK,
    "UPDATED": _html.escape(updated_fallback),
    "AUTO": _html.escape("Unknown" if autosomal_count is None else str(autosomal_count)),
    "DL": DOWNLOADS_BLOCK,
    "DNA_ABS": DNA_REGISTER_ABS
}

# Build table HTML and mark first row
table_html = df.to_html(index=False, border=1, classes="sortable", table_id="refactor-table")
table_html = table_html.replace("<tbody>\n<tr>", "<tbody>\n<tr id=\"first-row\">", 1)

# Inject table and JS count URL
final_html = full_html_template.replace("<!-- TABLE_PLACEHOLDER -->", table_html)
final_html = final_html.replace("{COUNT_URL}", COUNT_PUBLIC_URL)

# Build export DataFrame mirroring the visible table order (use current df as-is)
export_df = df.copy()

# Save CSV (ISO-8859-15) and XLSX
export_df.to_csv(LOCAL_CSV, index=False, encoding="iso-8859-15")
try:
    export_df.to_excel(LOCAL_XLSX, index=False)
except Exception:
    with pd.ExcelWriter(LOCAL_XLSX) as _writer:
        export_df.to_excel(_writer, index=False)
print("[OK] Wrote exports:", os.path.abspath(LOCAL_CSV), "and", os.path.abspath(LOCAL_XLSX))
# ====== CUT STOP  [4/6] XHTML TEMPLATE + TABLE + DOWNLOAD LINKS ================================
#####
#####
#####
#####
#####



# ====== CUT START [5/6] SAVE LOCALLY + FTP UPLOAD (HTML + CSV/XLSX -> /partials) ==============
# Save locally (iso-8859-15 safe)
try:
    with open(OUTPUT_NAME, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        f.write(final_html)
    print("[OK] Saved locally:", OUTPUT_NAME)
except Exception as e:
    print("[ERROR] Saving local file failed:", e)
    traceback.print_exc()

# Upload if credentials exist
ftp_host = os.environ.get('FTP_HOST')
ftp_user = os.environ.get('FTP_USER')
ftp_pass = os.environ.get('FTP_PASS')
ftp_port = os.environ.get('FTP_PORT', '21')
ftp_dir  = os.environ.get('FTP_DIR', '')

def _ftps_ensure_dir(ftps, name):
    if not name: return
    for p in [p for p in name.split('/') if p]:
        try:
            ftps.cwd(p)
        except Exception:
            try:
                ftps.mkd(p)
            except Exception:
                pass
            ftps.cwd(p)

if all([ftp_host, ftp_user, ftp_pass]):
    print("[INFO] Attempting FTP upload...")
    try:
        socket.setdefaulttimeout(30)
        with FTP_TLS(timeout=30) as ftps:
            ftps.connect(ftp_host, int(ftp_port))
            ftps.login(ftp_user, ftp_pass)
            try: ftps.prot_p()
            except Exception: pass
            try: ftps.set_pasv(True)
            except Exception: pass

            # Navigate to base dir
            _ftps_ensure_dir(ftps, ftp_dir.strip('/'))

            # Ensure /partials exists then upload HTML + CSV/XLSX there
            try:
                _ftps_ensure_dir(ftps, "partials")

                # Upload HTML into /partials/ as just-trees.htm
                with open(OUTPUT_NAME, "rb") as fh:
                    ftps.storbinary("STOR " + os.path.basename(REMOTE_HTML), fh)
                print("[OK] Uploaded HTML to /partials/:", OUTPUT_NAME, "->", os.path.basename(REMOTE_HTML))

                # Upload CSV/XLSX into /partials/
                with open(LOCAL_CSV, "rb") as fh:
                    ftps.storbinary("STOR " + os.path.basename(REMOTE_CSV), fh)
                with open(LOCAL_XLSX, "rb") as fh:
                    ftps.storbinary("STOR " + os.path.basename(REMOTE_XLSX), fh)
                print("[OK] Uploaded exports to /partials/:", LOCAL_CSV, LOCAL_XLSX)

                print("Open URL: https://yates.one-name.net/partials/just-trees.htm")
            except Exception as e:
                print("[ERROR] Upload to /partials/ failed:", e)

            print("[OK] Uploads complete.")
    except Exception as e:
        print("[ERROR] FTP session failed:", e)
        traceback.print_exc()
else:
    print("[INFO] Skipping FTP upload: missing FTP credentials.")
# ====== CUT STOP  [5/6] SAVE LOCALLY + FTP UPLOAD (HTML + CSV/XLSX -> /partials) ==============
#####
#####
#####
#####
#####



# ====== CUT START [6/6] PERSIST COUNT + DONE ====================================================
if autosomal_count is not None:
    try:
        with open("autosomal_count_prev.txt", "w") as f:
            f.write(str(autosomal_count))
        print("[OK] Persisted autosomal_count_prev.txt")
    except Exception as e:
        print("[WARN] Could not persist autosomal count:", e)

print("\n--- Ancestor Register Build + Exports Complete (HTML now at /partials/just-trees.htm; nav button -> justdna.htm) ---")
# ====== CUT STOP  [6/6] PERSIST COUNT + DONE ====================================================
#####
#####
#####
#####
#####



[OK] Loaded CSV: final_combined_df_with_value_labels.csv rows=7, cols=6
Using cached resolver: /content/match_to_unmasked.csv
[OK] Column B -> C mapping: 7 / 7 unmatched: 0
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Saved locally: just-trees.htm
[INFO] Attempting FTP upload...
[OK] Uploaded HTML to /partials/: just-trees.htm -> just-trees.htm
[OK] Uploaded exports to /partials/: yates_ancestor_register.csv yates_ancestor_register.xlsx
Open URL: https://yates.one-name.net/partials/just-trees.htm
[OK] Uploads complete.
[OK] Persisted autosomal_count_prev.txt

--- Ancestor Register Build + Exports Complete (HTML now at /partials/just-trees.htm; nav button -> justdna.htm) ---


Connected. PWD: /

--- Directory listing: web root (/) ---
drwxr-x---   23 onenamen   65534            4096 Nov  5 19:04 .
drwxr-x---   23 onenamen   65534            4096 Nov  5 19:04 ..
-rw-------    1 onenamen   onenamen           19 Oct 19 02:45 .ftpquota
-rw-r--r--    1 onenamen   onenamen          514 Jul 20 07:46 .htaccess
drwxr-xr-x    3 onenamen   onenamen         4096 Jan 23  2023 .well-known
-rw-r--r--    1 onenamen   onenamen       770679 May 25 01:55 DNA_Cousin_Surname_App.htm
drwxr-xr-x    5 onenamen   onenamen         4096 Nov  5 18:43 admin
-rw-r--r--    1 onenamen   onenamen            4 Nov  5 18:59 autosomal_count.txt
drwxr-xr-x    5 onenamen   onenamen         4096 Nov  5 05:44 blog
drwxr-xr-x    2 onenamen   onenamen         4096 Jan 23  2023 cgi-bin
drwxr-xr-x    4 onenamen   onenamen         4096 Mar 16  2023 content-yatesdb
-rw-r--r--    1 onenamen   onenamen         8620 Apr 25  2023 contribute.htm
drwxr-xr-x   12 onenamen   onenamen         4096 Mar 30  2023 c

In [ ]:
# ====== CUT START [1/1] CANONICAL RESET — Root wrapper -> partials (XHTML, FTPS upload) =========
# RON RULES — QUICK CODE CARD (v2025.11.05-Canonical-Reset)
# - Complete & runnable in Colab; ISO-8859-15 (ASCII-only in source).
# - No fabrication; no JSON/triple-quote artifacts beyond this single cell.

import os, socket, posixpath, traceback
from ftplib import FTP_TLS

# ---------- 0) Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try: os.environ['FTP_DIR'] = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
    try: os.environ['FTP_PORT'] = userdata.get('FTP_PORT') or '21'
    except Exception: os.environ.setdefault('FTP_PORT', '21')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_DIR', '')
    os.environ.setdefault('FTP_PORT', '21')

# ---------- 1) Config (edit if needed) ----------
# The true content lives here (ABSOLUTE URL in partials/)
PARTIALS_TARGET_ABS = "https://yates.one-name.net/partials/yates_ancestor_register.htm"

# Restore the old canonical "home" filename in the web root:
CANONICAL_HOME_NAME = "ons_yates_dna_register.htm"   # this will be created at site root

# Optionally refresh /index.html to point at the canonical home (set True to write/overwrite)
WRITE_INDEX_HTML = True

# If writing index.html, should it point to "canonical" or "partials"?
INDEX_TARGET_MODE = "canonical"  # options: "canonical" or "partials"

FTP_DIR      = (os.environ.get("FTP_DIR") or "").strip()
FTP_TIMEOUT  = int(os.environ.get("FTP_TIMEOUT") or "30")
FTP_PASSIVE  = True

def _remote_path(basename: str) -> str:
    return posixpath.join(FTP_DIR, basename) if FTP_DIR else basename

def ftp_connect() -> FTP_TLS:
    ftps = FTP_TLS(timeout=FTP_TIMEOUT)
    socket.setdefaulttimeout(FTP_TIMEOUT)
    ftps.connect(os.environ.get('FTP_HOST',''), int(os.environ.get('FTP_PORT','21')))
    ftps.login(os.environ.get('FTP_USER',''), os.environ.get('FTP_PASS',''))
    try: ftps.prot_p()
    except Exception: pass
    try: ftps.set_pasv(FTP_PASSIVE)
    except Exception: pass
    if FTP_DIR:
        try:
            ftps.cwd(FTP_DIR)
        except Exception:
            # Create path parts if missing
            parts = [p for p in FTP_DIR.split("/") if p]
            for p in parts:
                try: ftps.mkd(p)
                except Exception: pass
                ftps.cwd(p)
    return ftps

def ftp_upload_overwrite(ftps: FTP_TLS, local_path: str, remote_name: str):
    with open(local_path, "rb") as fh:
        ftps.storbinary(f"STOR {remote_name}", fh)
    print("Uploaded:", local_path, "->", remote_name)

# ---------- 2) Build XHTML redirect wrappers ----------
def _xhtml_redirect_page(title: str, target_abs_url: str, also_canonical: bool = True) -> str:
    # Old-school, XHTML 1.0 Transitional; includes <meta refresh>, JS replace, and clickable fallback link
    link_canon = f'\n<link rel="canonical" href="{target_abs_url}" />' if also_canonical else ''
    return (
        '<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" '
        '"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n'
        '<html xmlns="http://www.w3.org/1999/xhtml" lang="en">\n<head>\n'
        '<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-15" />\n'
        f'<title>{title}</title>\n'
        f'<meta http-equiv="refresh" content="0; url={target_abs_url}" />{link_canon}\n'
        '<style type="text/css">body{font-family:\'Times New Roman\',Georgia,serif;margin:40px;}</style>\n'
        '</head>\n<body>\n'
        f'<p>If you are not redirected, <a href="{target_abs_url}">click here</a>.</p>\n'
        f'<script type="text/javascript">\n//<![CDATA[\n'
        f'try{{ window.location.replace("{target_abs_url}"); }}catch(e){{ window.location.href="{target_abs_url}"; }}\n'
        f'//]]>\n</script>\n'
        '</body>\n</html>\n'
    )

# File 1: canonical home wrapper -> partials
canonical_local = CANONICAL_HOME_NAME
with open(canonical_local, "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
    f.write(_xhtml_redirect_page("ONS Yates DNA Register (Canonical Home)", PARTIALS_TARGET_ABS, True))
print("[OK] Wrote:", os.path.abspath(canonical_local))
canonical_remote = _remote_path(CANONICAL_HOME_NAME)

# File 2 (optional): index.html -> canonical or partials
index_local = None
index_remote = None
if WRITE_INDEX_HTML:
    index_target = PARTIALS_TARGET_ABS if INDEX_TARGET_MODE == "partials" else ("./" + CANONICAL_HOME_NAME)
    # If we point to "./ons_yates_dna_register.htm", browsers stay in root then hop to partials via the canonical page.
    with open("index.html", "w", encoding="iso-8859-15", errors="xmlcharrefreplace") as f:
        # canonical link here is not added to avoid confusion; the canonical tag exists in the canonical page above
        f.write(_xhtml_redirect_page("Home → ONS Yates DNA Register", index_target, False))
    index_local = "index.html"
    index_remote = _remote_path("index.html")
    print("[OK] Wrote:", os.path.abspath(index_local), "target:", index_target)

# ---------- 3) Upload ----------
if not all(os.environ.get(k) for k in ['FTP_HOST','FTP_USER','FTP_PASS']):
    print("Missing FTP creds; skipping uploads.")
else:
    try:
        ftps = ftp_connect()
        ftp_upload_overwrite(ftps, canonical_local, canonical_remote)
        if index_local and index_remote:
            ftp_upload_overwrite(ftps, index_local, index_remote)
        try: ftps.quit()
        except Exception: pass
        print("\n=== SUMMARY ===")
        print("Canonical target set to:", "/" + CANONICAL_HOME_NAME)
        print("Canonical page redirects to:", PARTIALS_TARGET_ABS)
        if index_remote:
            print("Home (/index.html) now points to:", ("canonical" if INDEX_TARGET_MODE=="canonical" else "partials"))
        else:
            print("Home (/index.html) unchanged (WRITE_INDEX_HTML=False).")
    except Exception as e:
        print("FTP session failed:", e)
        traceback.print_exc()

# ====== CUT STOP [1/1] CANONICAL RESET ==========================================================


[OK] Wrote: /content/ons_yates_dna_register.htm
[OK] Wrote: /content/index.html target: ./ons_yates_dna_register.htm
Uploaded: ons_yates_dna_register.htm -> ons_yates_dna_register.htm
Uploaded: index.html -> index.html

=== SUMMARY ===
Canonical target set to: /ons_yates_dna_register.htm
Canonical page redirects to: https://yates.one-name.net/partials/yates_ancestor_register.htm
Home (/index.html) now points to: canonical


# debug

In [63]:
# ====== CUT START [1/1] UPDATE /partials/dna_tree_styles.css — FIXED PX WIDTHS (1–4) ============
# RON RULES — QUICK CODE CARD (v2025.11.08-ColumnWidths-PX)
# - Complete & runnable Colab cell; ISO-8859-15 (ASCII-only source).
# - Adds explicit pixel widths for columns:
#     1 → 75px   ("Select")
#     2 → 250px  (main name / match)
#     3 → 40px
#     4 → 50px
# - Leaves centering, padding, and other layout rules unchanged.

import os, re, socket, time, traceback
from ftplib import FTP_TLS

# ---------- Secrets ----------
try:
    from google.colab import userdata  # type: ignore
    os.environ['FTP_HOST'] = userdata.get('FTP_HOST')
    os.environ['FTP_USER'] = userdata.get('FTP_USER')
    os.environ['FTP_PASS'] = userdata.get('FTP_PASS')
    try:  os.environ['FTP_PORT'] = userdata.get('FTP_PORT')
    except Exception: os.environ.setdefault('FTP_PORT', '21')
    try:  os.environ['FTP_DIR']  = userdata.get('FTP_DIR')
    except Exception: os.environ.setdefault('FTP_DIR', '')
except Exception:
    os.environ.setdefault('FTP_HOST', '')
    os.environ.setdefault('FTP_USER', '')
    os.environ.setdefault('FTP_PASS', '')
    os.environ.setdefault('FTP_PORT', '21')
    os.environ.setdefault('FTP_DIR', '')

FTP_HOST = os.environ.get('FTP_HOST','')
FTP_USER = os.environ.get('FTP_USER','')
FTP_PASS = os.environ.get('FTP_PASS','')
FTP_PORT = int(os.environ.get('FTP_PORT','21'))
FTP_DIR  = (os.environ.get('FTP_DIR','') or '').strip('/')

TARGET_DIR  = 'partials'
TARGET_NAME = 'dna_tree_styles.css'
LOCAL_ORIG  = 'dna_tree_styles.orig.css'
LOCAL_NEW   = 'dna_tree_styles.css'

if not all([FTP_HOST, FTP_USER, FTP_PASS]):
    raise SystemExit("[EXIT] Missing FTP creds in Colab userdata: FTP_HOST/FTP_USER/FTP_PASS")

# ---------- FTP helpers ----------
def _connect():
    socket.setdefaulttimeout(30)
    ftps = FTP_TLS(timeout=30)
    ftps.connect(FTP_HOST, FTP_PORT)
    ftps.login(FTP_USER, FTP_PASS)
    try: ftps.prot_p()
    except Exception: pass
    ftps.set_pasv(True)
    if FTP_DIR:
        for p in [p for p in FTP_DIR.split('/') if p]:
            try: ftps.cwd(p)
            except Exception:
                try: ftps.mkd(p)
                except Exception: pass
                ftps.cwd(p)
    return ftps

def _cwd(ftps, path):
    for p in [q for q in path.split('/') if q]:
        try: ftps.cwd(p)
        except Exception:
            try: ftps.mkd(p)
            except Exception: pass
            ftps.cwd(p)

def _download(ftps, remote_name, local_name):
    with open(local_name, 'wb') as f:
        ftps.retrbinary('RETR ' + remote_name, f.write)
    print("[OK] Downloaded:", remote_name)

def _upload(ftps, local_name, remote_name):
    with open(local_name, 'rb') as f:
        ftps.storbinary('STOR ' + remote_name, f)
    print("[OK] Uploaded:", local_name, "->", remote_name)

def _server_backup(ftps, remote_name):
    ts = time.strftime("%Y%m%d%H%M")
    bak = remote_name + "." + ts + ".bak"
    try:
        with open("__tmp_bak.css", "wb") as tmp:
            ftps.retrbinary('RETR ' + remote_name, tmp.write)
        with open("__tmp_bak.css", "rb") as tmp:
            ftps.storbinary('STOR ' + bak, tmp)
        os.remove("__tmp_bak.css")
        print("[OK] Server backup created:", bak)
    except Exception as e:
        print("[WARN] Backup skipped:", e)

# ---------- CSS injection ----------
def _inject_fixed_widths(css_text):
    rule = (
        "\n/* === Explicit fixed column widths (v2025.11.08) === */\n"
        "table { table-layout: fixed; width: 100%; }\n"
        "table th:first-child, table td:first-child { width: 75px !important; max-width: 75px !important; }\n"
        "table th:nth-child(2), table td:nth-child(2) { width: 250px !important; max-width: 250px !important; }\n"
        "table th:nth-child(3), table td:nth-child(3) { width: 400px !important;  max-width: 40px !important; }\n"
        "table th:nth-child(4), table td:nth-child(4) { width: 50px !important;  max-width: 50px !important; }\n"
    )
    if "Explicit fixed column widths" not in css_text:
        css_text += "\n" + rule
    else:
        css_text = re.sub(r'/\* === Explicit fixed column widths[^*]+\*/.*?(?=/\*|$)', rule, css_text, flags=re.S)
    return css_text

# ---------- Main ----------
try:
    ftps = _connect()
    _cwd(ftps, TARGET_DIR)

    _download(ftps, TARGET_NAME, LOCAL_ORIG)
    with open(LOCAL_ORIG, 'r', encoding='utf-8', errors='ignore') as f:
        css = f.read()

    new_css = _inject_fixed_widths(css)
    with open(LOCAL_NEW, 'w', encoding='iso-8859-15', errors='xmlcharrefreplace') as f:
        f.write(new_css)

    _server_backup(ftps, TARGET_NAME)
    _upload(ftps, LOCAL_NEW, TARGET_NAME)
    ftps.quit()

    print("\nDONE. Columns 1–4 now have fixed widths: 75, 250, 40, 50 px.  Refresh with Ctrl+F5 or ?v=1.")
except Exception as e:
    print("[ERROR]", e)
    traceback.print_exc()
# ====== CUT STOP  [1/1] UPDATE /partials/dna_tree_styles.css — FIXED PX WIDTHS (1–4) ============


[OK] Downloaded: dna_tree_styles.css
[OK] Server backup created: dna_tree_styles.css.202511081739.bak
[OK] Uploaded: dna_tree_styles.css -> dna_tree_styles.css

DONE. Columns 1–4 now have fixed widths: 75, 250, 40, 50 px.  Refresh with Ctrl+F5 or ?v=1.


[OK] Loaded CSV: 7 rows, 6 cols
[PULL] match_to_unmasked.csv -> /content/match_to_unmasked.server.csv
[OK] Resolver loaded: 79 codes
[OK] Wrote exports: /content/yates_ancestor_register.csv and /content/yates_ancestor_register.xlsx
[OK] Wrote partial: partials/match_count.htm
[OK] Wrote partial: partials/lineage_count.htm
[OK] Wrote partial: partials/cousin_list_print.htm
[OK] Saved canonical render: /content/yates_ancestor_register.htm
[OK] Saved: /content/partials/work_plus.htm (partials clone)
[PUT]  yates_ancestor_register.htm -> partials/yates_ancestor_register.htm
[PUT]  yates_ancestor_register.htm -> partials/ons_yates_dna_register.htm
[PUT]  yates_ancestor_register.htm -> partials/justdna.htm
[PUT]  yates_ancestor_register.csv -> partials/yates_ancestor_register.csv
[PUT]  yates_ancestor_register.xlsx -> partials/yates_ancestor_register.xlsx
[OK] Uploaded CSV/XLSX -> /partials/
[PUT]  /content/autosomal_count.txt -> autosomal_count.txt
[PUT]  partials/match_count.htm -> partial